#SinergIA Cobros

In [58]:
!pip -q install openpyxl==3.1.5 google-api-python-client google-auth-httplib2 google-auth-oauthlib

##Conexión y carga del sistema (0000-0100)

In [224]:
#SinergIA0020

# Celda A1 — Autenticación (Colab OAuth) + cliente gspread  · sin JSON
from google.colab import auth
import gspread
from google.auth import default as google_default

# Autenticación del usuario de Colab
auth.authenticate_user()

# Credenciales por defecto del entorno (mismo usuario)
creds, _ = google_default()

# Cliente de Google Sheets autorizado
gc = gspread.authorize(creds)

# Config del archivo de parámetros (hardcodeada)
CONFIG_SPREADSHEET_ID = "1q0yWNo35lPeNXmdOgMRKvYx1TcHgxJhPPDo4zXq9p_0"
CONFIG_SHEET_NAME = "SettingID"

In [225]:
#SinergIA0040

# Bloque A — Helpers de Drive y lectura segura (usar tras Autorización + MapaID)
import io, os, csv, warnings
from typing import List, Tuple, Dict, Optional
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.auth import default as google_default
import traceback

warnings.filterwarnings("ignore")

# Crea cliente de Drive con las mismas credenciales de Colab/Sheets (sin JSON)
_creds, _ = google_default()
_drive = build("drive", "v3", credentials=_creds, cache_discovery=False)

def drive_list_recent(folder_id: str, patterns=(".csv",".xlsx",".xls"), top: int = 10) -> List[Dict[str,str]]:
    """
    Esta función lista hasta 'top' archivos recientes dentro de una carpeta de Drive por ID.
    Retorna lista de dicts: {id, name, mimeType, modifiedTime, size}.
    """
    q = f"'{folder_id}' in parents and trashed = false"
    fields = "files(id,name,mimeType,modifiedTime,size),nextPageToken"
    items: List[Dict[str,str]] = []
    page_token = None
    while True:
        resp = _drive.files().list(q=q, fields=fields, orderBy="modifiedTime desc",
                                   pageSize=min(50, top), pageToken=page_token).execute()
        for f in resp.get("files", []):
            name_low = f.get("name","").lower()
            if any(name_low.endswith(p) for p in patterns):
                items.append(f)
                if len(items) >= top:
                    return items
        page_token = resp.get("nextPageToken")
        if not page_token: break
    return items


def drive_download_file(file_id: str) -> Tuple[str, bytes]:
    """
    Esta función descarga un archivo de Drive por file_id y devuelve (nombre, bytes).
    """
    meta = _drive.files().get(fileId=file_id, fields="name").execute()
    name = meta["name"]
    req = _drive.files().get_media(fileId=file_id)
    buf = io.BytesIO()
    downloader = MediaIoBaseDownload(buf, req)
    done = False
    while not done:
        status, done = downloader.next_chunk()
    buf.seek(0)
    return name, buf.read()

def read_preview_any(src: Tuple[str, bytes] or str, nrows: int = 200) -> Tuple[List[str], List[List[str]]]:
    """
    Esta función genera una vista previa de CSV/XLSX.
    - Si 'src' es ruta (str), lee desde disco.
    - Si 'src' es (nombre, bytes), lee desde memoria.
    - Usa pandas si está disponible; si falla, cae a csv/openpyxl.
    Retorna: (cols, rows) donde rows es lista de listas.
    """
    def _pandas_available() -> bool:
        try:
            import pandas as pd  # noqa
            return True
        except Exception:
            return False

    # Resolver extensión + stream
    if isinstance(src, str):
        name = os.path.basename(src)
        ext = os.path.splitext(name)[1].lower()
        path = src
        data = None
    else:
        name, data = src  # (nombre, bytes)
        ext = os.path.splitext(name)[1].lower()
        path = None

    # Intentar pandas primero (si existe y no rompe)
    if _pandas_available():
        try:
            import pandas as pd
            if ext == ".csv":
                if path:
                    df = pd.read_csv(path, nrows=nrows)
                else:
                    df = pd.read_csv(io.BytesIO(data), nrows=nrows)
            else:
                if path:
                    df = pd.read_excel(path, nrows=nrows)
                else:
                    df = pd.read_excel(io.BytesIO(data), nrows=nrows)
            cols = [str(c) for c in df.columns]
            rows = df.fillna("").astype(str).values.tolist()
            return cols, rows
        except Exception:
            pass  # fallback

    # Fallback sin pandas
    if ext == ".csv":
        if data is not None:
            txt = data.decode("utf-8", errors="replace").splitlines()
            rdr = csv.reader(txt)
        else:
            rdr = csv.reader(open(path, "r", newline="", encoding="utf-8"))
        cols = next(rdr, [])
        rows = []
        for i, row in enumerate(rdr):
            if i >= nrows: break
            rows.append([row[j] if j < len(cols) else "" for j in range(len(cols))])
        return [str(c) for c in cols], rows

    # XLSX via openpyxl
    try:
        import openpyxl
    except Exception as e:
        raise RuntimeError("Falta 'openpyxl'. Ejecuta:  !pip -q install openpyxl==3.1.5") from e

    wb = openpyxl.load_workbook(io.BytesIO(data) if data is not None else path, read_only=True, data_only=True)
    ws = wb.worksheets[0]
    it = ws.iter_rows(values_only=True)
    header = next(it, None)
    if header is None:
        wb.close(); return [], []
    cols = [str(c) if c is not None else f"col_{i}" for i, c in enumerate(header)]
    rows = []
    for i, row in enumerate(it):
        if i >= nrows: break
        rows.append([("" if row[j] is None else row[j]) for j in range(len(cols))])
    wb.close()
    return cols, rows

In [226]:
#SinergIA0060
#Lectura desde Drive (usa la lógica del test #SinergIA066)

import time
import pandas as pd

def _leer_df_desde_drive(tipo, folder_id, archivo, log, nrows=5000):
    """
    Lee un archivo (xlsx/csv) desde Drive y devuelve (df, log).
    - Si folder_id viene vacío, escoge el folder según el tipo.
    - Limita la lectura a nrows filas (por defecto 5000) para que no reviente la GUI.
    """
    # Si no llega folder_id, usamos el mapeo por tipo
    if not folder_id:
        mapping = {
            "Rutina": RUTINAS_ID,
            "AWS": AWS_LOG_ID,
            "Promesas": PROMESAS_ID,
            "Pagos": PAGOS_ID,
        }
        folder_id = mapping.get(tipo, RUTINAS_ID)

    try:
        # Buscar archivo por nombre
        fid, meta = drive_find_by_name(folder_id, archivo)
        if not fid:
            log = _log_append(log, f"No se encontró '{archivo}' en la carpeta seleccionada.")
            return None, log

        # Descargar bytes (igual que en tu test)
        t0 = time.time()
        name, data = drive_download_file(fid)
        log = _log_append(
            log,
            f"Descargado '{name}' en {time.time()-t0:0.2f}s · bytes={len(data)}."
        )

        # Leer preview (igual que en tu test)
        t0 = time.time()
        cols, rows = read_preview_any((name, data), nrows=min(int(nrows), 5000))
        log = _log_append(
            log,
            f"Preview leído en {time.time()-t0:0.2f}s · filas={len(rows)}."
        )

        if not cols:
            log = _log_append(log, "El archivo no tiene columnas detectables.")
            return None, log

        df = pd.DataFrame(rows, columns=[str(c) for c in cols])
        return df, log

    except Exception as e:
        import traceback
        log = _log_append(log, f"Error leyendo desde Drive: {e}")
        # opcional: log += "\n" + traceback.format_exc()[-600:]
        return None, log


In [227]:
#SinergIA0080

# === Drive helpers (bytes) — sin archivos locales ===
import io
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from google.auth import default as google_default

def drive_find_by_name(folder_id: str, name: str):
    """Busca un archivo por nombre exacto dentro de una carpeta (no imprime)."""
    creds, _ = google_default()
    svc = build("drive", "v3", credentials=creds)
    q = f"'{folder_id}' in parents and name = '{name}' and trashed = false"
    resp = svc.files().list(q=q, fields="files(id,name)", pageSize=1).execute()
    files = resp.get("files", [])
    return (files[0]["id"], files[0]["name"]) if files else (None, None)

def drive_upload_bytes(folder_id: str, name: str, mime: str, data_bytes: bytes) -> str:
    """Crea/sobrescribe por nombre en carpeta destino, desde memoria."""
    creds, _ = google_default()
    svc = build("drive", "v3", credentials=creds)
    # si ya existe con ese nombre, lo dejamos coexistir (versionado) o podrías borrarlo antes si deseas
    media = MediaIoBaseUpload(io.BytesIO(data_bytes), mimetype=mime, resumable=False)
    meta = {"name": name, "parents": [folder_id]}
    f = svc.files().create(body=meta, media_body=media, fields="id").execute()
    return f["id"]

In [228]:
#SinergIA0100

# ================================
# F1 — Núcleo de pipeline (v1)
# Builders: Entrada_AWS, Teletón, Listas
# Heurística: Ley2300 + ÁrbolHeurístico
# Requiere: gc, CONFIG_SPREADSHEET_ID, get_column_names_from_estructuras
# ================================
from typing import Dict, List, Optional, Tuple
import datetime as _dt

def _norm_cols(df):
    """Normaliza encabezados a snake_case simple para reducir errores por espacios/mayúsculas."""
    import pandas as pd
    if df is None or df is ...:
        return df
    df = df.copy()
    df.columns = (
        pd.Index(df.columns)
          .map(lambda x: str(x).strip())
          .map(lambda x: x.replace(" ", "_").replace("-", "_"))
          .str.replace(r"[^0-9A-Za-z_]+", "", regex=True)
    )
    return df

def _only_digits(x) -> str:
    """Devuelve solo dígitos de una cadena (útil para teléfonos/identificaciones)."""
    import re
    s = "" if x is None else str(x)
    return re.sub(r"\D+", "", s)

def _parse_date(x):
    """Parsea fechas variadas a YYYY-MM-DD; si falla, devuelve None."""
    import pandas as pd
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    try:
        dt = pd.to_datetime(x, dayfirst=False, errors="coerce")
        if pd.isna(dt):  # intento día/mes
            dt = pd.to_datetime(x, dayfirst=True, errors="coerce")
        if pd.isna(dt):
            return None
        return dt.date().isoformat()
    except Exception:
        return None

def l2300_default_params() -> dict:
    """Parámetros por defecto si no existen en Drive."""
    return {
        "recontact_days": 4,
        "wk_start": "07:00", "wk_end": "19:00",
        "sat_start": "08:00", "sat_end": "15:00",
        "allow_sun": False,
        "one_channel_per_week": True,
        "cooldown_by_result": {"RPC":72,"PROMESA":48,"BUZON":8,"NO_CONTESTA":4,"OCUPADO":2},
        "holidays": [],
    }

def l2300_get_params() -> dict:
    """Obtiene parámetros de Ley2300 desde Drive si hay funciones; si no, usa defaults."""
    try:
        return l2300_load_params()
    except Exception:
        return l2300_default_params()

def _hhmm_to_minutes(hhmm: str) -> int:
    """Convierte 'HH:MM' a minutos desde 00:00."""
    try:
        hh, mm = map(int, hhmm.split(":"))
        return hh*60 + mm
    except Exception:
        return 0

def l2300_is_time_allowed(now: Optional[_dt.datetime] = None, params: Optional[dict] = None) -> bool:
    """Verifica si la hora actual cae dentro de las ventanas legales (L–V, Sáb, Dom opcional)."""
    params = params or l2300_get_params()
    now = now or _dt.datetime.now()
    wd = now.weekday()  # 0=lunes
    mins = now.hour*60 + now.minute

    def in_range(lo, hi):
        return _hhmm_to_minutes(lo) <= mins <= _hhmm_to_minutes(hi)

    # Domingo
    if wd == 6:
        return bool(params.get("allow_sun", False))
    # Sábado
    if wd == 5:
        return in_range(params.get("sat_start","08:00"), params.get("sat_end","15:00"))
    # L–V
    return in_range(params.get("wk_start","07:00"), params.get("wk_end","19:00"))

def l2300_filter_recontact(df, params: Optional[dict] = None, fecha_col_candidates: Optional[List[str]] = None) -> "pd.DataFrame":
    """
    Aplica la regla 'recontact_days' (en días desde la última gestión).
    Si no encuentra la fecha de última gestión, no filtra por este criterio.
    """
    import pandas as pd
    params = params or l2300_get_params()
    recontact = int(params.get("recontact_days", 4))
    if df is None or df.empty:
        return df
    fecha_cols = fecha_col_candidates or ["UltimaGestion", "FechaUltGestion", "FechaUltimoContacto", "Ultima_Visita"]
    df = df.copy()

    def _coerce_dt(s):
        try:
            out = pd.to_datetime(s, errors="coerce")
        except Exception:
            out = pd.NaT
        return out

    last_col = None
    for c in df.columns:
        if c in fecha_cols:
            last_col = c
            break
    if last_col is None:
        # no hay columna clara de última gestión → retorno sin filtrar
        return df

    last_dt = _coerce_dt(df[last_col])
    now = pd.Timestamp.now()
    days = (now - last_dt).dt.days
    mask = (last_dt.isna()) | (days >= recontact)
    return df[mask].reset_index(drop=True)

def arbol_load_df() -> "pd.DataFrame":
    """Carga la hoja 'ArbolHeuristico' del gSheet de configuración."""
    import pandas as pd
    try:
        sh = gc.open_by_key(CONFIG_SPREADSHEET_ID)
        ws = sh.worksheet("ArbolHeuristico")
        df = pd.DataFrame(ws.get_all_records())
        return _norm_cols(df)
    except Exception:
        return pd.DataFrame()

def arbol_apply_scoring(df: "pd.DataFrame", arbol_df: "pd.DataFrame") -> "pd.DataFrame":
    """
    Aplica un scoring simple basado en reglas de ÁrbolHeurístico:
    Columnas esperadas (nombres flexibles tras normalización): campo, operador, valor, peso, accion.
    Operadores soportados: ==, !=, >, >=, <, <=, contains, in
    """
    import numpy as np
    import pandas as pd

    if df is None or df.empty or arbol_df is None or arbol_df.empty:
        out = df.copy()
        out["_score_rules"] = 0.0
        return out

    out = df.copy()
    out["_score_rules"] = 0.0

    # Normalizar nombres base del árbol
    cols = {c: c.lower() for c in arbol_df.columns}
    A = arbol_df.rename(columns=cols)
    # Heuristic mapping
    col_campo    = next((c for c in A.columns if c in ("campo","field","columna")), None)
    col_oper     = next((c for c in A.columns if c in ("operador","op")), None)
    col_valor    = next((c for c in A.columns if c in ("valor","value")), None)
    col_peso     = next((c for c in A.columns if c in ("peso","weight","puntaje")), None)
    col_accion   = next((c for c in A.columns if c in ("accion","action","tipo")), None)

    if not (col_campo and col_oper and col_valor and col_peso):
        return out  # estructura no reconocida → sin cambios

    for _, r in A.iterrows():
        campo = str(r.get(col_campo, "")).strip()
        oper  = str(r.get(col_oper, "")).strip().lower()
        valor = r.get(col_valor, "")
        try:
            peso  = float(r.get(col_peso, 0) or 0)
        except Exception:
            peso = 0.0
        accion = str(r.get(col_accion, "")).strip().lower()

        if not campo or campo not in out.columns or peso == 0:
            continue

        series = out[campo]
        m = None

        # Comparadores
        if oper in ("==", "eq"):
            m = (series.astype(str) == str(valor))
        elif oper in ("!=", "ne"):
            m = (series.astype(str) != str(valor))
        elif oper in (">", "gt"):
            m = pd.to_numeric(series, errors="coerce") > pd.to_numeric(valor, errors="coerce")
        elif oper in (">=", "ge"):
            m = pd.to_numeric(series, errors="coerce") >= pd.to_numeric(valor, errors="coerce")
        elif oper in ("<", "lt"):
            m = pd.to_numeric(series, errors="coerce") < pd.to_numeric(valor, errors="coerce")
        elif oper in ("<=", "le"):
            m = pd.to_numeric(series, errors="coerce") <= pd.to_numeric(valor, errors="coerce")
        elif oper in ("contains",):
            m = series.astype(str).str.contains(str(valor), case=False, na=False)
        elif oper in ("in",):
            vals = [v.strip() for v in str(valor).split("|")]
            m = series.astype(str).isin(vals)

        if m is None:
            continue

        sign = -1.0 if accion in ("penaliza","penalize","penalty","-") else 1.0
        out.loc[m, "_score_rules"] = out.loc[m, "_score_rules"] + sign * peso

    return out

def _derive_ani(df: "pd.DataFrame") -> "pd.Series":
    """Elige un teléfono móvil candidato a ANI según columnas comunes."""
    import pandas as pd
    if df is None or df.empty:
        return pd.Series([], dtype="object")
    cand_cols = [c for c in df.columns if c.lower() in ("celular","movil","telefono","telefono1","telefono2","tel1","tel2","ani")]
    if not cand_cols:
        return pd.Series([""]*len(df), dtype="object")

    def _pick(row):
        for c in cand_cols:
            num = _only_digits(row.get(c))
            if len(num) >= 7:
                return num
        return ""
    return df.apply(_pick, axis=1)

def _make_numeric(s):
    """Convierte a numérico tolerante (quita símbolos) para saldo/mora, etc."""
    import pandas as pd, numpy as np, re
    if s is None:
        return pd.Series([], dtype="float64")
    if hasattr(s, "apply"):
        def _coerce(x):
            if x is None:
                return np.nan
            xs = re.sub(r"[^0-9\.\-]+", "", str(x))
            try:
                return float(xs) if xs not in ("", ".", "-") else np.nan
            except Exception:
                return np.nan
        return s.apply(_coerce)
    return s

def build_entrada_aws(df_rutina, df_aws=None, df_promesas=None, df_pagos=None) -> "pd.DataFrame":
    """
    Construye la base **Entrada_AWS**:
    - Parte de la Rutina/IN (TER/IN) ya previsualizada.
    - Limpia strings, normaliza fechas y derive 'ANI' (móvil candidato).
    - Integra señales de Promesas/Pagos si están disponibles (banderas/resumen).
    - Aplica scoring por ÁrbolHeurístico y ordena por score total (reglas + señales básicas).
    - **No** incluye la columna Infinivirt (eso es del Teletón).
    """
    import pandas as pd, numpy as np

    if df_rutina is None or df_rutina.empty:
        return pd.DataFrame()

    df = _norm_cols(df_rutina)

    # Limpieza básica (trim strings)
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].astype(str).str.strip()

    # Identificación y ANI
    if "identificacion" not in df.columns and "documento" in df.columns:
        df["identificacion"] = df["documento"]
    df["ANI"] = _derive_ani(df)

    # Fechas usuarias a ISO
    for col in [c for c in df.columns if "fecha" in c.lower() or "gestion" in c.lower()]:
        df[col] = df[col].apply(_parse_date)

    # Señales básicas (saldo/mora/contactabilidad si existen)
    if "saldo" in df.columns:
        df["_saldo_num"] = _make_numeric(df["saldo"])
    else:
        df["_saldo_num"] = np.nan
    if "diasmora" in df.columns:
        df["_diasmora_num"] = _make_numeric(df["diasmora"])
    else:
        df["_diasmora_num"] = np.nan
    if "contactabilidad" in df.columns:
        df["_contact_num"] = _make_numeric(df["contactabilidad"]).fillna(0)
    else:
        df["_contact_num"] = 0.0

    # Árbol heurístico
    arbol = arbol_load_df()
    df = arbol_apply_scoring(df, arbol)

    # Score total sencillo (ajusta pesos rápido; reemplazable por tu “Árbol real”)
    df["_score_total"] = (
        df["_score_rules"].fillna(0)
        + df["_contact_num"].fillna(0) * 0.6
        + df["_saldo_num"].pow(0.25).replace([np.inf,-np.inf], np.nan).fillna(0) * 0.2
        + df["_diasmora_num"].rpow(0.25).replace([np.inf,-np.inf], np.nan).fillna(0) * 0.2
    )

    # Reglas base de recontacto (Ley2300 · días)
    df = l2300_filter_recontact(df)

    # Orden final (estable)
    df = df.sort_values(["_score_total"], ascending=False, kind="mergesort").reset_index(drop=True)

    # Ordenar columnas: primero claves/campaña
    prefer = [c for c in ["identificacion","upper","ani"] if c in df.columns]
    others = [c for c in df.columns if c not in prefer]
    df = df[prefer + others]
    return df

def build_teleton(df_entrada: "pd.DataFrame") -> "pd.DataFrame":
    """
    Construye el **Teletón** a partir de Entrada_AWS:
    - Selecciona columnas clave (ID/Upper/ANI/score si existen).
    - Añade columna **Infinivirt** vacía (para diligenciar en operación).
    - No reordena por Infinivirt aquí; ese efecto se aplica al priorizar listas.
    """
    import pandas as pd
    if df_entrada is None or df_entrada.empty:
        return pd.DataFrame()
    cols_base = [c for c in ["identificacion","upper","ani","_score_total"] if c in df_entrada.columns]
    if not cols_base:
        cols_base = list(df_entrada.columns)[:8]
    tel = df_entrada[cols_base].copy()
    tel["Infinivirt"] = ""  # diligenciable en operación/UX
    return tel

def build_listas_por_canal(df_entrada: "pd.DataFrame", params: Optional[dict] = None) -> Dict[str, "pd.DataFrame"]:
    """
    Deriva listas por canal aplicando una capa mínima de legalidad (hora actual) y disponibilidad de ANI.
    - Voz: requiere 'ANI' (móvil).
    - WhatsApp/SMS: reutilizan 'ANI' si no hay campos específicos.
    - Aplica filtro horario de Ley2300 (si no permitido ahora, devuelve vacío).
    """
    import pandas as pd
    params = params or l2300_get_params()
    allowed_now = l2300_is_time_allowed(params=params)

    def _safe(df):
        return pd.DataFrame() if df is None else df.copy()

    base = _safe(df_entrada)
    if base.empty:
        return {"Voz": base, "WhatsApp": base, "SMS": base}

    # Filtro horario (si no permitido, devolvemos estructura vacía — o podríamos marcar 'bloqueado')
    if not allowed_now:
        return {"Voz": base.iloc[0:0], "WhatsApp": base.iloc[0:0], "SMS": base.iloc[0:0]}

    # Voz: requiere ANI
    voz = base[base["ani"].astype(str).str.len() >= 7] if "ani" in base.columns else base.iloc[0:0]
    # WhatsApp/SMS: para v1 usamos también ANI; luego podremos ampliar por columnas específicas
    wsp = voz.copy()
    sms = voz.copy()

    return {"Voz": voz.reset_index(drop=True),
            "WhatsApp": wsp.reset_index(drop=True),
            "SMS": sms.reset_index(drop=True)}

def df_to_bytes(df, fmt: str) -> Tuple[str, bytes, str]:
    """
    Convierte un DataFrame a bytes según formato:
    - csv_comma / csv_semicolon / xlsx
    Devuelve (extensión_sin_punto, bytes, mime).
    """
    import pandas as pd, io
    if fmt == "xlsx":
        bio = io.BytesIO()
        with pd.ExcelWriter(bio, engine="xlsxwriter") as xw:
            df.to_excel(xw, index=False, sheet_name="data")
        return "xlsx", bio.getvalue(), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    sep = "," if fmt == "csv_comma" else ";"
    data = df.to_csv(index=False, sep=sep, line_terminator="\n").encode("utf-8")
    return "csv", data, "text/csv"

##Helpers & Configs (0120-0229)

In [229]:
#SinergIA0120
#Lectura desde Drive (usa la misma lógica del test)

import pandas as pd
import time

def _folder_for_tipo(tipo: str) -> str:
    """
    Si ya tienes esta función definida en otro bloque, NO la dupliques.
    En ese caso borra esta definición y deja solo el resto.
    """
    mapping = {
        "Rutina": RUTINAS_ID,
        "AWS": AWS_LOG_ID,
        "Promesas": PROMESAS_ID,
        "Pagos": PAGOS_ID,
    }
    return mapping.get(tipo, RUTINAS_ID)

def _leer_df_desde_drive(tipo, folder_id, archivo, log, nrows=5000):
    """
    Lee un archivo (xlsx/csv) desde Drive y devuelve (df, log).
    Lógica basada en el test #SinergIA066 que ya comprobaste que funciona.
    """
    folder_id = folder_id or _folder_for_tipo(tipo)
    try:
        # Buscar el archivo por nombre
        fid, meta = drive_find_by_name(folder_id, archivo)
        if not fid:
            log = _log_append(log, f"No se encontró '{archivo}' en la carpeta seleccionada.")
            return None, log

        # Descargar bytes
        t0 = time.time()
        name, data = drive_download_file(fid)
        log = _log_append(
            log,
            f"Descargado '{name}' en {time.time()-t0:0.2f}s · bytes={len(data)}."
        )

        # Leer preview (idéntico a tu prueba)
        t0 = time.time()
        cols, rows = read_preview_any((name, data), nrows=min(int(nrows), 5000))
        log = _log_append(
            log,
            f"Preview leído en {time.time()-t0:0.2f}s · filas={len(rows)}."
        )

        if not cols:
            log = _log_append(log, "El archivo no tiene columnas detectables.")
            return None, log

        df = pd.DataFrame(rows, columns=[str(c) for c in cols])
        return df, log

    except Exception as e:
        import traceback
        log = _log_append(log, f"Error leyendo desde Drive: {e}")
        # opcional: log += "\n" + traceback.format_exc()[-600:]
        return None, log


In [230]:
#SinergIA0140

# === Helpers de exportación: convertir a CSV/XLSX (sin pandas) ===
import io, csv, tempfile
from typing import Tuple, List
from googleapiclient.http import MediaFileUpload

def xlsx_bytes_to_rows(xlsx_bytes: bytes) -> Tuple[List[str], List[List[object]]]:
    """
    Lee la primera hoja de un XLSX y devuelve (cols, rows).
    """
    import openpyxl
    wb = openpyxl.load_workbook(io.BytesIO(xlsx_bytes), read_only=True, data_only=True)
    ws = wb.worksheets[0]
    it = ws.iter_rows(values_only=True)
    header = next(it, None) or []
    cols = [str(c) if c is not None else f"col_{i}" for i, c in enumerate(header)]
    rows = []
    for r in it:
        rows.append(["" if r[j] is None else r[j] for j in range(len(cols))])
    wb.close()
    return cols, rows

def table_to_csv_bytes(cols: List[str], rows: List[List[object]], sep: str = ",") -> bytes:
    """
    Escribe (cols, rows) a CSV en memoria con separador configurable.
    """
    buf = io.StringIO()
    w = csv.writer(buf, lineterminator="\n", delimiter=sep)
    w.writerow(cols)
    for r in rows:
        w.writerow(r)
    return buf.getvalue().encode("utf-8")

def table_to_xlsx_bytes(cols: List[str], rows: List[List[object]]) -> bytes:
    """
    Escribe (cols, rows) a XLSX en memoria (primera hoja).
    """
    import openpyxl
    from openpyxl.workbook import Workbook
    wb: Workbook = openpyxl.Workbook()
    ws = wb.active
    ws.append(cols)
    for r in rows:
        ws.append(r)
    out = io.BytesIO()
    wb.save(out)
    return out.getvalue()

def convert_primary_to_csv_full(file_id: str) -> Tuple[str, bytes, Tuple[List[str], List[List[object]]]]:
    """
    Descarga un primario de Drive (por id).  Si es XLSX lo convierte a CSV;
    si ya es CSV, lo normaliza.  Sube el CSV a 'Salidas' y retorna:
      (file_id_subido, csv_bytes, (cols, rows))
    """
    name, data = drive_download_file(file_id)
    base, ext = name.rsplit(".", 1)
    ext = ext.lower()

    # Normalizamos a (cols, rows)
    if ext == "csv":
        # Leer csv de origen a tabla
        s = io.StringIO(data.decode("utf-8"))
        rdr = csv.reader(s)
        cols = next(rdr, [])
        rows = [row for row in rdr]
    else:
        cols, rows = xlsx_bytes_to_rows(data)

    # CSV bytes (separador coma) para subir a Drive
    csv_bytes = table_to_csv_bytes(cols, rows, sep=",")

    # Subida por streaming a Salidas
    with tempfile.NamedTemporaryFile("wb", suffix=".csv", delete=False) as tmp:
        tmp.write(csv_bytes)
        tmp_path = tmp.name

    media = MediaFileUpload(tmp_path, mimetype="text/csv", resumable=False)
    meta = {"name": f"{base}.csv", "parents": [SALIDAS_ID]}
    created = _drive.files().create(body=meta, media_body=media, fields="id").execute()
    return created["id"], csv_bytes, (cols, rows)

In [231]:
#SinergIA0160

# A+ — Upload a Drive: crear archivo a partir de bytes
from googleapiclient.http import MediaIoBaseUpload

def drive_upload_bytes(folder_id: str, name: str, data: bytes, mime_type: str = "text/csv") -> str:
    """
    Sube un archivo a Google Drive (carpeta 'folder_id') usando bytes en memoria.
    Retorna el file_id del archivo creado.
    """
    file_metadata = {"name": name, "parents": [folder_id]}
    media = MediaIoBaseUpload(io.BytesIO(data), mimetype=mime_type, resumable=False)
    created = _drive.files().create(body=file_metadata, media_body=media, fields="id").execute()
    return created["id"]

In [232]:
#SinergIA0180
# Celda A2 — MapaID: lee 'SettingID' y construye FOLDERS + constantes de IDs
from typing import Dict, Any

def _norm_key(k: str) -> str:
    return str(k).strip().replace(" ", "_").replace("-", "_").lower()

def load_folder_map() -> Dict[str, str]:
    """
    Lee la hoja 'SettingID' del gSheet de configuración y construye
    un mapa NombreCarpeta → ID (ambos como str).
    - Sin pandas; usa gspread.get_all_records().
    - Tolera ligeras variaciones de encabezados (normaliza claves).
    """
    sh = gc.open_by_key(CONFIG_SPREADSHEET_ID)
    ws = sh.worksheet(CONFIG_SHEET_NAME)
    records = ws.get_all_records()

    rows = [{_norm_key(k): v for k, v in r.items()} for r in records]
    if not rows:
        raise RuntimeError("La hoja 'SettingID' está vacía o no es accesible.")

    # Detectar columnas 'nombre carpeta' e 'id'
    sample_keys = list(rows[0].keys())
    name_col = None
    id_col = None
    for c in sample_keys:
        if "nombre" in c and "carp" in c:
            name_col = c; break
    name_col = name_col or "nombre_carpeta"

    for c in sample_keys:
        if c == "id" or "folder" in c or "carpeta_id" in c:
            id_col = c; break
    id_col = id_col or "id"

    fm = {}
    for r in rows:
        name = str(r.get(name_col, "")).strip()
        fid  = str(r.get(id_col, "")).strip()
        if name and fid:
            fm[name] = fid

    if not fm:
        raise RuntimeError("No se encontraron pares válidos 'Nombre carpeta' y 'ID' en 'SettingID'.")
    return fm

FOLDERS = load_folder_map()

def _req(name: str) -> str:
    """Obtiene una carpeta requerida o lanza error con mensaje claro."""
    if name not in FOLDERS:
        raise KeyError(f"Falta la carpeta requerida en SettingID: '{name}'.  Verifique la hoja de configuración.")
    return FOLDERS[name]

# === Constantes de carpeta (usar exactamente estos nombres en SettingID) ===
SINERGIA_ROOT_ID              = _req("SinergIA_Cobros")
ENTRADA_DATOS_ID              = _req("EntradaDatos")
RUTINAS_ID                    = _req("Rutinas")
PROMESAS_ID                   = _req("Promesas")
PAGOS_ID                      = _req("Pagos")
GENESYS_ID                    = _req("Genesys")
DIGITAL_ID                    = _req("Digital")
AWS_LOG_ID                    = _req("AWS_Log")
TER_ID                        = _req("TER")
PROCESAMIENTO_ID              = _req("Procesamiento")
CONSOLIDADO_MEJORGEST_ID      = _req("Consolidado_MejorGest")
MODELOS_ID                    = _req("Modelos")
SALIDAS_ID                    = _req("Salidas")
LISTAS_TRABAJO_GENESYS_ID     = _req("Listas_Trabajo_Genesys")
REPORTES_ANALISIS_ID          = _req("Reportes_Analisis")
DOCUMENTACION_ID              = _req("Documentacion")
ESTRUCTURAS_COLUMNAS_ID       = _req("Estructuras_Columnas")
REQUERIMIENTOS_CHATS_ID       = _req("Requerimientos_y_Chats")

print(f"OK · FOLDERS={len(FOLDERS)}  · RUTINAS_ID={RUTINAS_ID}")

OK · FOLDERS=18  · RUTINAS_ID=1y69gb2Y73GslfEOu13tfCzx1lZv7ggv-


In [233]:
#SinergIA0200
# Skin + layout para la UI mínima (sin gradio). Reusa los widgets ya creados.
import ipywidgets as W
from IPython.display import display, HTML

def render_sinergia_styled_app(
    origen_tb, folder_id_t, refresh_b, files_dd, upload_w, preview_b, log_out, table_out
):
    """
    Esta función ‘viste’ la UI de ipywidgets con un layout tipo panel izquierdo/derecho,
    botones redondeados y tarjetas, sin depender de Gradio ni pandas.
    - No crea lógica nueva; solo compone y estiliza los widgets ya definidos.
    - Puede llamarse varias veces; cada llamada vuelve a pintar el contenedor.
    """
    # Estilos inyectados (ligeros, compatibles con Colab)
    display(HTML("""
    <style>
      .kiara-wrap { padding: 8px 10px; }
      .kiara-card { border:1px solid #444; border-radius:16px; padding:12px; box-shadow: 0 2px 10px rgba(0,0,0,.15); }
      .kiara-title { font-size: 20px; font-weight:600; margin: 0 0 6px; }
      .kiara-sub { color:#9aa0a6; font-size:12px; margin-bottom:8px; }
      .kiara-section { margin-top: 10px; }
    </style>
    """))

    # “Temas” para botones
    refresh_b.style.button_color = "#2563eb"   # azul
    preview_b.style.button_color = "#16a34a"   # verde
    refresh_b.layout.width = "140px"
    preview_b.layout.width = "140px"

    # Columnas
    left_box = W.VBox(
        [
            W.HTML("<div class='kiara-title'>Cargar archivos primarios</div>"),
            W.HTML("<div class='kiara-sub'>Rutina · IN_SISTECREDITO</div>"),
            origen_tb,
            W.HBox([folder_id_t, refresh_b]),
            files_dd,
            upload_w,
            W.HBox([preview_b]),
        ],
        layout=W.Layout(width="360px")
    )

    right_box = W.VBox(
        [
            W.HTML("<div class='kiara-title'>Log</div>"),
            log_out,
            W.HTML("<div class='kiara-title kiara-section'>Preview</div>"),
            table_out,
        ],
        layout=W.Layout(flex="1 1 auto")
    )

    container = W.HBox([left_box, right_box], layout=W.Layout(width="100%"))
    frame = W.VBox([W.HTML("<div class='kiara-title'>SinergIA Cobros</div>"), container],
                   layout=W.Layout(width="100%"))
    display(W.HTML("<div class='kiara-wrap kiara-card'>"))
    display(frame)
    display(HTML("</div>"))

# Lanzador opcional y seguro (no rompe si los widgets antiguos no existen)
try:
    render_sinergia_styled_app(origen_tb, folder_id_t, refresh_b, files_dd, upload_w, preview_b, log_out, table_out)
except NameError:
    print("UI estilo Kiara: esperando widgets del Bloque B (antiguo). Puedes ignorar este aviso.")

UI estilo Kiara: esperando widgets del Bloque B (antiguo). Puedes ignorar este aviso.


In [234]:
#SinergIA0220

# C1 — convertir archivo primario completo a CSV y subirlo a 'Salidas'
import io, csv
from googleapiclient.http import MediaFileUpload
import tempfile

def xlsx_bytes_to_csv_bytes(xlsx_bytes: bytes) -> bytes:
    """Convierte la 1ra hoja de un .xlsx a CSV (valores, sin fórmulas)."""
    import openpyxl
    wb = openpyxl.load_workbook(io.BytesIO(xlsx_bytes), read_only=True, data_only=True)
    ws = wb.worksheets[0]
    buf = io.StringIO()
    w = csv.writer(buf, lineterminator="\n")
    for row in ws.iter_rows(values_only=True):
        w.writerow(["" if v is None else v for v in row])
    wb.close()
    return buf.getvalue().encode("utf-8")

def drive_upload_path(folder_id: str, local_path: str, mime_type: str = "text/csv") -> str:
    """Sube un archivo local a Drive usando streaming desde disco."""
    media = MediaFileUpload(local_path, mimetype=mime_type, resumable=False)
    meta = {"name": local_path.split("/")[-1], "parents": [folder_id]}
    created = _drive.files().create(body=meta, media_body=media, fields="id").execute()
    return created["id"]

def convert_primary_to_csv_full(file_id: str, salidas_folder_id: str = SALIDAS_ID) -> str:
    """
    Descarga un primario de Drive por file_id.  Si es .xlsx, lo convierte completo a CSV.
    Si ya es .csv, solo lo normaliza a UTF-8.  Sube el CSV resultante a 'Salidas' y retorna su file_id.
    """
    name, data = drive_download_file(file_id)
    base, ext = name.rsplit(".", 1)
    ext = ext.lower()

    if ext == "csv":
        csv_bytes = data  # ya viene en CSV
        out_name = f"{base}.csv"
    else:
        csv_bytes = xlsx_bytes_to_csv_bytes(data)
        out_name = f"{base}.csv"

    # guarda temporal y sube por streaming (robusto para archivos grandes)
    with tempfile.NamedTemporaryFile("wb", suffix=".csv", delete=False) as tmp:
        tmp.write(csv_bytes)
        tmp_path = tmp.name

    fid = drive_upload_path(salidas_folder_id, tmp_path, "text/csv")
    return fid

##Ley2300 (0230-0599)

In [235]:
#SinergIA0230

# Ley2300 — filtro legal + learning básico de mejor hora (sin pandas)
import csv, io, datetime as dt

def _parse_hhmm(s: str) -> tuple:
    try:
        h, m = s.split(":"); return int(h), int(m)
    except Exception:
        return 0, 0

def _in_range(now: dt.datetime, h_ini: str, h_fin: str) -> bool:
    hi, mi = _parse_hhmm(h_ini); hf, mf = _parse_hhmm(h_fin)
    t = now.time()
    return (dt.time(hi, mi) <= t <= dt.time(hf, mf))

def _to_dt(s):
    if not s: return None
    s = str(s).strip()
    for fmt in ("%Y-%m-%d %H:%M:%S", "%Y-%m-%dT%H:%M:%S", "%d/%m/%Y %H:%M", "%Y-%m-%d"):
        try: return dt.datetime.strptime(s, fmt)
        except Exception: continue
    return None

def ley2300_filter_rows(cols, rows, params: dict, canal_destino: str = "voz") -> list:
    """
    Aplica reglas Ley2300 sobre 'rows' (lista de listas) según 'cols' (encabezados).
    Reglas: 1/día, ventana horaria, un canal/semana, cooldown por resultado, DNC.
    Columnas opcionales: dnc, ult_contacto_ts, canal_usado_semana, consent_horario_extendido,
                         ventana_autorizada_ini, ventana_autorizada_fin, ult_resultado.
    Devuelve lista filtrada.
    """
    def _idx(name):
        name = name.lower()
        try: return [c.lower() for c in cols].index(name)
        except ValueError: return -1

    ix_dnc   = _idx("dnc")
    ix_ucts  = _idx("ult_contacto_ts")
    ix_csem  = _idx("canal_usado_semana")
    ix_cons  = _idx("consent_horario_extendido")
    ix_ini   = _idx("ventana_autorizada_ini")
    ix_fin   = _idx("ventana_autorizada_fin")
    ix_ures  = _idx("ult_resultado")

    now = dt.datetime.now()
    ymd = now.strftime("%Y-%m-%d")
    wkday = now.weekday()  # 0=Lunes
    is_holiday = ymd in set(params.get("holidays") or [])
    allowed = False
    if not is_holiday:
        if wkday <= 4:
            allowed = _in_range(now, params["wk_start"], params["wk_end"])
        elif wkday == 5:
            allowed = _in_range(now, params["sat_start"], params["sat_end"])
        else:
            allowed = bool(params.get("allow_sun", False))
    # nota: si hay consentimiento expreso por fila, se reevalúa más abajo

    recontact_days = int(params.get("recontact_days", 4))
    cooldown_map = params.get("cooldown_by_result", {})

    out = []
    for r in rows:
        # DNC
        if ix_dnc >= 0 and str(r[ix_dnc]).strip().lower() in ("1","true","sí","si","x"):
            continue
        # Canal único por semana
        if params.get("one_channel_per_week", True) and ix_csem >= 0:
            prev = str(r[ix_csem]).strip().lower()
            if prev and prev == canal_destino.lower():
                continue
        # Recontacto (último contacto → días)
        if ix_ucts >= 0:
            dt_last = _to_dt(r[ix_ucts])
            if dt_last and (now - dt_last).days < recontact_days:
                continue
        # Cooldown por resultado
        if ix_ures >= 0:
            res = str(r[ix_ures]).strip().upper()
            if res in cooldown_map:
                dt_last = _to_dt(r[ix_ucts]) if ix_ucts >= 0 else None
                if dt_last:
                    need_h = int(cooldown_map[res])
                    if (now - dt_last).total_seconds() < need_h * 3600:
                        continue
        # Ventana horaria
        permit = allowed
        # ¿consentimiento expreso por fila?
        has_consent = (ix_cons >= 0 and str(r[ix_cons]).strip().lower() in ("1","true","sí","si"))
        if has_consent and ix_ini >= 0 and ix_fin >= 0:
            permit = _in_range(now, str(r[ix_ini]), str(r[ix_fin]))
        if not permit:
            continue

        out.append(r)
    return out

# ---- Aprendizaje mínimo: registrar eventos y calcular mejor hora (por ID)
def learn_register_event(id_value: str, telefono: str, canal: str, resultado: str, contesto: bool, lista_origen: str):
    os.makedirs(_SYS_DIR, exist_ok=True)
    path = f"{_SYS_DIR}/contact_events.csv"
    is_new = not os.path.exists(path)
    now = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    row = [now, id_value or "", telefono or "", canal or "", resultado or "", "1" if contesto else "0",
           str(dt.datetime.now().hour), "", lista_origen or ""]
    with open(path, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f);
        if is_new:
            w.writerow(["ts_iso","id","telefono","canal","resultado","contesto","hora","agente","lista_origen"])
        w.writerow(row)
    try:
        drive_upload_path(SALIDAS_ID, path, "text/csv")
    except Exception:
        pass

def learn_compute_best_hour_by_id():
    """Llama cuando quieras refrescar; produce best_hour_by_id.csv"""
    src = f"{_SYS_DIR}/contact_events.csv"
    if not os.path.exists(src): return
    hits = {}  # id -> [24]
    with open(src, "r", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        for d in rdr:
            if d.get("contesto") == "1":
                i = d.get("id","").strip()
                h = int(d.get("hora","0") or 0)
                hits.setdefault(i, [0]*24)[h] += 1
    out = f"{_SYS_DIR}/best_hour_by_id.csv"
    with open(out, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        cols = ["id"] + [f"h{h:02d}" for h in range(24)] + ["top_hour"]
        w.writerow(cols)
        for i, arr in hits.items():
            top = max(range(24), key=lambda k: arr[k]) if any(arr) else -1
            w.writerow([i] + arr + [top])
    try:
        drive_upload_path(SALIDAS_ID, out, "text/csv")
    except Exception:
        pass

In [236]:
#SinergIA0240

# === Ley2300 — persistencia en Drive usando IDs del SettingID ===
def resolve_system_folder_id() -> str:
    """Usa primero 'Sistema'/'Sistema_SinergIA'; si no, 'Procesamiento'; si no, 'Salidas'."""
    if FOLDERS.get("Sistema"): return FOLDERS["Sistema"]
    if FOLDERS.get("Sistema_SinergIA"): return FOLDERS["Sistema_SinergIA"]
    if FOLDERS.get("Procesamiento"): return FOLDERS["Procesamiento"]
    return FOLDERS["Salidas"]

SYSTEM_FOLDER_ID = resolve_system_folder_id()
_L2300_NAME = "ley2300_params.json"

def l2300_default_params() -> dict:
    return {
        "recontact_days": 4,
        "wk_start": "07:00", "wk_end": "19:00",
        "sat_start": "08:00", "sat_end": "15:00",
        "allow_sun": False,
        "one_channel_per_week": True,
        "cooldown_by_result": {"RPC":72,"PROMESA":48,"BUZON":8,"NO_CONTESTA":4,"OCUPADO":2},
        "holidays": []
    }

def l2300_load_params() -> dict:
    fid, _ = drive_find_by_name(SYSTEM_FOLDER_ID, _L2300_NAME)
    if not fid:
        params = l2300_default_params()
        drive_upload_bytes(SYSTEM_FOLDER_ID, _L2300_NAME, "application/json",
                           json.dumps(params, ensure_ascii=False, indent=2).encode("utf-8"))
        return params
    name, data = drive_download_file(fid)
    try:
        return json.loads(data.decode("utf-8"))
    except Exception:
        return l2300_default_params()

def l2300_save_params(params: dict) -> None:
    payload = json.dumps(params, ensure_ascii=False, indent=2).encode("utf-8")
    drive_upload_bytes(SYSTEM_FOLDER_ID, _L2300_NAME, "application/json", payload)

In [237]:
#SinergIA0260

# E1 — Estructuras (desde gSheet), mismas firmas que usamos en la GUI
from typing import List, Dict, Optional

ESTRUCTURAS_SHEET_NAME = "Estructuras"
_estructuras_cache: Optional[List[Dict[str, object]]] = None

def _nk(s: str) -> str:
    return str(s).strip().replace(" ", "_").replace("-", "_").lower()

def load_estructuras(refresh: bool = False) -> List[Dict[str, object]]:
    """Lee la hoja 'Estructuras' y normaliza encabezados (snake_case)."""
    global _estructuras_cache
    if _estructuras_cache is not None and not refresh:
        return _estructuras_cache
    sh = gc.open_by_key(CONFIG_SPREADSHEET_ID)
    ws = sh.worksheet(ESTRUCTURAS_SHEET_NAME)
    rows = [{_nk(k): v for k, v in r.items()} for r in ws.get_all_records()]
    _estructuras_cache = rows
    return rows

def get_column_defs(archivo_logico: str,
                    sistema: str = "SinergIA_Cobros",
                    hoja: Optional[str] = None) -> List[Dict[str, object]]:
    """Filtra por Sistema/ArchivoLogico/(Hoja) y ordena por columnaorden."""
    recs = load_estructuras()
    sel = [
        r for r in recs
        if str(r.get("sistema","")).strip() == sistema
        and str(r.get("archivologico","")).strip() == archivo_logico
        and (hoja is None or str(r.get("hoja","")).strip() == hoja)
    ]
    def _as_int(x):
        try: return int(str(x).strip()) if x not in (None, "") else 0
        except Exception: return 0
    return sorted(sel, key=lambda r: _as_int(r.get("columnaorden")))

def get_column_names_from_estructuras(archivo_logico: str,
                                      sistema: str = "SinergIA_Cobros",
                                      hoja: Optional[str] = None) -> List[str]:
    """
    Devuelve los nombres de columnas esperados (orden correcto).
    Regla: usa 'nombrecolumna'; si está vacío → 'aliascanonico'.
    """
    defs = get_column_defs(archivo_logico, sistema, hoja)
    out: List[str] = []
    for r in defs:
        nombre = str(r.get("nombrecolumna","") or "").strip()
        alias  = str(r.get("aliascanonico","") or "").strip()
        out.append(nombre if nombre else alias)
    return out

In [238]:
#SinergIA0280

# E1-check — confirma que la función existe y que lee algo (si hay definición)
try:
    cols_ter = get_column_names_from_estructuras("TER_UNISONO")
    print("E1 OK · TER_UNISONO columnas definidas:", len(cols_ter))
except NameError as e:
    print("Falta E1:", e)

E1 OK · TER_UNISONO columnas definidas: 29


In [239]:
#SinergIA0300

# L1 — Normalizadores mínimos (sin pandas)
import re, io, csv, datetime as _dt
from typing import List, Tuple, Dict, Optional

# --- UNICODE whitelist desde el gSheet (hoja 'UNICODE'); cache ligera
_unicode_cache: Optional[set] = None
def load_unicode_whitelist() -> set:
    global _unicode_cache
    if _unicode_cache is not None:
        return _unicode_cache
    try:
        sh = gc.open_by_key(CONFIG_SPREADSHEET_ID)
        ws = sh.worksheet("UNICODE")
        rows = ws.get_all_records()
        # usa la primera columna que tenga algo tipo 'char', 'permit', 'unicode' en el encabezado
        if not rows:
            _unicode_cache = set();  return _unicode_cache
        keys = [k for k in rows[0].keys()]
        def _pick(keys):
            kl = [k for k in keys if "char" in k.lower() or "permit" in k.lower() or "unicode" in k.lower()]
            return kl[0] if kl else list(keys)[0]
        col = _pick(keys)
        _unicode_cache = set(str(r.get(col,"")).strip() for r in rows if str(r.get(col,"")).strip())
        return _unicode_cache
    except Exception:
        _unicode_cache = set()
        return _unicode_cache

# --- Helpers de campo
def _as_str(x):
    return "" if x is None else (x if isinstance(x, str) else str(x))

def clean_trim_hash(s: str) -> str:
    """TRIM y reemplaza '#' por 'No.' en direcciones."""
    t = _as_str(s).strip()
    if not t: return t
    t = re.sub(r"\s+", " ", t)
    t = t.replace("#", " No. ")
    t = re.sub(r"\s+", " ", t).strip()
    return t

def whitelist_unicode(s: str, allowed: set) -> str:
    """Permite solo caracteres listados en 'allowed' si la lista existe; si no, retorna el original."""
    if not allowed:
        return _as_str(s)
    out = []
    for ch in _as_str(s):
        if ch in allowed or ch.isalnum() or ch.isspace() or ch in "._-/:,;()[]{}@+%^&*'\"!¿?¡=":
            out.append(ch)
    return "".join(out)

def normalize_phone(s: str) -> str:
    """Conserva solo dígitos; si >10 recorta a últimos 10; si 7/10 dígitos lo considera válido."""
    digits = re.sub(r"\D+", "", _as_str(s))
    if len(digits) > 10:
        digits = digits[-10:]
    return digits

def _excel_serial_to_iso(v) -> Optional[str]:
    """Convierte número serial de Excel a 'YYYY-MM-DD' si está en rango razonable."""
    try:
        n = float(v)
        if 20000 <= n <= 60000:  # ~1954–2064
            base = _dt.datetime(1899, 12, 30)  # regla Excel (incluye bug 1900)
            d = base + _dt.timedelta(days=int(n))
            return d.strftime("%Y-%m-%d")
    except Exception:
        pass
    return None

def parse_date(s) -> str:
    """Intenta varios formatos comunes y serial Excel."""
    if s is None or s == "":
        return ""
    iso = _excel_serial_to_iso(s)
    if iso: return iso
    txt = _as_str(s).strip()
    for fmt in ("%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y", "%Y/%m/%d", "%d-%m-%Y"):
        try:
            return _dt.datetime.strptime(txt, fmt).strftime("%Y-%m-%d")
        except Exception:
            continue
    # fallback: detecta yyyy-mm-dd parcial
    m = re.match(r"(\d{4})[-/](\d{1,2})[-/](\d{1,2})", txt)
    if m:
        y, mo, d = int(m.group(1)), int(m.group(2)), int(m.group(3))
        try: return _dt.date(y, mo, d).strftime("%Y-%m-%d")
        except Exception: pass
    return txt  # deja como viene si no se pudo

def to_int_no_dec(s) -> str:
    """Convierte '12.0'→'12' si aplica; si no, deja texto original."""
    txt = _as_str(s).strip()
    if txt == "": return ""
    try:
        f = float(txt.replace(",", "."))
        if abs(f - int(f)) < 1e-9:
            return str(int(f))
    except Exception:
        pass
    return txt

# --- Detección heurística (por nombre de columna)
_TOK_ADDR  = ("dir", "direc")
_TOK_TEL   = ("tel", "cel", "movil", "móvil", "telefono", "teléfono")
_TOK_DATE  = ("fecha", "fech", "date")
_TOK_INT   = ("id", "cedula", "cédula", "documento", "nit", "cuenta", "num")

def detect_column_roles(cols: List[str]) -> Dict[str, List[int]]:
    roles = {"addr":[], "tel":[], "date":[], "int":[]}
    for i, c in enumerate([_as_str(x).lower() for x in cols]):
        if any(tok in c for tok in _TOK_ADDR): roles["addr"].append(i)
        if any(tok in c for tok in _TOK_TEL):  roles["tel"].append(i)
        if any(tok in c for tok in _TOK_DATE): roles["date"].append(i)
        if any(tok in c for tok in _TOK_INT):  roles["int"].append(i)
    return roles

def apply_cleaning(cols: List[str], rows: List[List[object]], allowed_set: Optional[set]=None) -> List[List[object]]:
    """Aplica limpieza columna-específica sobre la tabla."""
    allowed_set = allowed_set or load_unicode_whitelist()
    roles = detect_column_roles(cols)
    out_rows: List[List[object]] = []

    for r in rows:
        rr = list(r)  # copia
        # addresses
        for j in roles["addr"]:
            rr[j] = whitelist_unicode(clean_trim_hash(rr[j]), allowed_set)
        # phones
        for j in roles["tel"]:
            rr[j] = normalize_phone(rr[j])
        # dates
        for j in roles["date"]:
            rr[j] = parse_date(rr[j])
        # integers
        for j in roles["int"]:
            rr[j] = to_int_no_dec(rr[j])
        # strip suave al resto
        for j in range(len(rr)):
            if isinstance(rr[j], str):
                rr[j] = rr[j].strip()
        out_rows.append(rr)
    return out_rows

In [240]:
#SinergIA0320

# A — Mapeo de columnas por 'Estructuras' con fallback heurístico
from typing import Dict, List, Tuple

def _lc_list(xs): return [str(x).strip() for x in xs]
def _first_match(name: str, tokens: Tuple[str,...]) -> bool:
    n = name.lower()
    return any(tok in n for tok in tokens)

def get_roles_from_defs(archivo_logico: str, sistema: str = "SinergIA_Cobros", hoja: str | None = None) -> Dict[str, List[str]]:
    """
    Lee la hoja 'Estructuras' y devuelve nombres de columnas por rol:
      roles: {'id': [...], 'tel': [...], 'franja': [...]}
    Busca campos 'Rol'/'Semantica'/'Tipo' si existen; si no, usa nombre/alias.
    """
    defs = get_column_defs(archivo_logico, sistema, hoja)  # ya lo tienes definido
    roles = {"id": [], "tel": [], "franja": []}
    for r in defs:
        nombre = (r.get("NombreColumna") or r.get("AliasCanonico") or "").strip()
        if not nombre:
            continue
        # toma el posible campo de rol/semántica si existe
        meta = " ".join([
            str(r.get(k, "")) for k in ("Rol","Semantica","Semántica","Tipo","Uso")
            if k in r
        ]).lower()
        if any(t in meta for t in ("id","documento","identificador","cedula","cédula","nit","cuenta","contrato")):
            roles["id"].append(nombre)
        if any(t in meta for t in ("tel","cel","móvil","movil","telefono","teléfono")):
            roles["tel"].append(nombre)
        if any(t in meta for t in ("franja","mora")):
            roles["franja"].append(nombre)

        # si no hubo meta, intenta por nombre
        if not roles["id"] and _first_match(nombre, ("id","documento","cedula","cédula","nit","cuenta","contrato")):
            roles["id"].append(nombre)
        if _first_match(nombre, ("tel","cel","móvil","movil","telefono","teléfono")):
            roles["tel"].append(nombre)
        if _first_match(nombre, ("franja","mora")):
            roles["franja"].append(nombre)
    return roles

def map_columns_prefer_estructuras(
    cols: List[str], archivo_logico: str, prefer_estructuras: bool
) -> Dict[str, List[int]]:
    """
    Devuelve índices de columnas para roles {'id': [idx], 'tel': [idxs...], 'franja': [idx]}
    Si prefer_estructuras=True, usa nombres definidos en 'Estructuras'; si no, heurística.
    """
    roles_idx = {"id": [], "tel": [], "franja": []}
    lc_cols = [str(c).strip() for c in cols]

    if prefer_estructuras:
        names = get_roles_from_defs(archivo_logico)
        for role, names_list in names.items():
            for nm in names_list:
                try:
                    j = lc_cols.index(nm)
                    if j not in roles_idx[role]:
                        roles_idx[role].append(j)
                except ValueError:
                    continue

    # Fallback/Completar con heurística
    if not roles_idx["id"]:
        for i, c in enumerate(lc_cols):
            if _first_match(c, ("id","documento","cedula","cédula","nit","cuenta","contrato")):
                roles_idx["id"] = [i]; break
    if not roles_idx["tel"]:
        for i, c in enumerate(lc_cols):
            if _first_match(c, ("tel","cel","móvil","movil","telefono","teléfono")):
                roles_idx["tel"].append(i)
    if not roles_idx["franja"]:
        for i, c in enumerate(lc_cols):
            if _first_match(c, ("franja","mora")):
                roles_idx["franja"] = [i]; break

    return roles_idx

In [241]:
#SinergIA0340

# B — Exportar lista Genesys mínima (id, telefono, franja) con opción 'Estructuras'
import io, csv

def _guess_archivo_logico_from_tab(tab_idx: int, name: str) -> str:
    n = (name or "").upper()
    if tab_idx == 0:  return "TER_UNISONO" if "TER" in n else "IN_SISTECREDITO"
    if tab_idx == 1:  return "ENTRADA_AWS"
    if tab_idx == 2:  return "PROMESAS"
    if tab_idx == 3:  return "PAGOS"
    return "IN_SISTECREDITO"

# === Export Genesys (streaming + Ley2300) — subida por bytes a Drive ===
def export_genesys_csv(
    file_id: str,
    apply_clean: bool,
    prefer_estructuras: bool,
    archivo_logico_hint: str | None = None,
    sample_preview: int = 500,
    params: dict | None = None,
    out_format: str = "csv_comma"   # "csv_comma" | "csv_semicolon" | "xlsx"
):
    name, data = drive_download_file(file_id)
    base, ext = name.rsplit(".", 1)
    ext = ext.lower()

    # Entrada streaming
    if ext == "csv":
        cols, row_iter = _stream_rows_from_csv_bytes(data)
    else:
        cols, row_iter = _stream_rows_from_xlsx_bytes(data)

    # Ley2300 en streaming + 1 teléfono por ID
    params = params or l2300_load_params()
    best_by_id = ley2300_filter_stream_and_pick_one(cols, row_iter, params, canal_destino="voz")

    # Map columnas por Estructuras
    alog = archivo_logico_hint or _guess_archivo_logico_from_tab(0, name)
    idxs = map_columns_prefer_estructuras(cols, alog, prefer_estructuras)

    out_cols = ["id", "telefono", "franja"]
    rows_out = []
    for idv, (row, _) in best_by_id.items():
        rid = idv if idv and not idv.startswith("__noid__") else ""
        fran = _as_str(row[idxs["franja"][0]]) if idxs["franja"] else ""
        telcands = [(_as_str(row[j]), j) for j in idxs["tel"]] if idxs["tel"] else []
        telcands = [(normalize_phone(t) if apply_clean else t, j) for t, j in telcands if t]
        if not telcands:
            continue
        telcands.sort(key=lambda x: (len(x[0])>=10, len(x[0])), reverse=True)
        telefono = telcands[0][0]
        if apply_clean and idxs["id"]:
            rid = to_int_no_dec(rid)
        rows_out.append([rid, telefono, fran])

    # Serializar a bytes según formato (sin tocar disco)
    if out_format == "xlsx":
        data_bytes = table_to_xlsx_bytes(out_cols, rows_out)
        mime = "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        out_name = f"genesys_min_{base}.xlsx"
    else:
        sep = "," if out_format == "csv_comma" else ";"
        data_bytes = table_to_csv_bytes(out_cols, rows_out, sep=sep)
        suf = "" if sep == "," else "_semicolon"
        mime = "text/csv"
        out_name = f"genesys_min_{base}{suf}.csv"

    # Subir directo a Drive (Listas_Trabajo_Genesys o Salidas)
    target = FOLDERS.get("Listas_Trabajo_Genesys") or FOLDERS.get("Salidas")
    fid = drive_upload_bytes(target, out_name, mime, data_bytes)

    # Preview (solo para GUI)
    preview = rows_out[:sample_preview]
    return fid, out_name, out_cols, preview, len(rows_out)


In [242]:
#SinergIA0360

# Ley2300 — parámetros persistentes (JSON) + helpers
# Nota: _SYS_DIR es SOLO para desarrollo local/Colab como fallback.
# En producción, la ruta canónica es la carpeta de sistema en Drive,
# resuelta desde FOLDERS (SettingID).
import os, json, datetime as dt

_SYS_DIR = "/content/sinergia_system"
os.makedirs(_SYS_DIR, exist_ok=True)
_L2300_JSON = f"{_SYS_DIR}/ley2300_params.json"

def l2300_default_params() -> dict:
    return {
        "recontact_days": 4,
        "wk_start": "07:00", "wk_end": "19:00",
        "sat_start": "08:00", "sat_end": "15:00",
        "allow_sun": False,
        "one_channel_per_week": True,
        "cooldown_by_result": {  # horas mínimas por último resultado
            "RPC": 72, "PROMESA": 48, "BUZON": 8, "NO_CONTESTA": 4, "OCUPADO": 2
        },
        "holidays": []  # ["2025-12-25", ...]
    }

def l2300_load_params() -> dict:
    try:
        with open(_L2300_JSON, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        p = l2300_default_params()
        l2300_save_params(p)
        return p

def l2300_save_params(params: dict) -> None:
    with open(_L2300_JSON, "w", encoding="utf-8") as f:
        json.dump(params, f, ensure_ascii=False, indent=2)
    # copia ligera a Drive (mientras no exista SISTEMA_ID, uso SALIDAS_ID)
    try:
        path = _L2300_JSON
        drive_upload_path(SALIDAS_ID, path, "application/json")
    except Exception:
        pass

In [243]:
#SinergIA0400

# E2 — Exportar TODO a CSV con limpieza opcional (streaming, sin pandas)
import io, csv, tempfile
from typing import Tuple, List

def _stream_rows_from_csv_bytes(data: bytes) -> Tuple[List[str], List[List[str]]]:
    s = io.StringIO(data.decode("utf-8"))
    rdr = csv.reader(s)
    cols = next(rdr, [])
    # devolvemos un iterador (no lista) para no cargar todo en memoria
    return cols, rdr  # ojo: r es un iterador consumible

def _stream_rows_from_xlsx_bytes(data: bytes):
    import openpyxl
    wb = openpyxl.load_workbook(io.BytesIO(data), read_only=True, data_only=True)
    ws = wb.worksheets[0]
    it = ws.iter_rows(values_only=True)
    header = next(it, None) or []
    cols = [str(c) if c is not None else f"col_{i}" for i, c in enumerate(header)]
    # generador de filas
    def _gen():
        for r in it:
            yield ["" if r[j] is None else r[j] for j in range(len(cols))]
    return cols, _gen()

def export_full_csv_with_optional_clean(file_id: str, apply_clean: bool) -> Tuple[str, str, int]:
    """
    Descarga file_id (xlsx/csv), lo convierte/normaliza a CSV y lo sube a 'Salidas'.
    Si apply_clean=True, aplica limpieza por streaming (usa roles por encabezados).
    Retorna (drive_file_id_subido, local_path_csv, total_filas_escritas).
    """
    name, data = drive_download_file(file_id)
    base, ext = name.rsplit(".", 1)
    ext = ext.lower()

    if ext == "csv":
        cols, row_iter = _stream_rows_from_csv_bytes(data)
    else:
        cols, row_iter = _stream_rows_from_xlsx_bytes(data)

    roles = detect_column_roles(cols) if apply_clean else None
    allowed = load_unicode_whitelist() if apply_clean else set()

    out_path = f"/content/export_full_{base}.csv"
    wrote = 0
    with open(out_path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f, lineterminator="\n")
        w.writerow(cols)
        for r in row_iter:
            row = r
            if apply_clean:
                # limpieza fila a fila evitando recomputar roles
                rr = list(row)
                for j in roles["addr"]:
                    rr[j] = whitelist_unicode(clean_trim_hash(rr[j]), allowed)
                for j in roles["tel"]:
                    rr[j] = normalize_phone(rr[j])
                for j in roles["date"]:
                    rr[j] = parse_date(rr[j])
                for j in roles["int"]:
                    rr[j] = to_int_no_dec(rr[j])
                for j in range(len(rr)):
                    if isinstance(rr[j], str): rr[j] = rr[j].strip()
                row = rr
            w.writerow(row)
            wrote += 1

    # subir a Drive (Salidas)
    fid = drive_upload_path(SALIDAS_ID, out_path, "text/csv")
    return fid, out_path, wrote

In [244]:
#SinergIA0420
# G1 — Generar lista Genesys mínima (id, telefono, franja) desde archivo de Drive
import io, csv, tempfile
from typing import List, Tuple

_ID_TOKS     = ("id", "documento", "cedula", "cédula", "nit", "cuenta", "contrato")
_TEL_TOKS    = ("tel", "cel", "móvil", "movil", "telefono", "teléfono")
_FRANJA_TOKS = ("franja", "mora")

def _pick_idx(cols: List[str], tokens: Tuple[str, ...]) -> int:
    lc = [str(c).lower() for c in cols]
    for i, c in enumerate(lc):
        if any(tok in c for tok in tokens):
            return i
    return -1

def _pick_all_idxs(cols: List[str], tokens: Tuple[str, ...]) -> List[int]:
    lc = [str(c).lower() for c in cols]
    idxs = []
    for i, c in enumerate(lc):
        if any(tok in c for tok in tokens):
            idxs.append(i)
    return idxs

def export_genesys_csv(file_id: str, apply_clean: bool, sample_preview: int = 500) -> Tuple[str, str, List[str], List[List[str]], int]:
    """
    Construye lista Genesys mínima: columnas ['id','telefono','franja'].
    - Lee por streaming (xlsx/csv).
    - Toma el primer ID que encuentre, todas las columnas de teléfono y una 'franja' si existe.
    - Genera 1 fila por teléfono no vacío. Deduplica por (id, telefono).
    - Si apply_clean=True, normaliza teléfonos/fechas/enteros y trim/UNICODE para direcciones (por si afecta franja).
    Retorna (file_id_drive_subido, local_path, cols_preview, rows_preview, total_filas_generadas).
    """
    name, data = drive_download_file(file_id)
    base, ext = name.rsplit(".", 1)
    ext = ext.lower()

    if ext == "csv":
        cols, row_iter = _stream_rows_from_csv_bytes(data)
    else:
        cols, row_iter = _stream_rows_from_xlsx_bytes(data)

    roles = detect_column_roles(cols) if apply_clean else None
    allowed = load_unicode_whitelist() if apply_clean else set()

    idx_id    = _pick_idx(cols, _ID_TOKS)
    tel_idxs  = _pick_all_idxs(cols, _TEL_TOKS)
    idx_fran  = _pick_idx(cols, _FRANJA_TOKS)

    out_cols = ["id", "telefono", "franja"]
    out_path = f"/content/genesys_min_{base}.csv"
    seen = set()
    total = 0
    preview_rows: List[List[str]] = []

    with open(out_path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f, lineterminator="\n")
        w.writerow(out_cols)
        for r in row_iter:
            row = r
            if apply_clean:
                rr = list(row)
                # limpiamos solo lo que puede impactar teléfono e id/franja
                for j in roles["tel"]:
                    rr[j] = normalize_phone(rr[j])
                if idx_id >= 0:
                    rr[idx_id] = to_int_no_dec(rr[idx_id])
                if idx_fran >= 0:
                    rr[idx_fran] = _as_str(rr[idx_fran]).strip()
                row = rr

            rid  = _as_str(row[idx_id]) if idx_id >= 0 else ""
            fran = _as_str(row[idx_fran]) if idx_fran >= 0 else ""
            for j in (tel_idxs or []):
                tel = normalize_phone(row[j]) if apply_clean else _as_str(row[j])
                if not tel:
                    continue
                key = (rid, tel)
                if key in seen:
                    continue
                seen.add(key)
                out_row = [rid, tel, fran]
                w.writerow(out_row)
                total += 1
                if len(preview_rows) < sample_preview:
                    preview_rows.append(out_row)

    fid = drive_upload_path(SALIDAS_ID, out_path, "text/csv")
    return fid, out_path, out_cols, preview_rows, total

In [245]:
#SinergIA0440
# === Limpieza (helpers) — PREVIEW ===
import re, unicodedata, datetime as dt

def _norm_spaces(s: str) -> str:
    s = "" if s is None else str(s)
    s = unicodedata.normalize("NFKC", s)
    return re.sub(r"\s+", " ", s.strip())

def _is_empty(x) -> bool:
    return x is None or (isinstance(x, str) and _norm_spaces(x) == "") or (not isinstance(x, str) and x=="")

def _norm_date_any(x) -> str:
    """Devuelve YYYY-MM-DD si reconoce la fecha, o el valor original si no."""
    if _is_empty(x): return ""
    s = _norm_spaces(str(x))
    m = re.match(r"^(\d{4})[-/](\d{1,2})[-/](\d{1,2})$", s)   # yyyy-mm-dd
    if m:
        y, mo, d = map(int, m.groups())
        try: return dt.date(y, mo, d).isoformat()
        except: return s
    m = re.match(r"^(\d{1,2})[-/](\d{1,2})[-/](\d{4})$", s)   # dd/mm/yyyy
    if m:
        d, mo, y = map(int, m.groups())
        try: return dt.date(y, mo, d).isoformat()
        except: return s
    try:                                          # Excel serial
        val = float(s); base = dt.date(1899, 12, 30)
        return (base + dt.timedelta(days=int(val))).isoformat()
    except:
        return s

def _digits_only(s) -> str:
    return "" if s is None else re.sub(r"\D+", "", str(s))

def _norm_phone_co(s: str) -> str:
    """Colombia: intenta dejar móviles (10 dígitos, empieza por 3). Si no, retorna dígitos limpios."""
    d = _digits_only(s)
    if d.startswith("57") and len(d) >= 12:  # quita +57/0057
        d = d[2:]
    if len(d) >= 10:
        d = d[-10:]
    return d

def _slug(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    return re.sub(r"[^a-z0-9]+", "", s.lower())

def _role_map(cols: list[str]) -> dict:
    """Mapea columnas conocidas a roles: mobile, phone1, last_date, last_result, propensity, address, etc."""
    roles = {c: None for c in cols}
    for c in cols:
        k = _slug(c)
        if any(x in k for x in ["telmov","movil","cel"]):         roles[c] = "mobile"
        elif any(x in k for x in ["telefono","tel1","tel_1"]):    roles[c] = "phone1"
        elif any(x in k for x in ["tel2","telefono2","tel_2"]):   roles[c] = "phone2"
        elif any(x in k for x in ["fecult","ultgest","fechagestion"]): roles[c] = "last_date"
        elif any(x in k for x in ["resultado","resul","gestion"]): roles[c] = "last_result"
        elif any(x in k for x in ["propen","score","prob"]):      roles[c] = "propensity"
        elif "direccion" in k:                                    roles[c] = "address"
        elif any(x in k for x in ["nacim","exped"]):              roles[c] = "date_misc"
    return roles

def apply_cleaning_to_preview(cols: list[str], rows: list[list]) -> tuple[list[list], str]:
    """Limpia el preview en memoria y retorna (rows_limpias, reporte)."""
    roles = _role_map(cols)
    cleaned, n_phone_norm, n_dates, n_addr = [], 0, 0, 0

    for r in rows:
        rr = list(r)
        for j, c in enumerate(cols):
            role = roles.get(c); val = rr[j]
            if role in ("mobile","phone1","phone2"):
                newv = _norm_phone_co(val)
                if newv != ("" if val is None else _digits_only(val)): n_phone_norm += 1
                rr[j] = newv
            elif role in ("last_date","date_misc"):
                newv = _norm_date_any(val)
                if newv != ("" if val is None else str(val)): n_dates += 1
                rr[j] = newv
            elif role == "address":
                newv = _norm_spaces(str(val).replace("#", " # "))
                if newv != ("" if val is None else str(val)): n_addr += 1
                rr[j] = newv
            elif isinstance(val, str):
                rr[j] = _norm_spaces(val)
        cleaned.append(rr)

    rep = f"Limpieza preview: teléfonos≈{n_phone_norm}, fechas≈{n_dates}, direcciones≈{n_addr}."
    return cleaned, rep

In [246]:
#SinergIA0460
# === Normalización de direcciones (solo filas, nunca encabezados) ===

def _direccion_normalizada_valor(x) -> str:
    """
    Aplica reglas de dirección:
    1) Vacío / nulo -> "SIN DIRECCION"
    2) Solo números -> "SIN DIRECCION"
    3) Números al inicio antes de la primera palabra (≥2 letras) ->
       "CERCA A " + resto de la dirección
    """
    # Regla 1: vacío / nulo
    if _is_empty(x):
        return "SIN DIRECCION"

    texto = _norm_spaces(str(x))

    # Regla 2: solo números (ignorando espacios)
    solo_digitos = True
    for ch in texto:
        if not ch.isdigit() and not ch.isspace():
            solo_digitos = False
            break
    if solo_digitos:
        return "SIN DIRECCION"

    # Buscamos tokens
    tokens = texto.split()
    if not tokens:
        return "SIN DIRECCION"

    def _cuenta_letras(tok: str) -> int:
        c = 0
        for ch in tok:
            if ch.isalpha():
                c += 1
        return c

    # Primera palabra con al menos 2 letras
    idx_palabra = None
    for i, tok in enumerate(tokens):
        if _cuenta_letras(tok) >= 2:
            idx_palabra = i
            break

    # Si no hay palabra "real" -> consideramos dirección inválida
    if idx_palabra is None:
        return "SIN DIRECCION"

    # Verificar si antes de esa palabra solo hay números
    if idx_palabra > 0:
        solo_prefijo_numerico = True
        for tok in tokens[:idx_palabra]:
            for ch in tok:
                if not ch.isdigit():
                    solo_prefijo_numerico = False
                    break
            if not solo_prefijo_numerico:
                break

        if solo_prefijo_numerico:
            resto = " ".join(tokens[idx_palabra:])
            # Regla 3: números iniciales -> "CERCA A " + resto
            return "CERCA A " + resto

    # Si no aplicó ninguna transformación especial, devolvemos tal cual
    return texto


def _normalizar_serie_direccion(serie):
    """
    Aplica _direccion_normalizada_valor a una Serie pandas.
    Trabaja solo sobre los valores, no toca nombres de columnas.
    """
    return serie.apply(_direccion_normalizada_valor)


In [247]:
#SinergIA0480
# ---- normalización de direcciones TER_UNISONO (usa helpers globales _norm_spaces / _is_empty)
def _direccion_columns_ter(df: pd.DataFrame) -> list:
    """
    Determina qué columnas del df se consideran direcciones de cliente para TER/IN.
    Usa Config_SinergIA.Estructuras como base y cae a heurística por nombre.
    """
    cols_cfg = []
    try:
        defs = get_column_names_from_estructuras("TER_UNISONO") or []
        # nos quedamos solo con las que parecen direcciones
        cols_cfg = [c for c in defs if "direcci" in str(c).lower()]
    except Exception:
        cols_cfg = []

    # intersección con df real
    from_cfg = [c for c in cols_cfg if c in df.columns]

    # fallback heurístico: cualquier columna del df que contenga "direcci"
    heur = [c for c in df.columns if "direcci" in str(c).lower()]

    # unión preservando orden de aparición en df
    seen = set()
    out = []
    for c in list(from_cfg) + list(heur):
        if c not in seen and c in df.columns:
            seen.add(c)
            out.append(c)
    return out

In [248]:
#SinergIA0500
def normalize_direcciones_ter(df: pd.DataFrame) -> tuple:
    """
    Normaliza los campos de dirección de TER_UNISONO:
    - Usa get_direccion_columns_ter() para identificar columnas.
    - Aplica reglas SIN DIRECCION / CERCA A ... solo sobre filas.

    Devuelve (df_normalizado, lista_columnas_direccion).
    """
    df = df.copy()
    cols_dir = get_direccion_columns_ter()
    cols_dir = [c for c in cols_dir if c in df.columns]

    if not cols_dir:
        print("Aviso normalize_direcciones_ter: no se encontraron columnas de dirección en df.")
        return df, cols_dir

    for c in cols_dir:
        df[c] = _normalizar_serie_direccion(df[c])

    return df, cols_dir


In [249]:
#SinergIA0520
# === Heurística & armado de salidas (no toca tus helpers de Drive) ===
import pandas as _pd
import numpy as _np
from datetime import datetime as _dt, timedelta as _td

# Si tienes gc y CONFIG_SHEET_ID, toma el árbol del GSheet; si no, usa el Excel local de apoyo.
def load_arbol_heuristico(gc=None, CONFIG_SHEET_ID=None, local_path="/mnt/data/Config_SinergIA.xlsx"):
    try:
        if gc and CONFIG_SHEET_ID:
            ws = gc.open_by_key(CONFIG_SHEET_ID).worksheet("ArbolHeuristico")
            data = ws.get_all_records()
            df = _pd.DataFrame(data)
        else:
            df = _pd.read_excel(local_path, sheet_name="ArbolHeuristico")
    except Exception:
        # último intento: Excel local
        df = _pd.read_excel(local_path, sheet_name="ArbolHeuristico")
    # normalizaciones ligeras
    for c in ("Tipificacion (TER)", "Tipificacion TER", "Tipificación (TER)"):
        if c in df.columns:
            df["tip_key"] = df[c].astype(str).str.upper().str.strip()
            break
    if "tip_key" not in df.columns:
        # si no existe columna de mapeo explícita, dejamos tip_key vacío (merge tolerante más abajo)
        df["tip_key"] = ""
    # columnas mínimas esperadas
    for col, val in {"Canal": "LLAMADA", "Score": 0, "Prioridad":"MUST", "Estrategia de Carga":"BASE"}.items():
        if col not in df.columns:
            df[col] = val
    return df

def _guess(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def apply_ley2300_filter(df, ley_params):
    """Aplica recontacto por días y franjas. No elimina nada si faltan columnas."""
    if df is None or df.empty:
        return df
    # columnas posibles:
    col_fecha_ult = _guess(df, ["UltimaGestionFecha","fecha_ultima_gestion","FechaUltimoContacto","last_contact_dt"])
    col_dow = _guess(df, ["dayofweek","DOW"])
    col_hora = _guess(df, ["HoraUltimoContacto","last_contact_hhmm"])
    days = int(ley_params.get("recontact_days", 4))
    wk_start, wk_end = ley_params.get("wk_start","07:00"), ley_params.get("wk_end","19:00")
    sa_start, sa_end = ley_params.get("sat_start","08:00"), ley_params.get("sat_end","15:00")
    allow_sun = bool(ley_params.get("allow_sun", False))

    out = df.copy()
    now = _dt.now()
    # Regla de días
    if col_fecha_ult and col_fecha_ult in out.columns:
        def _ok_days(x):
            try:
                dt = _pd.to_datetime(x, errors="coerce")
                if _pd.isna(dt):
                    return True
                return (now - dt) >= _td(days=days)
            except Exception:
                return True
        mask_days = out[col_fecha_ult].apply(_ok_days)
    else:
        mask_days = _pd.Series([True]*len(out), index=out.index)

    # Reglas de franja (simple: validamos que el próximo intento caiga dentro de LV/Sáb; domingo según flag)
    # Si no hay info de día/hora, no filtramos por franja.
    mask_franja = _pd.Series([True]*len(out), index=out.index)
    if col_dow in out.columns and col_hora in out.columns:
        def _ok_slot(dow, hhmm):
            try:
                h = str(hhmm)
                if dow in (0,1,2,3,4):   # L-V
                    return wk_start <= h <= wk_end
                if dow == 5:            # Sáb
                    return sa_start <= h <= sa_end
                if dow == 6:            # Dom
                    return allow_sun
            except Exception:
                return True
            return True
        mask_franja = out[[col_dow, col_hora]].apply(lambda r: _ok_slot(r[col_dow], r[col_hora]), axis=1)

    return out[mask_days & mask_franja].copy()

def build_entrada_aws(df_in, df_prom=None, df_pagos=None, df_arbol=None, ley_params=None):
    """
    - Recibe la tabla 'IN' (TER/UNISONO) pre-leída.
    - Enlaza tipificación contra ArbolHeuristico para obtener Canal, Score y Prioridad.
    - Aplica filtro Ley2300 (si llegan parámetros).
    - Devuelve (entrada_aws_sorted, teleton_df, listas_por_canal_dict)
    """
    if df_in is None or df_in.empty:
        return _pd.DataFrame(), _pd.DataFrame(), {}

    df = df_in.copy()

    # === 1) Mapeo tipificación → (Canal, Score, Prioridad, ...)
    col_tip = _guess(df, ["Tipificacion","TIPIFICACION","ResultadoGestion","Resultado","Tipificación"])
    if col_tip:
        df["tip_key"] = df[col_tip].astype(str).str.upper().str.strip()
    else:
        df["tip_key"] = ""

    if df_arbol is None:
        df_arbol = _pd.DataFrame(columns=["tip_key","Canal","Score","Prioridad","Estrategia de Carga"])
    cols_pick = [c for c in ["tip_key","Canal","Score","Prioridad","Estrategia de Carga","Tipo de Marcacion"] if c in df_arbol.columns]
    df = df.merge(df_arbol[cols_pick].drop_duplicates("tip_key"), on="tip_key", how="left")

    # Defaults si algo no mapeó
    if "Canal" not in df.columns: df["Canal"] = "LLAMADA"
    if "Score" not in df.columns: df["Score"] = 0
    if "Prioridad" not in df.columns: df["Prioridad"] = "SHOULD"

    # === 2) Señales de promesas/pagos (opcionales)
    if df_prom is not None and not df_prom.empty:
        # ejemplo básico: marcar si tuvo promesa reciente
        c_id = _guess(df, ["Identificacion","Documento","ID","ClienteId","NumDocumento"])
        c_id_p = _guess(df_prom, ["Identificacion","Documento","ID","ClienteId","NumDocumento"])
        if c_id and c_id_p:
            prom_rec = df_prom.groupby(c_id_p).size().rename("cnt_prom")
            df = df.join(prom_rec, on=c_id)
            df["cnt_prom"] = df["cnt_prom"].fillna(0)

    if df_pagos is not None and not df_pagos.empty:
        c_id = _guess(df, ["Identificacion","Documento","ID","ClienteId","NumDocumento"])
        c_id_g = _guess(df_pagos, ["Identificacion","Documento","ID","ClienteId","NumDocumento"])
        if c_id and c_id_g:
            pagos_rec = df_pagos.groupby(c_id_g)["ValorPago" if "ValorPago" in df_pagos.columns else df_pagos.columns[-1]].sum().rename("sum_pagos")
            df = df.join(pagos_rec, on=c_id)
            df["sum_pagos"] = df["sum_pagos"].fillna(0)

    # === 3) Score final simple (heurístico + señales); si tienes 'ScoreML' lo sumas ponderado
    col_ml = _guess(df, ["ScoreML","PropensionPago","propension","score_ml"])
    df["score_final"] = df["Score"].fillna(0) + (0.5 * df[col_ml] if col_ml else 0)

    # === 4) Ley2300
    if ley_params:
        df = apply_ley2300_filter(df, ley_params)

    # === 5) Ordenamiento (prioridad → score → algo de recencia si existe)
    col_fecha = _guess(df, ["UltimaGestionFecha","FechaUltimoContacto","fecha_ultima_gestion"])
    prio_order = _pd.CategoricalDtype(categories=["MUST","SHOULD","COULD","WONT"], ordered=True)
    df["__prio"] = df["Prioridad"].astype(str).str.upper().map(lambda x: x if x in prio_order.categories else "SHOULD").astype(prio_order)
    sort_cols = ["__prio", "score_final"]
    ascend = [True, False]
    if col_fecha:
        df[col_fecha] = _pd.to_datetime(df[col_fecha], errors="coerce")
        sort_cols += [col_fecha]
        ascend += [True]  # más antiguas primero
    entrada_aws = df.sort_values(by=sort_cols, ascending=ascend).drop(columns=["__prio"])

    # === 6) Teleton (no incluye Infinivirt en Entrada_AWS; aquí sí, para digitación)
    teleton_cols = []
    for c in ["Canal","score_final","Prioridad"]:
        if c in entrada_aws.columns: teleton_cols.append(c)
    teleton = entrada_aws.copy()
    teleton["Infinivirt"] = ""       # campo editable en operación
    teleton["Observacion"] = ""      # opcional
    # (si luego Infinivirt == 'NO', podrás penalizar y reordenar)

    # === 7) Listas por canal (dict de DataFrames)
    listas = {}
    for canal in ["LLAMADA","WHATSAPP","SMS","EMAIL"]:
        if "Canal" in entrada_aws.columns:
            sub = entrada_aws[entrada_aws["Canal"].str.upper()==canal].copy()
        else:
            sub = _pd.DataFrame()
        listas[canal] = sub

    return entrada_aws, teleton, listas

In [250]:
#SinergIA0540
# === Heurístico configurable: defaults, load/save JSON en MODELOS_ID, scoring y Árbol ===
import json, math, re
import pandas as pd
import numpy as np

# Archivo de parámetros en la carpeta MODELOS
MODEL_CFG_FILENAME = "HeuristicoParams.json"

def modelcfg_default_params() -> dict:
    return {
        "w_contact": 0.60,
        "w_saldo":   0.20,
        "w_mora":    0.20,
        "saldo_norm": "log1p",      # "log1p" | "sqrt" | "minmax"
        "mora_half_life_days": 15,  # half-life para exp(-ln(2)*dias/HL)
        "use_tree": True,           # aplicar ÁrbolHeuristico como bono/penal
        "max_rule_weight_abs": 0.50 # cap por seguridad
    }

def _drive_find_by_name(folder_id: str, name: str):
    # Búsqueda simple por nombre dentro de los recientes.  Si no aparece, retorna None.
    try:
        items = drive_list_recent(folder_id, top=100)
        for it in items:
            if it.get("name") == name:
                return it.get("id")
    except Exception:
        pass
    return None

def modelcfg_load_params() -> dict:
    try:
        fid = _drive_find_by_name(MODELOS_ID, MODEL_CFG_FILENAME)
        if not fid:
            return modelcfg_default_params()
        name, data = drive_download_file(fid)
        return json.loads(data.decode("utf-8"))
    except Exception:
        return modelcfg_default_params()

def modelcfg_save_params(d: dict) -> str:
    data = json.dumps(d, ensure_ascii=False, indent=2).encode("utf-8")
    fid = drive_upload_bytes(MODELOS_ID, MODEL_CFG_FILENAME, data, "application/json")
    return fid

# --- Carga del ÁrbolHeuristico desde el gSheet canon ---
def load_arbol_heuristico() -> pd.DataFrame:
    try:
        sh = gc.open_by_key(CONFIG_SPREADSHEET_ID)
        ws = sh.worksheet("ArbolHeuristico")
        df = pd.DataFrame(ws.get_all_records())
        # normaliza encabezados
        df.columns = [str(c).strip() for c in df.columns]
        # nombres tolerados
        ren = {
            "Campo":"Campo","campo":"Campo",
            "Operador":"Operador","operador":"Operador",
            "Valor":"Valor","valor":"Valor",
            "Peso":"Peso","peso":"Peso"
        }
        df = df.rename(columns={k:v for k,v in ren.items() if k in df.columns})
        return df
    except Exception:
        return pd.DataFrame()

# --- Detección robusta de columnas útiles en la base ---
def _pick_first(df: pd.DataFrame, candidates: list[str]):
    for c in candidates:
        if c in df.columns:
            return c
    # tolerancia por case o underscores
    low = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c.lower() in low:
            return low[c.lower()]
    return None

def _to_numeric_safe(s: pd.Series) -> pd.Series:
    if s.dtype.kind in "ifc":
        return pd.to_numeric(s, errors="coerce")
    # limpia símbolos y normaliza coma/punto
    t = (
        s.astype(str)
         .str.replace(r"[^\d,.\-]", "", regex=True)
         .str.replace(",", ".", regex=False)
    )
    return pd.to_numeric(t, errors="coerce")

# --- Términos base normalizados ---
def _contact_term(df: pd.DataFrame) -> pd.Series:
    c = _pick_first(df, ["contact_mean","Contactabilidad","final_mean","propension_mean"])
    if not c:
        return pd.Series(0.0, index=df.index)
    x = pd.to_numeric(df[c], errors="coerce").fillna(0.0)
    return x.clip(0, 1)

def _saldo_term(df: pd.DataFrame, modo: str) -> pd.Series:
    c = _pick_first(df, ["Saldo","SaldoCapital","Saldo_total","Monto","Deuda"])
    if not c:
        return pd.Series(0.0, index=df.index)
    x = _to_numeric_safe(df[c]).fillna(0.0).clip(lower=0)
    if modo == "sqrt":
        num = np.sqrt(x)
        p99 = np.nanpercentile(num, 99) or 1.0
        return (num / p99).clip(0,1)
    if modo == "minmax":
        lo, hi = np.nanpercentile(x, 1), np.nanpercentile(x, 99)
        if hi <= lo:
            return pd.Series(0.0, index=df.index)
        return ((x - lo)/(hi - lo)).clip(0,1)
    # default: log1p
    num = np.log1p(x)
    p99 = np.nanpercentile(num, 99) or 1.0
    return (num / p99).clip(0,1)

def _mora_term(df: pd.DataFrame, half_life_days: int) -> pd.Series:
    c = _pick_first(df, ["DiasMora","dias_mora","Días de mora","DMora"])
    if not c:
        # intenta ‘Franja’ como aproximación muy básica
        f = _pick_first(df, ["Franja","franja","FranjaMora"])
        if not f:
            return pd.Series(1.0, index=df.index)  # sin info → no penaliza
        # mapeo simple por regex
        fran = df[f].astype(str).str.upper()
        approx = np.where(fran.str.contains("0-30"), 15,
                 np.where(fran.str.contains("31-60"), 45,
                 np.where(fran.str.contains("61-90"), 75,
                 np.where(fran.str.contains("90|>90|MÁS"), 120, 30))))
        dias = pd.Series(approx, index=df.index, dtype="float")
    else:
        dias = pd.to_numeric(df[c], errors="coerce").fillna(0.0).clip(lower=0)
    hl = max(1, int(half_life_days))
    return np.exp(-math.log(2) * dias / hl).clip(0,1)

# --- Reglas del Árbol: Campo, Operador, Valor, Peso ---
def score_rules_from_tree(df: pd.DataFrame, heur_df: pd.DataFrame, cap_abs: float=0.5) -> pd.Series:
    if heur_df is None or heur_df.empty:
        return pd.Series(0.0, index=df.index)
    out = pd.Series(0.0, index=df.index, dtype="float")
    for _, rule in heur_df.iterrows():
        campo = str(rule.get("Campo","")).strip()
        op    = str(rule.get("Operador","")).strip()
        val   = rule.get("Valor", "")
        peso  = pd.to_numeric(pd.Series([rule.get("Peso",0.0)])).iloc[0]
        if not campo or campo not in df.columns:
            continue
        s = df[campo]
        try:
            if op in ("==","="):
                mask = s.astype(str) == str(val)
            elif op in ("!=", "<>"):
                mask = s.astype(str) != str(val)
            elif op in (">",">=","<","<="):
                a = pd.to_numeric(s, errors="coerce"); b = pd.to_numeric(pd.Series([val]), errors="coerce").iloc[0]
                if op == ">":  mask = a >  b
                if op == ">=": mask = a >= b
                if op == "<":  mask = a <  b
                if op == "<=": mask = a <= b
            elif op.lower() == "in":
                vals = {v.strip() for v in str(val).split(",")}
                mask = s.astype(str).isin(vals)
            elif op.lower() == "contains":
                mask = s.astype(str).str.contains(str(val), na=False, case=False)
            elif op.lower() == "startswith":
                mask = s.astype(str).str.startswith(str(val), na=False)
            elif op.lower() == "endswith":
                mask = s.astype(str).str.endswith(str(val), na=False)
            else:
                continue
            out = out + np.where(mask, float(peso), 0.0)
        except Exception:
            continue
    return out.clip(lower=-abs(cap_abs), upper=abs(cap_abs))

def apply_model_heuristico(df_in: pd.DataFrame,
                           heur_df: pd.DataFrame|None=None,
                           params: dict|None=None) -> pd.DataFrame:
    """
    Aplica score = w_contact*contact + w_saldo*saldo + w_mora*mora + score_rules(Árbol).
    Devuelve copia ordenada desc por _score, con columnas diagnósticas.
    """
    if df_in is None or df_in.empty:
        return df_in
    p = modelcfg_default_params() | (params or {})
    c = _contact_term(df_in)
    s = _saldo_term(df_in, p.get("saldo_norm","log1p"))
    m = _mora_term(df_in,  p.get("mora_half_life_days",15))
    r = score_rules_from_tree(df_in, heur_df if p.get("use_tree", True) else None,
                              cap_abs=p.get("max_rule_weight_abs", 0.5))
    score = p["w_contact"]*c + p["w_saldo"]*s + p["w_mora"]*m + r
    out = df_in.copy()
    out["_contact"] = c.round(6)
    out["_saldo"]   = s.round(6)
    out["_mora"]    = m.round(6)
    out["_rules"]   = r.round(6)
    out["_score"]   = score.round(6)
    return out.sort_values("_score", ascending=False, kind="mergesort")

In [251]:
#SinergIA0560
# === ML helpers (XGBoost / LightGBM / RandomForest) + apply_model_ml ===
import io, json, numpy as np, pandas as pd

# Reutiliza el finder que ya tenemos:
# _drive_find_by_name(MODELOS_ID, name) -> file_id or None
# drive_download_file(file_id) -> (name, bytes)

def _load_features_list() -> list[str] | None:
    """Lee features desde MODELOS_ID si existe 'features_Entrada_AWS.json'."""
    try:
        fid = _drive_find_by_name(MODELOS_ID, "features_Entrada_AWS.json")
        if not fid:
            return None
        _, data = drive_download_file(fid)
        obj = json.loads(data.decode("utf-8"))
        feats = obj.get("features") if isinstance(obj, dict) else obj
        if isinstance(feats, list) and all(isinstance(x, str) for x in feats):
            return feats
    except Exception:
        pass
    return None

def _select_features(df: pd.DataFrame, fallback_extra: list[str] | None = None) -> pd.DataFrame:
    """Prioriza features de archivo; si no, selecciona numéricas + algunas útiles."""
    feats = _load_features_list()
    if feats:
        cols = [c for c in feats if c in df.columns]
        if cols:
            return df[cols].copy()
    # fallback robusto: numéricas + candidatos comunes
    numeric = df.select_dtypes(include=["number"]).columns.tolist()
    extras  = []
    for c in (fallback_extra or ["DiasMora","Saldo","contact_mean","final_mean","propension_mean"]):
        if c in df.columns and c not in numeric:
            extras.append(c)
    cols = list(dict.fromkeys(numeric + extras))[:128]  # cap defensivo
    return df[cols].copy() if cols else pd.DataFrame(index=df.index)

def _try_load_rf():
    try:
        from joblib import load as joblib_load
    except Exception as e:
        raise ImportError("Falta joblib para RandomForest (instalar joblib).") from e
    fid = _drive_find_by_name(MODELOS_ID, "rf_Entrada_AWS.pkl")
    if not fid:
        raise FileNotFoundError("No encontré rf_Entrada_AWS.pkl en MODELOS.")
    _, data = drive_download_file(fid)
    return joblib_load(io.BytesIO(data))

def _try_load_xgb():
    try:
        import xgboost as xgb
    except Exception as e:
        raise ImportError("Falta xgboost para XGBoost.") from e
    fid = _drive_find_by_name(MODELOS_ID, "xgb_Entrada_AWS.json")
    if not fid:
        raise FileNotFoundError("No encontré xgb_Entrada_AWS.json en MODELOS.")
    _, data = drive_download_file(fid)
    booster = xgb.Booster()
    booster.load_model(io.BytesIO(data))
    return booster

def _try_load_lgbm():
    try:
        import lightgbm as lgb
    except Exception as e:
        raise ImportError("Falta lightgbm para LightGBM.") from e
    fid = _drive_find_by_name(MODELOS_ID, "lgbm_Entrada_AWS.txt")
    if not fid:
        raise FileNotFoundError("No encontré lgbm_Entrada_AWS.txt en MODELOS.")
    _, data = drive_download_file(fid)
    booster = lgb.Booster(model_str=data.decode("utf-8"))
    return booster

def apply_model_ml(df_base: pd.DataFrame, model_name: str) -> pd.DataFrame:
    """
    Aplica modelo aprendido y devuelve df con columnas:
      _prob (0..1), _score (= _prob), y mantiene columnas originales.
    model_name: 'xgb' | 'lgbm' | 'rf'
    Requiere artefactos en MODELOS_ID:
      - XGB:  xgb_Entrada_AWS.json
      - LGBM: lgbm_Entrada_AWS.txt
      - RF:   rf_Entrada_AWS.pkl
      (opcional) features_Entrada_AWS.json con lista de columnas
    """
    if df_base is None or df_base.empty:
        return df_base
    X = _select_features(df_base)
    if X.empty:
        raise RuntimeError("No hay features numéricas reconocibles para el modelo.")

    if model_name == "rf":
        mdl = _try_load_rf()
        if hasattr(mdl, "predict_proba"):
            prob = mdl.predict_proba(X)[:, -1]
        else:
            pred = mdl.predict(X)
            # normalización simple si no hay proba
            pred = pd.to_numeric(pd.Series(pred), errors="coerce").fillna(0.0)
            prob = (pred - pred.min()) / (pred.max() - pred.min() + 1e-9)
    elif model_name == "xgb":
        import xgboost as xgb  # ya validado arriba en _try_load_xgb
        booster = _try_load_xgb()
        dm = xgb.DMatrix(X)
        prob = booster.predict(dm)
    elif model_name == "lgbm":
        booster = _try_load_lgbm()
        prob = booster.predict(X)
    else:
        raise ValueError("Modelo no soportado.")

    out = df_base.copy()
    out["_prob"]  = np.asarray(prob, dtype="float64")
    out["_score"] = out["_prob"].clip(0,1)
    out = out.sort_values("_score", ascending=False, kind="mergesort")
    return out

In [252]:
#SinergIA0580
# === Panel GUI: Modelos de análisis (selector de modelo + heurístico calibrable) ===
import ipywidgets as W
from IPython.display import display, clear_output, HTML

def modelos_make_panel(gui_refs: dict|None=None):
    """
    Acordeón 'Modelos de análisis' con:
      - Selector de modelo (con descripciones en la etiqueta)
      - Controles del heurístico (pesos, normalización, half-life, Árbol)
      - Botones Guardar/Restablecer/Aplicar ahora
    Si gui_refs incluye: STATE, out_log, out_aws, right_tabs, se aplica y refresca pestaña Entrada_AWS.
    """
    # === Selector de modelo con descripciones cortas
    # label visible · value interno
    w_model = W.Dropdown(
        description="Modelo:",
        options=[
            ("Modelo asistido (Heurístico) · interpretable y editable", "heuristico"),
            ("XGBoost aprendido · no lineal rápido para ranking", "xgb"),
            ("LightGBM · árboles eficientes para grandes volúmenes", "lgbm"),
            ("Random Forest · baseline robusto con poca afinación", "rf"),
        ],
        value="heuristico",
        layout=W.Layout(width="100%")
    )

    # === Controles del heurístico (cargamos defaults)
    p0 = modelcfg_load_params()
    w_w_contact = W.FloatSlider(description="w_contact", value=p0["w_contact"], min=0, max=1, step=0.05, readout_format=".2f")
    w_w_saldo   = W.FloatSlider(description="w_saldo",   value=p0["w_saldo"],   min=0, max=1, step=0.05, readout_format=".2f")
    w_w_mora    = W.FloatSlider(description="w_mora",    value=p0["w_mora"],    min=0, max=1, step=0.05, readout_format=".2f")
    w_norm      = W.Dropdown(description="Saldo · norm", options=[("log1p","log1p"),("raíz","sqrt"),("min–max","minmax")],
                             value=p0.get("saldo_norm","log1p"))
    w_half      = W.BoundedIntText(description="Mora · half-life días", value=p0.get("mora_half_life_days",15), min=1, max=120)
    w_tree      = W.Checkbox(description="Usar ÁrbolHeuristico", value=bool(p0.get("use_tree", True)))
    w_cap       = W.FloatSlider(description="Cap |rules|", value=p0.get("max_rule_weight_abs",0.5), min=0, max=1, step=0.05)

    # mostrar/ocultar controles del heurístico según modelo
    heur_box = W.VBox([
        W.HBox([w_w_contact, w_w_saldo, w_w_mora]),
        W.HBox([w_norm, w_half]),
        W.HBox([w_tree, w_cap]),
    ])
    def _toggle_heur(_=None):
        heur_box.layout.display = "" if w_model.value == "heuristico" else "none"
    _toggle_heur()
    w_model.observe(_toggle_heur, "value")

    # Botones
    btn_save  = W.Button(description="Guardar", button_style="success")
    btn_reset = W.Button(description="Restablecer")
    btn_apply = W.Button(description="Aplicar ahora", button_style="info")
    out_msg   = W.Output()

    def _widgets_to_params():
        return {
            "w_contact": float(w_w_contact.value),
            "w_saldo":   float(w_w_saldo.value),
            "w_mora":    float(w_w_mora.value),
            "saldo_norm": str(w_norm.value),
            "mora_half_life_days": int(w_half.value),
            "use_tree": bool(w_tree.value),
            "max_rule_weight_abs": float(w_cap.value),
        }

    def _on_save(_):
        with out_msg:
            out_msg.clear_output()
            try:
                fid = modelcfg_save_params(_widgets_to_params())
                print(f"Parámetros guardados en MODELOS · file_id={fid}")
            except Exception as e:
                print("[ERROR] Guardando parámetros:", e)

    def _on_reset(_):
        with out_msg:
            out_msg.clear_output()
            try:
                p = modelcfg_default_params()
                w_w_contact.value = p["w_contact"]
                w_w_saldo.value   = p["w_saldo"]
                w_w_mora.value    = p["w_mora"]
                w_norm.value      = p["saldo_norm"]
                w_half.value      = p["mora_half_life_days"]
                w_tree.value      = p["use_tree"]
                w_cap.value       = p["max_rule_weight_abs"]
                print("Parámetros restablecidos (no guardado).")
            except Exception as e:
                print("[ERROR] Reset:", e)

    def _on_apply(_):
        if gui_refs is None:
            with out_msg:
                out_msg.clear_output()
                print("Aplicación local: necesito gui_refs para refrescar la pestaña Entrada_AWS.")
            return
        STATE     = gui_refs.get("STATE")
        out_log   = gui_refs.get("out_log")
        out_aws   = gui_refs.get("out_aws")
        right_tab = gui_refs.get("right_tabs")
        with out_log:
            out_log.clear_output()
            try:
                df_base = STATE.get("Entrada_AWS") if STATE.get("Entrada_AWS") is not None else STATE.get("Rutina")
                if df_base is None or df_base.empty:
                    print("No hay datos en memoria.  Previsualiza y construye Entrada_AWS primero.")
                    return

                if w_model.value == "heuristico":
                    heur_df = load_arbol_heuristico() if w_tree.value else pd.DataFrame()
                    params  = _widgets_to_params()
                    scored  = apply_model_heuristico(df_base, heur_df=heur_df, params=params)
                else:
                    # Modelos aprendidos
                    scored = apply_model_ml(df_base, w_model.value)

                STATE["Entrada_AWS"] = scored
                with out_aws:
                    out_aws.clear_output()
                    display(HTML(f"<b>Entrada_AWS ({w_model.label.split('·')[0].strip()})</b>"))
                    display(scored.head(300))
                right_tab.selected_index = 3
                print("Aplicado OK.  Ranking actualizado.")
            except Exception as e:
                print("[ERROR] Aplicando modelo:", e)

    btn_save.on_click(_on_save)
    btn_reset.on_click(_on_reset)
    btn_apply.on_click(_on_apply)

    panel_heur = W.VBox([
        W.HTML("<b>Modelos de análisis</b>"),
        w_model,
        heur_box,
        W.HBox([btn_save, btn_reset, btn_apply]),
        out_msg
    ])

    acc = W.Accordion(children=[panel_heur])
    acc.set_title(0, "Modelos de análisis")
    acc.selected_index = None
    return acc

# Montaje (igual que antes)
try:
    gui_refs  # si ya existe por ejecución previa
except NameError:
    gui_refs = None
panel_modelos = modelos_make_panel(gui_refs)
display(panel_modelos)

Accordion(children=(VBox(children=(HTML(value='<b>Modelos de análisis</b>'), Dropdown(description='Modelo:', l…

##Modelado de la GUI (0600-0799)

In [253]:
#SinergIA0600
# GUI — panel Ley2300 (acordeón) y wiring a exportación
import ipywidgets as W

def l2300_make_panel():
    p = l2300_load_params()
    w_recontact = W.IntSlider(description="Recontacto (días)", value=p["recontact_days"], min=1, max=14)
    w_wk_s = W.Text(value=p["wk_start"], description="L-V desde");  w_wk_e = W.Text(value=p["wk_end"], description="L-V hasta")
    w_sa_s = W.Text(value=p["sat_start"], description="Sáb desde"); w_sa_e = W.Text(value=p["sat_end"], description="Sáb hasta")
    w_sun   = W.Checkbox(value=p["allow_sun"], description="Permitir domingo")
    w_onech = W.Checkbox(value=p.get("one_channel_per_week", True), description="1 canal por semana")
    save_b  = W.Button(description="Guardar Ley2300")

    def _save(_=None):
        params = {
            "recontact_days": int(w_recontact.value),
            "wk_start": w_wk_s.value, "wk_end": w_wk_e.value,
            "sat_start": w_sa_s.value, "sat_end": w_sa_e.value,
            "allow_sun": bool(w_sun.value),
            "one_channel_per_week": bool(w_onech.value),
            "cooldown_by_result": l2300_load_params().get("cooldown_by_result", {}),
            "holidays": l2300_load_params().get("holidays", [])
        }
        l2300_save_params(params)
    save_b.on_click(_save)

    box = W.Accordion(children=[W.VBox([
        w_recontact, W.HBox([w_wk_s, w_wk_e]), W.HBox([w_sa_s, w_sa_e]), W.HBox([w_sun, w_onech]), save_b
    ])])
    box.set_title(0, "Ley2300")
    return box

In [254]:
#SinergIA0620
# GUI v7 — añade selector de detección de columnas (Automática / Estructuras)
import ipywidgets as W
from IPython.display import display, HTML, clear_output
from google.colab import files as _colab_files
import datetime as _dt
import io, csv, os

def render_sinergia_gui_like_gradio():
    display(HTML("""
    <style>
      .sg-card { border:1px solid #3d3f44; border-radius:18px; padding:14px; box-shadow:0 3px 12px rgba(0,0,0,.18); }
      .sg-title { font-weight:700; font-size:22px; margin:2px 0 8px; }
      .sg-sub   { color:#a6adb4; font-size:12px; margin:0 0 8px; }
      .sg-btn  button { border-radius:999px !important; }
    </style>
    """))

    def _render_table(cols, rows, max_rows=300):
        head = "".join(f"<th style='padding:6px 8px;border-bottom:1px solid #ddd'>{c}</th>" for c in cols)
        body = []
        for r in rows[:max_rows]:
            tds = "".join(f"<td style='padding:4px 8px;border-bottom:1px solid #eee'>{'' if v is None else v}</td>" for v in r)
            body.append(f"<tr>{tds}</tr>")
        return HTML(f"""
          <div style='max-height:480px;overflow:auto;border:1px solid #aaa;border-radius:10px'>
            <table style='border-collapse:collapse;width:100%'>
              <thead style='position:sticky;top:0;background:#f5f5f5'><tr>{head}</tr></thead>
              <tbody>{''.join(body)}</tbody>
            </table>
          </div>
        """)

    def _make_source_box(default_folder_id: str, subtitulo: str):
        origen = W.ToggleButtons(options=["Drive","Local"], value="Drive", description="Origen:")
        folder = W.Text(value=default_folder_id, description="Folder ID:", layout=W.Layout(flex="1 1 auto"))
        refresh = W.Button(description="Actualizar", layout=W.Layout(width="130px"))
        files = W.Dropdown(options=[], description="Archivo:", layout=W.Layout(width="100%"))
        upload = W.FileUpload(accept=".csv,.xlsx,.xls", multiple=False, description="Subir local")
        def _toggle(change):
            drive = (change["new"] == "Drive")
            folder.layout.display  = "" if drive else "none"
            refresh.layout.display = "" if drive else "none"
            files.layout.display   = "" if drive else "none"
            upload.layout.display  = "none" if drive else ""
        origen.observe(_toggle, names="value"); _toggle({"new":"Drive"})
        box = W.VBox([W.HTML(f"<div class='sg-sub'>{subtitulo}</div>"),
                       origen, W.HBox([folder, refresh]), files, upload])
        return {"origen": origen, "folder": folder, "refresh": refresh, "files": files, "upload": upload, "box": box}

    # pestañas izquierda
    rut = _make_source_box(RUTINAS_ID, "Rutina · IN_SISTECREDITO")
    aws = _make_source_box(AWS_LOG_ID, "AWS · Logs/Base")
    pro = _make_source_box(PROMESAS_ID, "Promesas")
    pag = _make_source_box(PAGOS_ID, "Pagos")
    left_tabs = W.Tab(children=[rut["box"], aws["box"], pro["box"], pag["box"]])
    for i, t in enumerate(["Rutina","AWS","Promesas","Pagos"]): left_tabs.set_title(i, t)

    # derecha
    out_prev, out_log, out_clean = W.Output(), W.Output(), W.Output()
    right_tabs = W.Tab(children=[out_prev, out_log, out_clean])
    for i, t in enumerate(["Preview / IN","Log","Datos limpios"]): right_tabs.set_title(i, t)

    # acciones (filas compactas + Ley2300 integrado)
    btn_preview  = W.Button(description="Previsualizar");             btn_preview.add_class("sg-btn")
    btn_validate = W.Button(description="Validar estructura");        btn_validate.add_class("sg-btn")
    btn_clean    = W.Button(description="Aplicar limpieza");          btn_clean.add_class("sg-btn")
    btn_savecsv  = W.Button(description="Guardar CSV (muestra)");     btn_savecsv.add_class("sg-btn")
    btn_export   = W.Button(description="Exportar CSV completo");     btn_export.add_class("sg-btn")
    chk_clean_exp= W.Checkbox(value=True, description="Limpiar al exportar completo")
    det_cols     = W.ToggleButtons(options=["Automática","Estructuras"], value="Estructuras", description="Detección:")
    btn_genesys  = W.Button(description="Exportar Genesys CSV");      btn_genesys.add_class("sg-btn")
    btn_dwl      = W.Button(description="Descargar local (muestra)"); btn_dwl.add_class("sg-btn")
    fmt_dd       = W.Dropdown(
        options=[("CSV (coma)","csv_comma"), ("CSV (punto y coma)","csv_semicolon"), ("XLSX","xlsx")],
        value="csv_comma", description="Formato:"
    )

    # filas de acciones
    actions = W.VBox([
        W.HBox([btn_preview, btn_validate, btn_clean], layout=W.Layout(gap="8px")),
        W.HBox([btn_savecsv, btn_export, chk_clean_exp], layout=W.Layout(gap="8px")),
        W.HBox([det_cols, btn_genesys], layout=W.Layout(gap="8px")),
        W.HBox([fmt_dd, btn_dwl], layout=W.Layout(gap="8px")),
    ])

    # panel Ley2300 (ya tienes l2300_make_panel definido)
    panel_l2300 = l2300_make_panel()

    # layout principal — ancho un poco mayor para que quepan los botones
    col_left  = W.VBox([
        W.HTML("<div class='sg-title'>Cargar archivos primarios</div>"),
        left_tabs,
        actions,
        panel_l2300,   # ← aquí queda embebido
    ], layout=W.Layout(width="520px"))

    col_right = W.VBox([right_tabs], layout=W.Layout(flex="1 1 auto"))

    display(HTML("<div class='sg-card'>"))
    display(W.VBox([
        W.HTML("<div class='sg-title'>SinergIA Cobros</div>"),
        W.HBox([col_left, W.HTML("<div style='width:16px'></div>"), col_right])
    ]))
    display(HTML("</div>"))

    _last = {"name": None, "cols": [], "rows": [], "clean": [], "tab": 0}

    def _refresh(tab):
        with out_log: clear_output(wait=True)
        try:
            items = drive_list_recent(tab["folder"].value, top=12)
            tab["files"].options = [(f'{it["name"]}  —  {it.get("modifiedTime","")}', it["id"]) for it in items] or []
            with out_log: print(f"Listados {len(tab['files'].options)} archivos.")
        except Exception as e:
            with out_log: print("[ERROR]", e)

    for t in (rut, aws, pro, pag):
        t["refresh"].on_click(lambda _=None, tt=t: _refresh(tt))

    def _cur_tab():
        idx = left_tabs.selected_index or 0
        return idx, [rut, aws, pro, pag][idx]

    # acciones
    def _do_preview():
        idx, tab = _cur_tab()
        with out_log: clear_output(wait=True)
        with out_prev: clear_output(wait=True)
        with out_clean: clear_output(wait=True)
        try:
            if tab["origen"].value == "Drive":
                if not tab["files"].value:
                    with out_log: print("Seleccione un archivo del listado."); return
                name, data = drive_download_file(tab["files"].value)
                cols, rows = read_preview_any((name, data), nrows=300)
            else:
                if not tab["upload"].value:
                    with out_log: print("Suba un archivo local .csv/.xlsx."); return
                up = list(tab["upload"].value.values())[0]
                name = up["metadata"]["name"]; data = up["content"]
                cols, rows = read_preview_any((name, data), nrows=300)

            _last.update({"name": name, "cols": cols, "rows": rows, "clean": [], "tab": idx})
            with out_log: print(f"Preview de '{name}' · Filas {len(rows)} · Columnas {len(cols)}")
            with out_prev: display(_render_table(cols, rows))
            right_tabs.selected_index = 0
        except Exception as e:
            with out_log: print("[ERROR]", e); right_tabs.selected_index = 1

    def _guess_archivo_logico(name: str, tab_idx: int) -> str:
        return _guess_archivo_logico_from_tab(tab_idx, name)

    def _do_validate():
        with out_log: clear_output(wait=True)
        if not _last["cols"]:
            with out_log: print("Genere una preview antes de validar."); return
        archivo_logico = _guess_archivo_logico(_last["name"], _last["tab"])
        expected = get_column_names_from_estructuras(archivo_logico)
        if not expected:
            with out_log: print(f"No hay definición en 'Estructuras' para '{archivo_logico}'."); return
        cols = [str(c) for c in _last["cols"]]
        missing = [c for c in expected if c not in cols]
        extras  = [c for c in cols if c not in expected]
        if not missing and not extras:
            with out_log: print(f"Estructura esperada: {archivo_logico} · columnas {len(expected)}")
            print("Validación OK: las columnas coinciden.")
        else:
            with out_log:
                print(f"Desalineado vs {archivo_logico}.  Esperadas={len(expected)} · Leídas={len(cols)}")
                if missing: print("Faltan:", ", ".join(missing))
                if extras:  print("Sobran:", ", ".join(extras))
        right_tabs.selected_index = 1

    def _do_clean():
        with out_log: clear_output(wait=True)
        with out_clean: clear_output(wait=True)
        if not _last["rows"]:
            with out_log: print("Genere una preview antes de limpiar."); return
        try:
            cleaned = apply_cleaning(_last["cols"], _last["rows"], load_unicode_whitelist())
            _last["clean"] = cleaned
            with out_clean: display(_render_table(_last["cols"], cleaned))
            with out_log: print("Limpieza aplicada sobre la muestra.  Guardar/descargar usará la versión limpia.")
            right_tabs.selected_index = 2
        except Exception as e:
            with out_log: print("[ERROR] Limpieza:", e); right_tabs.selected_index = 1

    def _choose_rows_for_output():
        return _last["clean"] if _last["clean"] else _last["rows"]

    def _do_savecsv():
        with out_log: clear_output(wait=True)
        if not _last["rows"]:
            with out_log: print("Genere una preview antes de guardar."); return
        cols, rows = _last["cols"], _choose_rows_for_output()
        base = (_last["name"] or "data").rsplit(".", 1)[0]
        ts   = _dt.datetime.now().strftime("%Y%m%d_%H%M%S")

        csv_bytes = table_to_csv_bytes(cols, rows, sep=",")
        try:
            path_csv = f"/content/preview_{base}_{ts}.csv"
            with open(path_csv, "wb") as f: f.write(csv_bytes)
            fid = drive_upload_path(SALIDAS_ID, path_csv, "text/csv")
            with out_log: print(f"CSV (muestra) subido a 'Salidas' (file_id={fid}).")
        except Exception as e:
            with out_log: print("[WARN] No se pudo subir muestra a Salidas:", e)

        fmt = fmt_dd.value
        try:
            if fmt == "csv_comma":
                lp = f"/content/preview_{base}_{ts}.csv"
                with open(lp, "wb") as f: f.write(csv_bytes)
            elif fmt == "csv_semicolon":
                data = table_to_csv_bytes(cols, rows, sep=";")
                lp = f"/content/preview_{base}_{ts}_semicolon.csv"
                with open(lp, "wb") as f: f.write(data)
            else:
                data = table_to_xlsx_bytes(cols, rows)
                lp = f"/content/preview_{base}_{ts}.xlsx"
                with open(lp, "wb") as f: f.write(data)
            _colab_files.download(lp)
            with out_log: print("Descarga local iniciada.")
        except Exception as e:
            with out_log: print("[ERROR] Descarga local:", e)
        right_tabs.selected_index = 1

    def _do_export_full():
        with out_log: clear_output(wait=True)
        idx, tab = _cur_tab()
        if tab["origen"].value != "Drive":
            with out_log: print("Para exportar completo, seleccione un archivo desde Drive."); return
        if not tab["files"].value:
            with out_log: print("Seleccione un archivo del listado."); return
        try:
            fid, local_path, wrote = export_full_csv_with_optional_clean(tab["files"].value, chk_clean_exp.value)
            print(f"Exportado completo → 'Salidas' (file_id={fid}) · Filas: {wrote}")
            _colab_files.download(local_path)
        except Exception as e:
            print("[ERROR] al exportar completo:", e)
        right_tabs.selected_index = 1

        def _do_genesys():
            with out_log: clear_output(wait=True)
            with out_clean: clear_output(wait=True)
            idx, tab = _cur_tab()
            if tab["origen"].value != "Drive":
                with out_log: print("Para Genesys, seleccione un archivo desde Drive."); return
            if not tab["files"].value:
                with out_log: print("Seleccione un archivo del listado."); return
            try:
                # hint del archivo lógico según pestaña/archivo
                name = ""
                try:
                    name = next(lbl for lbl, fid in tab["files"].options if fid == tab["files"].value)
                    name = name.split("  —  ")[0]
                except Exception:
                    pass
                hint = _guess_archivo_logico_from_tab(idx, name)
                prefer = (det_cols.value == "Estructuras")

                # >>> LEE PARÁMETROS ACTUALES DEL PANEL (JSON)
                params = _widgets_to_ley()  # toma lo que el usuario vea en pantalla

                # >>> PASA 'params' AL EXPORT (ver siguiente paso para habilitarlo)
                fid, local_path, g_cols, g_rows, tot = export_genesys_csv(
                    tab["files"].value, chk_clean_exp.value, prefer,
                    archivo_logico_hint=hint, params=params
                )
                with out_log: print(f"Genesys CSV generado → 'Salidas' (file_id={fid}) · Filas: {tot}")
                with out_clean: display(_render_table(g_cols, g_rows))
                _colab_files.download(local_path)
                right_tabs.selected_index = 2
            except Exception as e:
                with out_log: print("[ERROR] Genesys:", e); right_tabs.selected_index = 1


    btn_preview.on_click(lambda _: _do_preview())
    btn_validate.on_click(lambda _: _do_validate())
    btn_clean.on_click(lambda _: _do_clean())
    btn_savecsv.on_click(lambda _: _do_savecsv())
    btn_export.on_click(lambda _: _do_export_full())
    btn_genesys.on_click(lambda _: _do_genesys())

In [255]:
#SinergIA0640
# v4.1 — GUI ipywidgets estable: preview/validación/guardar-muestra/exportar-completo/descargar-local
def render_sinergia_gui_like_gradio():
    import ipywidgets as W
    from IPython.display import display, HTML, clear_output
    import csv, io, datetime as _dt

    # ---------- estilo ----------
    display(HTML("""
    <style>
      .sg-card { border:1px solid #3d3f44; border-radius:18px; padding:14px; box-shadow:0 3px 12px rgba(0,0,0,.18); }
      .sg-title { font-weight:700; font-size:22px; margin:2px 0 8px; }
      .sg-sub   { color:#a6adb4; font-size:12px; margin:0 0 8px; }
      .sg-btn  button { border-radius:999px !important; }
    </style>
    """))

    # ---------- helpers de tabla ----------
    def _render_table(cols, rows, max_rows=300):
        head = "".join(f"<th style='padding:6px 8px;border-bottom:1px solid #ddd'>{c}</th>" for c in cols)
        body = []
        for r in rows[:max_rows]:
            tds = "".join(
                f"<td style='padding:4px 8px;border-bottom:1px solid #eee'>{'' if v is None else v}</td>" for v in r
            )
            body.append(f"<tr>{tds}</tr>")
        return HTML(
            "<div style='max-height:480px;overflow:auto;border:1px solid #aaa;border-radius:10px'>"
            "<table style='border-collapse:collapse;width:100%'>"
            f"<thead style='position:sticky;top:0;background:#f5f5f5'><tr>{head}</tr></thead>"
            f"<tbody>{''.join(body)}</tbody>"
            "</table></div>"
        )

    def _make_source_box(default_folder_id: str, subtitulo: str):
        origen  = W.ToggleButtons(options=["Drive","Local"], value="Drive", description="Origen:")
        folder  = W.Text(value=default_folder_id, description="Folder ID:", layout=W.Layout(flex="1 1 auto"))
        refresh = W.Button(description="Actualizar", layout=W.Layout(width="130px"))
        files   = W.Dropdown(options=[], description="Archivo:", layout=W.Layout(width="100%"))
        upload  = W.FileUpload(accept=".csv,.xlsx,.xls", multiple=False, description="Subir local")

        def _toggle(change):
            drive = (change["new"] == "Drive")
            folder.layout.display  = "" if drive else "none"
            refresh.layout.display = "" if drive else "none"
            files.layout.display   = "" if drive else "none"
            upload.layout.display  = "none" if drive else ""
        origen.observe(_toggle, names="value"); _toggle({"new":"Drive"})

        box = W.VBox([W.HTML(f"<div class='sg-sub'>{subtitulo}</div>"),
                       origen, W.HBox([folder, refresh]), files, upload])
        return {"origen": origen, "folder": folder, "refresh": refresh, "files": files, "upload": upload, "box": box}

    # ---------- pestañas izquierda ----------
    rut = _make_source_box(globals().get("RUTINAS_ID",""),    "Rutina · IN_SISTECREDITO")
    aws = _make_source_box(globals().get("AWS_LOG_ID",""),    "AWS · Logs/Base")
    pro = _make_source_box(globals().get("PROMESAS_ID",""),   "Promesas")
    pag = _make_source_box(globals().get("PAGOS_ID",""),      "Pagos")
    left_tabs = W.Tab(children=[rut["box"], aws["box"], pro["box"], pag["box"]])
    for i, t in enumerate(["Rutina","AWS","Promesas","Pagos"]): left_tabs.set_title(i, t)

    # ---------- derecha ----------
    out_preview, out_log, out_clean = W.Output(), W.Output(), W.Output()
    right_tabs = W.Tab(children=[out_preview, out_log, out_clean])
    for i, t in enumerate(["Preview / IN","Log","Datos limpios"]): right_tabs.set_title(i, t)

    # ---------- acciones ----------
    btn_preview   = W.Button(description="Previsualizar");              btn_preview.add_class("sg-btn")
    btn_validate  = W.Button(description="Validar estructura");         btn_validate.add_class("sg-btn")
    btn_savecsv   = W.Button(description="Guardar CSV (muestra)");     btn_savecsv.add_class("sg-btn")
    btn_export    = W.Button(description="Exportar CSV completo");      btn_export.add_class("sg-btn")
    btn_dwl       = W.Button(description="Descargar local (muestra)");  btn_dwl.add_class("sg-btn")
    fmt_dd        = W.Dropdown(
        options=[("CSV (coma)", "csv_comma"), ("CSV (punto y coma)", "csv_semicolon"), ("XLSX", "xlsx")],
        value="csv_comma", description="Formato:"
    )

    # ---------- layout principal ----------
    col_left  = W.VBox(
        [W.HTML("<div class='sg-title'>Cargar archivos primarios</div>"),
         left_tabs,
         W.HBox([btn_preview, btn_validate, btn_savecsv]),
         W.HBox([btn_export, fmt_dd, btn_dwl])],
        layout=W.Layout(width="360px")
    )
    col_right = W.VBox([right_tabs], layout=W.Layout(flex="1 1 auto"))
    display(HTML("<div class='sg-card'>"))
    display(W.VBox([
        W.HTML("<div class='sg-title'>SinergIA Cobros</div>"),
        W.HBox([col_left, W.HTML("<div style='width:16px'></div>"), col_right])
    ]))
    display(HTML("</div>"))

    # ---------- estado de la vista ----------
    _last = {"name": None, "cols": [], "rows": [], "tab": 0}

    # ---------- lógica ----------
    def _refresh(tab):
        with out_log: clear_output(wait=True)
        try:
            f_list = globals().get("drive_list_recent")
            if not callable(f_list):
                print("Falta drive_list_recent()."); return
            items = f_list(tab["folder"].value, top=12)
            tab["files"].options = [(f'{it.get("name","")}  —  {it.get("modifiedTime","")}', it.get("id","")) for it in items] or []
            print(f"Listados {len(tab['files'].options)} archivos.")
        except Exception as e:
            print("[ERROR]", e)

    for t in (rut, aws, pro, pag):
        t["refresh"].on_click(lambda _=None, tt=t: _refresh(tt))

    def _cur_tab():
        idx = left_tabs.selected_index or 0
        return idx, [rut, aws, pro, pag][idx]

    def _do_preview():
        with out_log: clear_output(wait=True)
        with out_preview: clear_output(wait=True)
        with out_clean: clear_output(wait=True)
        try:
            idx, tab = _cur_tab()
            if tab["origen"].value == "Drive":
                if not tab["files"].value:
                    with out_log: print("Seleccione un archivo del listado."); return
                dld = globals().get("drive_download_file")
                if not callable(dld):
                    with out_log: print("Falta drive_download_file()."); return
                name, data = dld(tab["files"].value)
                rp = globals().get("read_preview_any")
                cols, rows = rp((name, data), nrows=300)
            else:
                if not tab["upload"].value:
                    with out_log: print("Suba un archivo local .csv/.xlsx."); return
                up = list(tab["upload"].value.values())[0]
                name = up["metadata"]["name"]; data = up["content"]
                rp = globals().get("read_preview_any")
                cols, rows = rp((name, data), nrows=300)

            _last.update({"name": name, "cols": cols, "rows": rows, "tab": idx})
            with out_log: print(f"Preview de '{name}' · Filas {len(rows)} · Columnas {len(cols)}")
            with out_preview: display(_render_table(cols, rows))
            with out_clean:   display(_render_table(cols, rows))
            right_tabs.selected_index = 0
        except Exception as e:
            with out_log: print("[ERROR]", e); right_tabs.selected_index = 1

    def _guess_archivo_logico(name: str, tab_idx: int) -> str:
        n = (name or "").upper()
        if tab_idx == 0:  # Rutina
            return "TER_UNISONO" if "TER" in n else "IN_SISTECREDITO"
        if tab_idx == 1:  # AWS
            return "ENTRADA_AWS"
        if tab_idx == 2:  # Promesas
            return "PROMESAS"
        if tab_idx == 3:  # Pagos
            return "PAGOS"
        return "IN_SISTECREDITO"

    def _do_validate():
        with out_log: clear_output(wait=True)
        if not _last["cols"]:
            with out_log: print("Genere una preview antes de validar."); return
        archivo_logico = _guess_archivo_logico(_last["name"], _last["tab"])
        get_cols = globals().get("get_column_names_from_estructuras")
        if not callable(get_cols):
            with out_log: print("Falta get_column_names_from_estructuras()."); return
        expected = get_cols(archivo_logico)
        if not expected:
            with out_log: print(f"No hay definición en 'Estructuras' para '{archivo_logico}'."); return
        cols = [str(c) for c in _last["cols"]]
        missing = [c for c in expected if c not in cols]
        extras  = [c for c in cols if c not in expected]
        if not missing and not extras:
            with out_log: print(f"Validación OK: {archivo_logico} coincide ({len(cols)} columnas).")
        else:
            with out_log:
                print(f"Desalineado vs {archivo_logico}.  Esperadas={len(expected)} · Leídas={len(cols)}")
                if missing: print("Faltan:", ", ".join(missing))
                if extras:  print("Sobran:", ", ".join(extras))
        right_tabs.selected_index = 1

    def _do_savecsv():
        with out_log: clear_output(wait=True)
        if not _last["rows"]:
            with out_log: print("Genere una preview antes de guardar."); return
        try:
            buf = io.StringIO()
            w = csv.writer(buf, lineterminator="\n")
            w.writerow(_last["cols"]); [w.writerow(r) for r in _last["rows"]]
            data = buf.getvalue().encode("utf-8")
            ts = _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
            base = (_last["name"] or "data").rsplit(".",1)[0]
            name = f"preview_{base}_{ts}.csv"
            upb  = globals().get("drive_upload_bytes")
            fid = upb(globals().get("SALIDAS_ID"), name, data, "text/csv")
            with out_log: print(f"CSV guardado en Salidas · file_id={fid}")
        except Exception as e:
            with out_log: print("[ERROR] al subir CSV a Salidas:", e)
        right_tabs.selected_index = 1

    def _do_export_full():
        with out_log: clear_output(wait=True)
        if not _last["rows"]:
            print("Genere una preview antes de exportar."); right_tabs.selected_index = 1; return
        try:
            fmt  = fmt_dd.value  # "csv_comma" | "csv_semicolon" | "xlsx"
            cols, rows = _last["cols"], _last["rows"]

            if fmt == "xlsx":
                to_bytes = globals().get("table_to_xlsx_bytes")
                data = to_bytes(cols, rows)
                mime, ext = "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet", "xlsx"
            else:
                sep = "," if fmt == "csv_comma" else ";"
                to_bytes = globals().get("table_to_csv_bytes")
                data = to_bytes(cols, rows, sep=sep)
                mime, ext = "text/csv", "csv"

            base = (_last["name"] or "data").rsplit(".", 1)[0]
            name = f"export_full_{base}.{ext}"
            upb  = globals().get("drive_upload_bytes")
            fid  = upb(globals().get("SALIDAS_ID"), name, data, mime)
            print(f"Export completo OK → Salidas: {name} (file_id={fid})")
        except Exception as e:
            print("[ERROR] Export completo:", e)
        right_tabs.selected_index = 1

    def _do_download_local():
        with out_log: clear_output(wait=True)
        if not _last["rows"]:
            print("Genere una preview antes de descargar."); right_tabs.selected_index = 1; return
        try:
            from google.colab import files
            to_bytes = globals().get("table_to_csv_bytes")
            data = to_bytes(_last["cols"], _last["rows"], sep=",")
            tmp = "/tmp/preview_sample.csv"
            open(tmp, "wb").write(data)
            files.download(tmp)
            print("Descarga iniciada (preview_sample.csv).")
        except Exception as e:
            print("[ERROR] Descarga local:", e)
        right_tabs.selected_index = 1

    # ---------- eventos ----------
    btn_preview.on_click(lambda _: _do_preview())
    btn_validate.on_click(lambda _: _do_validate())
    btn_savecsv.on_click(lambda _: _do_savecsv())
    btn_export.on_click(lambda _: _do_export_full())
    btn_dwl.on_click(lambda _: _do_download_local())

    # Retorno opcional (por si luego quieres referenciar widgets)
    return {
        "left_tabs": left_tabs, "right_tabs": right_tabs,
        "out_preview": out_preview, "out_log": out_log, "out_clean": out_clean
    }

In [256]:
#SinergIA0660
# =========================
# GUI v8 — con Ley2300 embebida + pestañas: Entrada AWS, Teletón, Listas
# Reemplaza por completo la definición anterior de render_sinergia_gui_like_gradio
# =========================
import ipywidgets as W
from IPython.display import display, HTML, clear_output
import pandas as pd, numpy as np
import io, csv, datetime as _dt

def render_sinergia_gui_like_gradio():
    # ---- Estilos
    display(HTML("""
    <style>
      .sg-card { border:1px solid #3d3f44; border-radius:18px; padding:14px; box-shadow:0 3px 12px rgba(0,0,0,.18); }
      .sg-title { font-weight:700; font-size:22px; margin:2px 0 8px; }
      .sg-sub   { color:#a6adb4; font-size:12px; margin:0 0 8px; }
      .sg-btn  button { border-radius:999px !important; }
    </style>
    """))

    # ---- helpers de tabla
    def _render_table(cols, rows, max_rows=300):
        head = "".join(f"<th style='padding:6px 8px;border-bottom:1px solid #ddd'>{c}</th>" for c in cols)
        body = []
        for r in rows[:max_rows]:
            tds = "".join(f"<td style='padding:4px 8px;border-bottom:1px solid #eee'>{'' if v is None else v}</td>" for v in r)
            body.append(f"<tr>{tds}</tr>")
        return HTML(f"""
          <div style='max-height:480px;overflow:auto;border:1px solid #aaa;border-radius:10px'>
            <table style='border-collapse:collapse;width:100%'>
              <thead style='position:sticky;top:0;background:#f5f5f5'><tr>{head}</tr></thead>
              <tbody>{''.join(body)}</tbody>
            </table>
          </div>
        """)

    def _table_to_df(cols, rows):
        try:
            return pd.DataFrame(rows, columns=cols)
        except Exception:
            return pd.DataFrame(rows).rename(columns={i:c for i,c in enumerate(cols)})

    # ---- fuentes (Drive/Local) por pestaña
    def _make_source_box(default_folder_id, subtitulo):
        origen  = W.ToggleButtons(options=["Drive","Local"], value="Drive", description="Origen:")
        folder  = W.Text(value=default_folder_id, description="Folder ID:", layout=W.Layout(flex="1 1 auto"))
        refresh = W.Button(description="Actualizar", layout=W.Layout(width="130px"), tooltip="Listar 12 más recientes")
        files   = W.Dropdown(options=[], description="Archivo:", layout=W.Layout(width="100%"))
        upload  = W.FileUpload(accept=".csv,.xlsx,.xls", multiple=False, description="Subir local")

        def _toggle(change):
            drive = (change["new"] == "Drive")
            folder.layout.display  = "" if drive else "none"
            refresh.layout.display = "" if drive else "none"
            files.layout.display   = "" if drive else "none"
            upload.layout.display  = "none" if drive else ""

        origen.observe(_toggle, names="value")
        _toggle({"new":"Drive"})
        box = W.VBox([
            W.HTML(f"<div class='sg-sub'>{subtitulo}</div>"),
            origen, W.HBox([folder, refresh]), files, upload
        ])
        return {"origen": origen, "folder": folder, "refresh": refresh, "files": files, "upload": upload, "box": box}

    # ---- pestañas izquierda (IDs vienen de tu config)
    rut = _make_source_box(RUTINAS_ID,   "Rutina · IN_SISTECREDITO")
    aws = _make_source_box(AWS_LOG_ID,   "AWS · Logs/Base")
    pro = _make_source_box(PROMESAS_ID,  "Promesas")
    pag = _make_source_box(PAGOS_ID,     "Pagos")

    left_tabs = W.Tab(children=[rut["box"], aws["box"], pro["box"], pag["box"]])
    for i, t in enumerate(["Rutina","AWS","Promesas","Pagos"]): left_tabs.set_title(i, t)

    # ---- pestañas derecha (outputs)
    out_preview, out_log, out_clean  = W.Output(), W.Output(), W.Output()
    out_aws, out_teleton, out_listas = W.Output(), W.Output(), W.Output()

    right_tabs = W.Tab(children=[out_preview, out_log, out_clean, out_aws, out_teleton, out_listas])
    for i, t in enumerate(["Preview / IN","Log","Datos limpios","Entrada AWS","Teletón","Listas"]): right_tabs.set_title(i, t)

    # ---- acciones
    btn_preview  = W.Button(description="Previsualizar");                    btn_preview.add_class("sg-btn")
    btn_validate = W.Button(description="Validar estructura");               btn_validate.add_class("sg-btn")
    btn_clean    = W.Button(description="Aplicar limpieza");                 btn_clean.add_class("sg-btn")
    btn_savecsv  = W.Button(description="Guardar CSV (muestra)");           btn_savecsv.add_class("sg-btn")
    btn_export   = W.Button(description="Exportar CSV completo");           btn_export.add_class("sg-btn")
    btn_genesys  = W.Button(description="Exportar Genesys …");              btn_genesys.add_class("sg-btn")
    btn_dwl      = W.Button(description="Descargar local (muestra)");       btn_dwl.add_class("sg-btn")

    chk_clean_exp = W.Checkbox(value=True, description="Limpiar al exportar completo")
    det_cols      = W.ToggleButtons(options=["Automática","Estructuras"], value="Automática", description="Detección:")
    fmt_dd        = W.Dropdown(
        options=[("CSV (coma)", "csv_comma"), ("CSV (punto y coma)", "csv_semicolon"), ("XLSX", "xlsx")],
        value="csv_comma", description="Formato:"
    )

    actions = W.VBox([
        W.HBox([btn_preview, btn_validate, btn_clean]),
        W.HBox([btn_savecsv, btn_export, chk_clean_exp]),
        W.HBox([det_cols, btn_genesys]),
        W.HBox([fmt_dd,   btn_dwl]),
    ])

    # ---- Panel Ley2300 (acordeón dentro de la columna izquierda)
    def _valid_hhmm(s):
        import re; return bool(re.fullmatch(r"\d{2}:\d{2}", str(s).strip()))
    w_days     = W.BoundedIntText(description="Recontacto (días):", min=0, max=30, value=4, layout=W.Layout(width="230px"))
    w_wk_start = W.Text(value="07:00", description="L–V desde:", layout=W.Layout(width="230px"))
    w_wk_end   = W.Text(value="19:00", description="L–V hasta:", layout=W.Layout(width="230px"))
    w_sa_start = W.Text(value="08:00", description="Sáb desde:", layout=W.Layout(width="230px"))
    w_sa_end   = W.Text(value="15:00", description="Sáb hasta:", layout=W.Layout(width="230px"))
    w_sun      = W.Checkbox(value=False, description="Permitir domingo")
    w_onech    = W.Checkbox(value=True,  description="1 canal por semana")
    btn_lsave  = W.Button(description="Guardar Ley2300", button_style="success")
    btn_lreset = W.Button(description="Restablecer predeterminado")

    def _ley_to_widgets(p):
        w_days.value     = int(p.get("recontact_days", 4))
        w_wk_start.value = p.get("wk_start",  "07:00")
        w_wk_end.value   = p.get("wk_end",    "19:00")
        w_sa_start.value = p.get("sat_start", "08:00")
        w_sa_end.value   = p.get("sat_end",   "15:00")
        w_sun.value      = bool(p.get("allow_sun", False))
        w_onech.value    = bool(p.get("one_channel_per_week", True))

    def _widgets_to_ley():
        for fld in (w_wk_start, w_wk_end, w_sa_start, w_sa_end):
            if not _valid_hhmm(fld.value):
                raise ValueError(f"Hora inválida: {fld.description} '{fld.value}' (HH:MM)")
        return {
            "recontact_days": int(w_days.value),
            "wk_start": w_wk_start.value.strip(), "wk_end": w_wk_end.value.strip(),
            "sat_start": w_sa_start.value.strip(), "sat_end": w_sa_end.value.strip(),
            "allow_sun": bool(w_sun.value), "one_channel_per_week": bool(w_onech.value),
            "cooldown_by_result": {"RPC":72,"PROMESA":48,"BUZON":8,"NO_CONTESTA":4,"OCUPADO":2},
            "holidays": [],
        }

    with out_log:
        try:
            _ley_to_widgets(l2300_load_params())
        except Exception as e:
            print("[WARN] Ley2300 default:", e)

    def _on_ley_save(_):
        with out_log:
            try:
                l2300_save_params(_widgets_to_ley()); print("Ley2300 guardada (Drive).")
            except Exception as e:
                print("[ERROR] Guardando Ley2300:", e)

    def _on_ley_reset(_):
        with out_log:
            try:
                _ley_to_widgets(l2300_default_params()); print("Ley2300 restablecida (sin guardar).")
            except Exception as e:
                print("[ERROR] Reset Ley2300:", e)

    btn_lsave.on_click(_on_ley_save)
    btn_lreset.on_click(_on_ley_reset)

    panel_ley = W.Accordion(children=[W.VBox([
        W.HTML("<b>Parámetros legales de contacto</b>"),
        W.HBox([w_days]),
        W.HBox([w_wk_start, w_wk_end]),
        W.HBox([w_sa_start, w_sa_end]),
        W.HBox([w_sun, w_onech]),
        W.HBox([btn_lsave, btn_lreset]),
    ])])
    panel_ley.set_title(0, "Ley2300")
    panel_ley.selected_index = None

    # ---- layout
    col_left  = W.VBox([W.HTML("<div class='sg-title'>Cargar archivos primarios</div>"), left_tabs, actions, panel_ley],
                       layout=W.Layout(width="520px"))
    col_right = W.VBox([right_tabs], layout=W.Layout(flex="1 1 auto"))

    display(HTML("<div class='sg-card'>"))
    display(W.VBox([W.HTML("<div class='sg-title'>SinergIA Cobros</div>"),
                    W.HBox([col_left, W.HTML("<div style='width:16px'></div>"), col_right])]))
    display(HTML("</div>"))

    # ---- estado
    _state = {
        "last": {"name": None, "cols": [], "rows": [], "tab": 0},
        "entrada_df": None, "prom_df": None, "pag_df": None, "heur_df": None,
    }

    # ---- utilidades
    def _refresh(tab):
        with out_log: clear_output(wait=True)
        try:
            items = drive_list_recent(tab["folder"].value, top=12)
            tab["files"].options = [(f'{it["name"]}  —  {it.get("modifiedTime","")}', it["id"]) for it in items] or []
            with out_log: print(f"Listados {len(tab['files'].options)} archivos.")
        except Exception as e:
            with out_log: print("[ERROR] Listar:", e)

    rut["refresh"].on_click(lambda _: _refresh(rut))
    aws["refresh"].on_click(lambda _: _refresh(aws))
    pro["refresh"].on_click(lambda _: _refresh(pro))
    pag["refresh"].on_click(lambda _: _refresh(pag))

    def _cur_tab():
        idx = left_tabs.selected_index or 0
        return idx, [rut, aws, pro, pag][idx]

    def _read_any_from_tab(tab):
        if tab["origen"].value == "Drive":
            if not tab["files"].value:
                raise RuntimeError("Seleccione un archivo del listado (Drive).")
            name, data = drive_download_file(tab["files"].value)
            return name, data
        else:
            if not tab["upload"].value:
                raise RuntimeError("Suba un archivo local .csv/.xlsx.")
            up = list(tab["upload"].value.values())[0]
            return up["metadata"]["name"], up["content"]

    # ---- Árbol Heurístico
    def _load_arbol_heuristico():
        try:
            sh = gc.open_by_key(CONFIG_ID)   # si tu variable se llama distinto, cámbiala aquí
            ws = sh.worksheet("ArbolHeuristico")
            return pd.DataFrame(ws.get_all_records())
        except Exception as e:
            with out_log: print("[WARN] ÁrbolHeurístico no disponible:", e)
            return pd.DataFrame()

    # ---- Heurística muy básica (placeholder) + orden
    def _rank_entrada(df, heur_df):
        # TODO: reemplazar por tu lógica de 'ÁrbolHeurístico' (mapeos + pesos por evidencia)
        df = df.copy()
        # Ejemplos de columnas típicas que ayudan (usa las reales si existen):
        for c in ["UltimaGestion", "DiasMora", "Saldo", "Contactabilidad"]:
            if c not in df.columns: df[c] = np.nan
        # Score base simple:
        df["_score"] = (
            (df["Contactabilidad"].fillna(0).astype(float)) * 0.6
            + (df["Saldo"].replace(",", ".", regex=True).astype(str).str.replace("[^0-9.]", "", regex=True).astype(float).pow(0.25, errors="ignore").fillna(0)) * 0.2
            + (df["DiasMora"].fillna(0).astype(float).rpow(0.25, errors="ignore").fillna(0)) * 0.2
        )
        df.sort_values(by=["_score"], ascending=False, inplace=True, kind="mergesort")
        return df

    # ---- normalización de direcciones TER_UNISONO (usa helpers globales _norm_spaces / _is_empty)
    def _direccion_columns_ter(df: pd.DataFrame) -> list:
        """
        Determina qué columnas del df se consideran direcciones de cliente para TER/IN.
        Usa Config_SinergIA.Estructuras como base y cae a heurística por nombre.

        Regla: ningún encabezado se modifica.  Solo se usan los nombres para identificar
        columnas de dirección.
        """
        cols_cfg = []
        try:
            # nombres tal como están definidos en Estructuras para TER_UNISONO
            defs = get_column_names_from_estructuras("TER_UNISONO") or []
            cols_cfg = [c for c in defs if "direcci" in str(c).lower()]
        except Exception:
            cols_cfg = []

        # intersección con df real
        from_cfg = [c for c in cols_cfg if c in df.columns]

        # fallback heurístico: cualquier columna del df que contenga "direcci"
        heur = [c for c in df.columns if "direcci" in str(c).lower()]

        # unión preservando orden de aparición en df
        seen = set()
        out = []
        for c in list(from_cfg) + list(heur):
            if c not in seen and c in df.columns:
                seen.add(c)
                out.append(c)
        return out

    def _direccion_normalizada_valor(x) -> str:
        """
        Reglas de normalización de dirección (solo valores, nunca encabezados):

        1) Vacío / None -> "SIN DIRECCION"
        2) Solo números (ignorando espacios) -> "SIN DIRECCION"
        3) Números al inicio antes de la primera palabra (>=2 letras) ->
           "CERCA A " + resto de la dirección
        """
        # usamos helper global si existe
        if "_is_empty" in globals() and _is_empty(x):
            return "SIN DIRECCION"

        # usamos _norm_spaces si existe; si no, degradamos suave
        if "_norm_spaces" in globals():
            texto = _norm_spaces(x)
        else:
            texto = "" if x is None else str(x).strip()

        if texto == "":
            return "SIN DIRECCION"

        # Regla 2: solo dígitos/espacios
        solo_digitos = True
        for ch in texto:
            if not (ch.isdigit() or ch.isspace()):
                solo_digitos = False
                break
        if solo_digitos:
            return "SIN DIRECCION"

        tokens = texto.split()
        if not tokens:
            return "SIN DIRECCION"

        def _cuenta_letras(tok: str) -> int:
            cnt = 0
            for ch in tok:
                if ch.isalpha():
                    cnt += 1
            return cnt

        # primera palabra “real” (>= 2 letras)
        idx_palabra = None
        for i, tok in enumerate(tokens):
            if _cuenta_letras(tok) >= 2:
                idx_palabra = i
                break

        if idx_palabra is None:
            return "SIN DIRECCION"

        # Si antes de esa palabra solo hay números → CERCA A + resto
        if idx_palabra > 0:
            solo_prefijo_numerico = True
            for tok in tokens[:idx_palabra]:
                for ch in tok:
                    if not ch.isdigit():
                        solo_prefijo_numerico = False
                        break
                if not solo_prefijo_numerico:
                    break
            if solo_prefijo_numerico:
                resto = " ".join(tokens[idx_palabra:])
                return "CERCA A " + resto

        # Si no aplica transformación especial, devolvemos el texto normalizado
        return texto

    def _normalizar_direcciones_ter(df: pd.DataFrame) -> pd.DataFrame:
        """
        Aplica _direccion_normalizada_valor sobre todas las columnas de dirección
        detectadas en _direccion_columns_ter.

        No toca nombres de columnas.  Solo modifica valores de filas.
        """
        df = df.copy()
        cols_dir = _direccion_columns_ter(df)
        if not cols_dir:
            return df
        for c in cols_dir:
            df[c] = df[c].apply(_direccion_normalizada_valor)
        return df


    # ---- botones
    def _do_preview(_=None):
        idx, tab = _cur_tab()
        with out_log: clear_output(wait=True)
        with out_preview: clear_output(wait=True)
        with out_clean: clear_output(wait=True)
        try:
            name, data = _read_any_from_tab(tab)
            cols, rows = read_preview_any((name, data), nrows=300)
            _state["last"].update({"name": name, "cols": cols, "rows": rows, "tab": idx})
            with out_log: print(f"Preview de '{name}' · Filas {len(rows)} · Columnas {len(cols)}")
            with out_preview: display(_render_table(cols, rows))
            right_tabs.selected_index = 0
        except Exception as e:
            with out_log: print("[ERROR] Preview:", e); right_tabs.selected_index = 1

    def _guess_archivo_logico(name, tab_idx):
        n = (name or "").upper()
        if tab_idx == 0:  # Rutina
            return "TER_UNISONO" if "TER" in n else "IN_SISTECREDITO"
        if tab_idx == 1:  return "ENTRADA_AWS"
        if tab_idx == 2:  return "PROMESAS"
        if tab_idx == 3:  return "PAGOS"
        return "IN_SISTECREDITO"

    def _do_validate(_=None):
        with out_log: clear_output(wait=True)
        if not _state["last"]["cols"]:
            with out_log: print("Genere una preview antes de validar."); return
        archivo_logico = _guess_archivo_logico(_state["last"]["name"], _state["last"]["tab"])
        expected = get_column_names_from_estructuras(archivo_logico)
        if not expected:
            with out_log: print(f"No hay definición en 'Estructuras' para '{archivo_logico}'."); return
        cols = [str(c) for c in _state["last"]["cols"]]
        missing = [c for c in expected if c not in cols]
        extras  = [c for c in cols if c not in expected]
        if not missing and not extras:
            with out_log: print(f"Validación OK: {archivo_logico} ({len(cols)} columnas).")
        else:
            with out_log:
                print(f"Desalineado vs {archivo_logico}.  Esperadas={len(expected)} · Leídas={len(cols)}")
                if missing: print("Faltan:", ", ".join(missing))
                if extras:  print("Sobran:", ", ".join(extras))
        right_tabs.selected_index = 1

    def _do_clean(_=None):
        with out_log: clear_output(wait=True)
        with out_clean: clear_output(wait=True)
        with out_aws: clear_output(wait=True)
        with out_teleton: clear_output(wait=True)
        with out_listas: clear_output(wait=True)
        try:
            if not _state["last"]["rows"]:
                print("Primero previsualiza el TER/IN."); return

            # base IN (TER/IN)
            df_in = _table_to_df(_state["last"]["cols"], _state["last"]["rows"])

            # detectar qué tipo de archivo lógico es (reusa helper existente)
            archivo_logico = _guess_archivo_logico(_state["last"]["name"], _state["last"]["tab"])
            if archivo_logico in ("TER_UNISONO", "IN_SISTECREDITO"):
                # normalizamos direcciones SOLO para la rutina / IN
                df_in = _normalizar_direcciones_ter(df_in)


            # promesas/pagos si existen en memoria (última vista de sus pestañas)

            def _tab_df(tabdict):
                try:
                    name, data = _read_any_from_tab(tabdict)
                    c,r = read_preview_any((name, data), nrows=200000)
                    return _table_to_df(c,r)
                except Exception:
                    return None
            _state["prom_df"] = _tab_df(pro)
            _state["pag_df"]  = _tab_df(pag)

            heur_df = _load_arbol_heuristico()
            _state["heur_df"] = heur_df

            entrada = _rank_entrada(df_in, heur_df)   # <- aquí enchufamos la lógica real del Árbol
            _state["entrada_df"] = entrada

            # Teletón (placeholder mínimo, sin 'Infinivirt'; se edita allí y afecta orden en una versión posterior)
            tele_cols = [c for c in ["Identificacion","Upper","Celular","_score"] if c in entrada.columns]
            if not tele_cols: tele_cols = list(entrada.columns)[:8]
            teleton = entrada[tele_cols].copy()
            teleton["Infinivirt"] = ""   # se diligencia desde UI/reportes
            # Listas por canal (muy básico; se reemplaza por reglas de Árbol / Ley2300)
            listas = {
                "Voz":  teleton.rename(columns={"Celular":"ANI"}) if "Celular" in teleton.columns else teleton.copy(),
                "WhatsApp": teleton.copy(),
                "SMS": teleton.copy(),
            }

            # pintar
            with out_clean:   display(HTML("<b>Datos limpios (muestra)</b>")); display(entrada.head(300))
            with out_aws:     display(HTML("<b>Entrada AWS (ordenada)</b>"));   display(entrada.head(300))
            with out_teleton: display(HTML("<b>Teletón</b>"));                  display(teleton.head(300))
            with out_listas:
                for k,v in listas.items():
                    display(HTML(f"<b>Lista {k}</b>")); display(v.head(200))
            right_tabs.selected_index = 3
            print("Limpieza/orden: OK")
        except Exception as e:
            with out_log: print("[ERROR] Limpiar/ordenar:", e); right_tabs.selected_index = 1

    def _bytes_from_df(df, fmt):
        if fmt == "xlsx":
            bio = io.BytesIO()
            with pd.ExcelWriter(bio, engine="xlsxwriter") as xw:
                df.to_excel(xw, index=False, sheet_name="data")
            return bio.getvalue(), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet", ".xlsx"
        else:
            sep = "," if fmt=="csv_comma" else ";"
            data = df.to_csv(index=False, sep=sep, line_terminator="\n").encode("utf-8")
            return data, "text/csv", ".csv"

    def _do_export(_=None):
        with out_log: clear_output(wait=True)
        try:
            if chk_clean_exp.value:
                _do_clean()
            df = _state["entrada_df"]
            if df is None or df.empty:
                print("No hay 'Entrada AWS' en memoria."); return
            data, mime, ext = _bytes_from_df(df, fmt_dd.value)
            ts = _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
            name = f"Entrada_AWS_{ts}{ext}"
            fid = drive_upload_bytes(SALIDAS_ID, name, data, mime)
            print(f"Exportado a Salidas: {name} · file_id={fid}")
            right_tabs.selected_index = 1
        except Exception as e:
            print("[ERROR] Exportar:", e); right_tabs.selected_index = 1

    def _do_savecsv(_=None):
        with out_log: clear_output(wait=True)
        try:
            if not _state["last"]["rows"]:
                print("Genere una preview antes de guardar."); return
            buf = io.StringIO(); w = csv.writer(buf, lineterminator="\n")
            w.writerow(_state["last"]["cols"]); [w.writerow(r) for r in _state["last"]["rows"]]
            data = buf.getvalue().encode("utf-8")
            ts = _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
            base = (_state["last"]["name"] or "data").rsplit(".",1)[0]
            name = f"preview_{base}_{ts}.csv"
            fid = drive_upload_bytes(SALIDAS_ID, name, data, "text/csv")
            print(f"CSV guardado en Salidas · file_id={fid}")
        except Exception as e:
            print("[ERROR] Guardar CSV muestra:", e)

    def _do_genesys(_=None):
        with out_log: clear_output(wait=True)
        try:
            df = _state["entrada_df"]
            if df is None or df.empty:
                print("No hay 'Entrada AWS' en memoria."); return
            # Mínimo viable de columnas Genesys (ajústalo a tu layout)
            cand_cols = [c for c in ["Celular","Identificacion","Upper"] if c in df.columns]
            g = df[cand_cols].copy()
            if "Celular" in g.columns:
                g = g.rename(columns={"Celular":"ANI"})
            g["Campaign"] = "SinergIA"
            data, mime, ext = _bytes_from_df(g, "csv_comma")
            ts = _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
            name = f"Genesys_{ts}{ext}"
            fid = drive_upload_bytes(SALIDAS_ID, name, data, mime)
            print(f"Genesys CSV exportado: {name} · file_id={fid}")
        except Exception as e:
            print("[ERROR] Exportar Genesys:", e)

    # wiring
    btn_preview.on_click(_do_preview)
    btn_validate.on_click(_do_validate)
    btn_clean.on_click(_do_clean)
    btn_export.on_click(_do_export)
    btn_savecsv.on_click(_do_savecsv)
    btn_genesys.on_click(_do_genesys)

    # retorno para uso avanzado
    return {
        "left_tabs": left_tabs, "right_tabs": right_tabs,
        "out_preview": out_preview, "out_log": out_log, "out_clean": out_clean,
        "out_aws": out_aws, "out_teleton": out_teleton, "out_listas": out_listas,
        "panel_ley": panel_ley
    }

In [257]:
#SinergIA0680
# === GUI v8: integra Ley2300 en la izquierda y añade pestañas de resultados a la derecha ===
import ipywidgets as W
from IPython.display import display, HTML, clear_output
import io, csv, datetime as _dt

def render_sinergia_gui_like_gradio():
    display(HTML("""
    <style>
      .sg-card { border:1px solid #3d3f44; border-radius:18px; padding:14px; box-shadow:0 3px 12px rgba(0,0,0,.18); }
      .sg-title { font-weight:700; font-size:22px; margin:2px 0 8px; }
      .sg-sub   { color:#a6adb4; font-size:12px; margin:0 0 8px; }
      .sg-btn  button { border-radius:999px !important; }
    </style>
    """))

    def _render_table(cols, rows, max_rows=250):
        head = "".join(f"<th style='padding:6px 8px;border-bottom:1px solid #ddd'>{c}</th>" for c in cols)
        body = []
        for r in rows[:max_rows]:
            tds = "".join(f"<td style='padding:4px 8px;border-bottom:1px solid #eee'>{'' if v is None else v}</td>" for v in r)
            body.append(f"<tr>{tds}</tr>")
        return HTML(f"""
          <div style='max-height:480px;overflow:auto;border:1px solid #aaa;border-radius:10px'>
            <table style='border-collapse:collapse;width:100%'>
              <thead style='position:sticky;top:0;background:#f5f5f5'><tr>{head}</tr></thead>
              <tbody>{''.join(body)}</tbody>
            </table>
          </div>""")

    def _make_source_box(default_folder_id: str, subtitulo: str):
        origen = W.ToggleButtons(options=["Drive","Local"], value="Drive", description="Origen:")
        folder = W.Text(value=default_folder_id, description="Folder ID:", layout=W.Layout(flex="1 1 auto"))
        refresh = W.Button(description="Actualizar", layout=W.Layout(width="130px"))
        files = W.Dropdown(options=[], description="Archivo:", layout=W.Layout(width="100%"))
        upload = W.FileUpload(accept=".csv,.xlsx,.xls", multiple=False, description="Subir local")
        def _toggle(change):
            drive = (change["new"] == "Drive")
            folder.layout.display  = "" if drive else "none"
            refresh.layout.display = "" if drive else "none"
            files.layout.display   = "" if drive else "none"
            upload.layout.display  = "none" if drive else ""
        origen.observe(_toggle, names="value"); _toggle({"new":"Drive"})
        box = W.VBox([W.HTML(f"<div class='sg-sub'>{subtitulo}</div>"),
                       origen, W.HBox([folder, refresh]), files, upload])
        return {"origen": origen, "folder": folder, "refresh": refresh, "files": files, "upload": upload, "box": box}

    # === pestañas de origen (izquierda)
    rut = _make_source_box(RUTINAS_ID, "Rutina · IN_SISTECREDITO")
    aws = _make_source_box(AWS_LOG_ID, "AWS · Logs/Base")
    pro = _make_source_box(PROMESAS_ID, "Promesas")
    pag = _make_source_box(PAGOS_ID, "Pagos")
    left_tabs = W.Tab(children=[rut["box"], aws["box"], pro["box"], pag["box"]])
    for i, t in enumerate(["Rutina","AWS","Promesas","Pagos"]): left_tabs.set_title(i, t)

    # === pestañas de salida (derecha)
    out_preview, out_log = W.Output(), W.Output()
    out_aws, out_tele, out_listas = W.Output(), W.Output(), W.Output()
    right_tabs = W.Tab(children=[out_preview, out_log, out_aws, out_tele, out_listas])
    for i, t in enumerate(["Preview / IN","Log","Entrada_AWS","Teleton","Listas por canal"]): right_tabs.set_title(i, t)

    # === acciones
    btn_preview   = W.Button(description="Previsualizar");  btn_preview.add_class("sg-btn")
    btn_validate  = W.Button(description="Validar estructura"); btn_validate.add_class("sg-btn")
    btn_clean     = W.Button(description="Aplicar limpieza");  btn_clean.add_class("sg-btn")
    btn_savecsv   = W.Button(description="Guardar CSV (muestra)"); btn_savecsv.add_class("sg-btn")
    btn_export    = W.Button(description="Exportar completo"); btn_export.add_class("sg-btn")
    chk_clean_exp = W.Checkbox(value=True, description="Limpiar al exportar completo")
    det_cols      = W.ToggleButtons(options=["Automática","Estructuras"], value="Automática", description="Detección:")
    btn_genesys   = W.Button(description="Exportar Genesys …"); btn_genesys.add_class("sg-btn")
    btn_dwl       = W.Button(description="Descargar local (muestra)"); btn_dwl.add_class("sg-btn")
    fmt_dd        = W.Dropdown(options=[("CSV (coma)", "csv_comma"), ("CSV (punto y coma)", "csv_semicolon"), ("XLSX", "xlsx")],
                               value="csv_comma", description="Formato:")

    # === Ley2300 (en la izquierda, debajo de acciones)
    # usa tus funciones l2300_load_params / l2300_save_params si existen
    def l2300_default_params():
        return {"recontact_days":4, "wk_start":"07:00","wk_end":"19:00","sat_start":"08:00","sat_end":"15:00","allow_sun":False,"one_channel_per_week":True}

    try:
        params_ley = l2300_load_params()
    except Exception:
        params_ley = l2300_default_params()

    w_days     = W.BoundedIntText(description="Recontacto (días):", min=0, max=30, value=int(params_ley.get("recontact_days",4)), layout=W.Layout(width="220px"))
    w_wk_start = W.Text(value=params_ley.get("wk_start","07:00"), description="L–V inicio:", layout=W.Layout(width="220px"))
    w_wk_end   = W.Text(value=params_ley.get("wk_end","19:00"), description="L–V fin:",    layout=W.Layout(width="220px"))
    w_sa_start = W.Text(value=params_ley.get("sat_start","08:00"), description="Sáb inicio:", layout=W.Layout(width="220px"))
    w_sa_end   = W.Text(value=params_ley.get("sat_end","15:00"), description="Sáb fin:",   layout=W.Layout(width="220px"))
    w_sun      = W.Checkbox(value=bool(params_ley.get("allow_sun",False)), description="Permitir domingo")
    w_onech    = W.Checkbox(value=bool(params_ley.get("one_channel_per_week",True)), description="1 canal por semana")

    def _ley_read():
        return {"recontact_days": int(w_days.value),
                "wk_start": w_wk_start.value.strip(), "wk_end": w_wk_end.value.strip(),
                "sat_start": w_sa_start.value.strip(), "sat_end": w_sa_end.value.strip(),
                "allow_sun": bool(w_sun.value), "one_channel_per_week": bool(w_onech.value)}

    btn_lsave  = W.Button(description="Guardar Ley2300", button_style="success")
    def _on_ley_save(_):
        try:
            l2300_save_params(_ley_read())
            with out_log: print("Ley2300 guardada (OK).")
        except Exception as e:
            with out_log: print("[WARN] Guardado Ley2300:", e)
    btn_lsave.on_click(_on_ley_save)

    panel_ley = W.Accordion([W.VBox([
        W.HBox([w_days]),
        W.HBox([w_wk_start, w_wk_end]),
        W.HBox([w_sa_start, w_sa_end]),
        W.HBox([w_sun, w_onech]),
        W.HBox([btn_lsave]),
    ])])
    panel_ley.set_title(0, "Ley2300")

    actions = W.VBox([
        W.HBox([btn_preview, btn_validate, btn_clean]),
        W.HBox([btn_savecsv, btn_export, chk_clean_exp]),
        W.HBox([det_cols, btn_genesys]),
        W.HBox([fmt_dd, btn_dwl]),
        panel_ley,
    ])

    col_left  = W.VBox([W.HTML("<div class='sg-title'>Cargar archivos primarios</div>"),
                        left_tabs, actions], layout=W.Layout(width="520px"))
    col_right = W.VBox([right_tabs], layout=W.Layout(flex="1 1 auto"))
    display(HTML("<div class='sg-card'>"))
    display(W.VBox([W.HTML("<div class='sg-title'>SinergIA Cobros</div>"),
                    W.HBox([col_left, W.HTML("<div style='width:16px'></div>"), col_right])]))
    display(HTML("</div>"))

    # ===== Estado en memoria de la GUI =====
    _last = {"name": None, "cols": [], "rows": [], "tab": 0,
             "df_in": None, "df_prom": None, "df_pagos": None,
             "entrada_aws": None, "teleton": None, "listas": {}}

    def _refresh(tab):
        with out_log: clear_output(wait=True)
        try:
            items = drive_list_recent(tab["folder"].value, top=12)
            tab["files"].options = [(f'{it["name"]}  —  {it.get("modifiedTime","")}', it["id"]) for it in items] or []
            with out_log: print(f"Listados {len(tab['files'].options)} archivos.")
        except Exception as e:
            with out_log: print("[ERROR]", e)

    for t in (rut, aws, pro, pag):
        t["refresh"].on_click(lambda _btn, _t=t: _refresh(_t))

    def _cur_tab():
        idx = left_tabs.selected_index or 0
        return idx, [rut, aws, pro, pag][idx]

    def _read_any(tab):
        if tab["origen"].value == "Drive":
            if not tab["files"].value:
                raise RuntimeError("Seleccione un archivo del listado.")
            name, data = drive_download_file(tab["files"].value)
            cols, rows = read_preview_any((name, data), nrows=20000)  # suficiente para construir
            return name, cols, rows
        else:
            if not tab["upload"].value:
                raise RuntimeError("Suba un archivo local .csv/.xlsx.")
            up = list(tab["upload"].value.values())[0]
            name = up["metadata"]["name"]; data = up["content"]
            cols, rows = read_preview_any((name, data), nrows=20000)
            return name, cols, rows

    def _to_df(cols, rows):
        try:
            return _pd.DataFrame(rows, columns=cols)
        except Exception:
            # si hay longitud dispar, rellenamos
            mx = max(len(r) for r in rows) if rows else 0
            rows2 = [list(r)+[None]*(mx-len(r)) for r in rows]
            return _pd.DataFrame(rows2, columns=cols[:mx])

    def _do_preview():
        idx, tab = _cur_tab()
        with out_log: clear_output(wait=True)
        for o in (out_preview, out_aws, out_tele, out_listas):
            o.clear_output(wait=True)
        try:
            name, cols, rows = _read_any(tab)
            _last.update({"name": name, "cols": cols, "rows": rows, "tab": idx})
            with out_log: print(f"Preview de '{name}' · Filas {len(rows)} · Columnas {len(cols)}")
            with out_preview: display(_render_table(cols, rows))
            # guarda df en memoria según pestaña
            df = _to_df(cols, rows)
            if idx == 0: _last["df_in"] = df
            if idx == 2: _last["df_prom"] = df
            if idx == 3: _last["df_pagos"] = df
            right_tabs.selected_index = 0
        except Exception as e:
            with out_log: print("[ERROR]", e); right_tabs.selected_index = 1

    def _do_validate():
        with out_log: clear_output(wait=True)
        if not _last["cols"]:
            with out_log: print("Genere una preview antes de validar."); return
        # heurística de nombre lógico simple:
        def _guess_logico(nm, tab_idx):
            n = (nm or "").upper()
            if tab_idx == 0:
                return "TER_UNISONO" if "TER" in n else "IN_SISTECREDITO"
            if tab_idx == 1: return "ENTRADA_AWS"
            if tab_idx == 2: return "PROMESAS"
            if tab_idx == 3: return "PAGOS"
            return "IN_SISTECREDITO"
        archivo_logico = _guess_logico(_last["name"], _last["tab"])
        expected = get_column_names_from_estructuras(archivo_logico)
        if not expected:
            with out_log: print(f"No hay definición en 'Estructuras' para '{archivo_logico}'."); return
        cols = [str(c) for c in _last["cols"]]
        missing = [c for c in expected if c not in cols]
        extras  = [c for c in cols if c not in expected]
        if not missing and not extras:
            with out_log: print(f"Validación OK: {archivo_logico} coincide ({len(cols)} columnas).")
        else:
            with out_log:
                print(f"Desalineado vs {archivo_logico}. Esperadas={len(expected)} · Leídas={len(cols)}")
                if missing: print("Faltan:", ", ".join(missing))
                if extras:  print("Sobran:", ", ".join(extras))
        right_tabs.selected_index = 1

    def _run_clean_and_build():
        """Construye Entrada_AWS + Teleton + Listas usando árbol + ley2300."""
        with out_log: print("Construyendo salidas…")
        if _last.get("df_in") is None:
            with out_log: print("Falta la tabla de Rutina (IN_SISTECREDITO / TER)."); return
        # lee árbol
        try:
            df_arbol = load_arbol_heuristico(gc=gc, CONFIG_SHEET_ID=CONFIG_SHEET_ID)
        except Exception:
            df_arbol = load_arbol_heuristico(gc=None, CONFIG_SHEET_ID=None)  # Excel local de apoyo
        ley = _ley_read()
        entrada_aws, teleton, listas = build_entrada_aws(_last["df_in"], _last.get("df_prom"), _last.get("df_pagos"), df_arbol, ley)
        _last["entrada_aws"], _last["teleton"], _last["listas"] = entrada_aws, teleton, listas

        # pinta pestañas derechas
        with out_aws:
            out_aws.clear_output(wait=True)
            if not entrada_aws.empty:
                display(HTML("<b>Entrada_AWS (ordenada)</b>"));
                display(_render_table(list(entrada_aws.columns), entrada_aws.head(400).values.tolist()))
            else:
                display(HTML("<i>Vacío.</i>"))
        with out_tele:
            out_tele.clear_output(wait=True)
            if not teleton.empty:
                display(HTML("<b>Teleton</b> (edita Infinivirt en operación)"));
                display(_render_table(list(teleton.columns), teleton.head(400).values.tolist()))
            else:
                display(HTML("<i>Vacío.</i>"))
        with out_listas:
            out_listas.clear_output(wait=True)
            if listas:
                for k, dfk in listas.items():
                    display(HTML(f"<b>Lista · {k}</b> — {len(dfk)} registros"))
                    if not dfk.empty:
                        display(_render_table(list(dfk.columns), dfk.head(300).values.tolist()))
            else:
                display(HTML("<i>Sin listas</i>"))
        right_tabs.selected_index = 2

    def _export_full(df, basename):
        if df is None or df.empty:
            with out_log: print("Nada para exportar."); return
        fmt = fmt_dd.value
        ts  = _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        name_noext = f"{basename}_{ts}"
        try:
            if fmt.startswith("csv"):
                sep = "," if fmt=="csv_comma" else ";"
                data = df.to_csv(index=False, sep=sep).encode("utf-8")
                fid = drive_upload_bytes(SALIDAS_ID, name_noext + ".csv", data, "text/csv")
            else:
                # usa tu helper si existe; si no, pandas to_excel
                try:
                    data = table_to_xlsx_bytes(df)
                    fid = drive_upload_bytes(SALIDAS_ID, name_noext + ".xlsx", data,
                                            "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
                except Exception:
                    buf = io.BytesIO()
                    with _pd.ExcelWriter(buf, engine="xlsxwriter") as writer:
                        df.to_excel(writer, index=False)
                    fid = drive_upload_bytes(SALIDAS_ID, name_noext + ".xlsx", buf.getvalue(),
                                            "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
            with out_log: print(f"Exportado '{name_noext}' → file_id={fid}")
        except Exception as e:
            with out_log: print("[ERROR] exportando:", e)

    # wire-up
    btn_preview.on_click(lambda _ : _do_preview())
    btn_validate.on_click(lambda _ : _do_validate())
    btn_clean.on_click(lambda _ : _run_clean_and_build())
    btn_export.on_click(lambda _ : _export_full(_last.get("entrada_aws"), "ENTRADA_AWS"))
    btn_genesys.on_click(lambda _ : _export_full(_last.get("entrada_aws"), "ENTRADA_AWS_Genesys"))

    # devuelve por si luego quieres acceder a algo desde fuera
    # >>> PATCH: devuelve también los handles que usa el "Modo UI"
    return {
        # salidas mínimas que ya tienes
        "left_tabs": left_tabs,
        "right_tabs": right_tabs,
        "out_preview": out_preview,
        "out_log": out_log,
        "out_clean": out_clean,

        # ----- handles para Modo UI (nuevo) -----
        # el dict de la pestaña Rutina (tiene folder/refresh/files/upload/origen)
        "rut": rut,
        # el ToggleButtons "Detección: Automática / Estructuras"
        "det_cols": det_cols,
        # el selector de formato CSV/XLSX
        "fmt_dd": fmt_dd,
    }

In [258]:
#SinergIA0700
###  ESTE WIRING HAY QUE REVISARLO CON LUPA!!!

# === Parche seguro: Wiring Export + Genesys + Teletón ===
import ipywidgets as W
from IPython.display import HTML, display

# Validar existencia de objetos base antes de usarlos
btn_export = globals().get("btn_export")
btn_genesys = globals().get("btn_genesys")
out_log = globals().get("out_log") or W.Output()
right_tabs = globals().get("right_tabs") or W.Tab()

# 1) Asegurar pestaña Teletón
if "out_teleton" not in globals():
    out_teleton = W.Output()
    try:
        # Agregar como última pestaña visible
        right_tabs.children = tuple(list(right_tabs.children) + [out_teleton])
        right_tabs.set_title(len(right_tabs.children) - 1, "Teletón")
    except Exception:
        pass

def _load_ley_params_safe():
    try:
        return l2300_load_params()
    except Exception as e:
        with out_log:
            print("[WARN] Ley2300: usando defaults:", e)
        return {
            "recontact_days": 4,
            "wk_start":"07:00", "wk_end":"19:00",
            "sat_start":"08:00", "sat_end":"15:00",
            "allow_sun": False, "one_channel_per_week": True
        }

def _do_export_full():
    with out_log:
        out_log.clear_output(wait=True)
        print("Ejecutando exportación completa...")
    ley = _load_ley_params_safe()
    # si no hay función real, solo mostrar mensaje
    if "export_full_dataset" not in globals():
        with out_log:
            print("[INFO] export_full_dataset() aún no definido. Simulando exportación...")
        return
    try:
        res = export_full_dataset(
            cols=_last.get("cols", []),
            rows=_last.get("rows", []),
            fmt_key=fmt_dd.value if "fmt_dd" in globals() else "csv_comma",
            file_stub="Entrada_AWS",
            limpiar=bool(chk_clean_exp.value) if "chk_clean_exp" in globals() else True,
            ley_params=ley
        )
        with out_log:
            print(f"Export OK → {res['name']} · file_id={res['file_id']}")
            print(f"Filas exportadas: {len(res['entrada'])}")
        with out_teleton:
            out_teleton.clear_output(wait=True)
            display(HTML("<b>Teletón — resumen por canal y franja</b>"))
            display(res["teleton"])
        right_tabs.selected_index = len(right_tabs.children) - 1
    except Exception as e:
        with out_log:
            print("[ERROR] Exportando Entrada_AWS:", e)
        right_tabs.selected_index = 1

def _do_export_genesys():
    with out_log:
        print("[INFO] Exportando Genesys...")
    _do_export_full()

# Conectar botones si existen
if btn_export:
    btn_export.on_click(lambda _: _do_export_full())
else:
    with out_log: print("[WARN] btn_export no existe en este contexto.")

if btn_genesys:
    btn_genesys.on_click(lambda _: _do_export_genesys())
else:
    with out_log: print("[WARN] btn_genesys no existe en este contexto.")

display(out_log)

Output()

In [259]:
#SinergIA0720
# CORE v1 · Entrada_AWS + orden + export
import io, csv, math, hashlib, datetime as dt
import pandas as pd

# ---------- utilidades mínimas ----------
def _rows_to_df(cols, rows):
    df = pd.DataFrame(rows, columns=[str(c) for c in cols])
    return df

def _to_csv_bytes(df: pd.DataFrame, sep=","):
    buf = io.StringIO()
    df.to_csv(buf, index=False, sep=sep, lineterminator="\n")
    return buf.getvalue().encode("utf-8")

def _to_xlsx_bytes(df: pd.DataFrame):
    # Sin estilos: liviano y seguro
    import xlsxwriter
    bio = io.BytesIO()
    with pd.ExcelWriter(bio, engine="xlsxwriter") as xw:
        df.to_excel(xw, sheet_name="data", index=False)
    return bio.getvalue()

def _upload_bytes_salida(name: str, data: bytes, mime: str):
    # usa SALIDAS_ID definido en tu config
    return drive_upload_bytes(SALIDAS_ID, name, data, mime)

def _safe_get(r: dict, key: str, default=None):
    return r.get(key, default) if isinstance(r, dict) else default

# ---------- normalización básica ----------
def normalize_phones(df: pd.DataFrame):
    """Normaliza posibles teléfonos en columnas conocidas."""
    phone_cols = [c for c in df.columns if str(c).lower() in
                  {"celular","telefono","teléfono","movil","móvil","phone","mobile","ani"}]
    for c in phone_cols:
        df[c] = df[c].astype(str).str.replace(r"\D+","", regex=True)
        # típico en CO: largos 10; si 57... quedarnos con 10 finales
        df[c] = df[c].str.replace(r"^57", "", regex=True).str[-10:]
    return df, phone_cols

def basic_clean(df: pd.DataFrame):
    df = df.copy()
    # Trimear strings
    for c in df.columns:
        if pd.api.types.is_string_dtype(df[c]):
            df[c] = df[c].astype(str).str.strip()
    df, phone_cols = normalize_phones(df)
    # Tipos numéricos suaves
    for c in df.columns:
        if df[c].dtype == object:
            try:
                df[c] = pd.to_numeric(df[c], errors="ignore")
            except Exception:
                pass
    return df, phone_cols

# ---------- heurística de priorización ----------
def _parse_dt(s):
    for fmt in ("%Y-%m-%d %H:%M:%S", "%Y-%m-%d", "%d/%m/%Y", "%d/%m/%Y %H:%M:%S"):
        try:
            return dt.datetime.strptime(str(s), fmt)
        except Exception:
            pass
    return None

def score_df(df: pd.DataFrame):
    """
    Score simple y defensivo. Usa si existen:
      - propension / prob / score (mayor=mejor)
      - ultima_gestion / last_contact / fecha_ult_gestion (más reciente=menor penalización)
      - saldo / valorcap / deuda (mayor=mejor prioridad)
    """
    s = pd.Series(0.0, index=df.index, dtype="float64")

    # 1) Propensión
    for cand in ("propension","propensión","prob","score_model","final_mean"):
        if cand in df.columns:
            v = pd.to_numeric(df[cand], errors="coerce")
            s = s + v.fillna(0).clip(lower=0)
            break

    # 2) Antigüedad de última gestión (más reciente = +)
    last_cols = [c for c in ("ultima_gestion","última_gestión","last_contact","fecha_ult_gestion") if c in df.columns]
    if last_cols:
        lc = df[last_cols[0]].map(_parse_dt)
        age_h = (dt.datetime.now() - lc).dt.total_seconds()/3600
        s = s + age_h.fillna(age_h.max() if len(age_h.dropna()) else 0).rsub(age_h.max() or 0) * 0.1

    # 3) Importe (mayor saldo = +)
    for cand in ("saldo","valorcap","deuda","montopendiente","valorpago"):
        if cand in df.columns:
            v = pd.to_numeric(df[cand], errors="coerce")
            s = s + (v.fillna(0) / (v.fillna(0).abs().max() or 1)) * 0.5
            break

    # Normalizar 0..1
    if s.max() > s.min():
        s = (s - s.min()) / (s.max() - s.min())
    return s

# ---------- Ley 2300 (si hay señales) ----------
def apply_ley2300(df: pd.DataFrame, ley: dict):
    """
    Filtro 'streaming' suave: solo actúa si el dataset tiene columnas de última gestión y canal.
    - recontact_days
    - horarios L-V y sáb
    - one_channel_per_week (si hay 'canal_ult' y 'fecha_ult_gestion')
    """
    if not ley:
        return df, {"aplicada": False, "omitido": "sin parámetros"}

    # 1) cooldown por días
    last_cols = [c for c in ("ultima_gestion","última_gestión","last_contact","fecha_ult_gestion") if c in df.columns]
    if last_cols:
        cutoff = dt.datetime.now() - dt.timedelta(days=int(ley.get("recontact_days",4)))
        lc = df[last_cols[0]].map(_parse_dt)
        df = df[lc.isna() | (lc < cutoff)]

    # 2) restricción por franja (se evalúa para ahora → generación de listas actuales)
    now = dt.datetime.now()
    wk_start = dt.datetime.strptime(ley.get("wk_start","07:00"), "%H:%M").time()
    wk_end   = dt.datetime.strptime(ley.get("wk_end","19:00"), "%H:%M").time()
    sa_start = dt.datetime.strptime(ley.get("sat_start","08:00"), "%H:%M").time()
    sa_end   = dt.datetime.strptime(ley.get("sat_end","15:00"), "%H:%M").time()
    allow_sun = bool(ley.get("allow_sun", False))
    wd = now.weekday()  # 0=Mon .. 6=Sun
    t  = now.time()

    def ok_time():
        if wd <= 4:   # L–V
            return wk_start <= t <= wk_end
        if wd == 5:   # Sáb
            return sa_start <= t <= sa_end
        return allow_sun

    if not ok_time():
        # Si estamos fuera de franja, no filtramos filas (sería una lista vacía),
        # solo anotamos que la salida está "fuera de franja".
        pass

    # 3) un canal por semana (si hay canal_ult/fecha_ult_gestion)
    if ley.get("one_channel_per_week", True):
        if {"canal_ult","fecha_ult_gestion"}.issubset({c.lower() for c in df.columns}):
            # Ejemplo mínimo: nada que hacer aquí sin mapeo de canales;
            # mantenemos placeholder para futuras iteraciones.
            pass

    return df, {"aplicada": True}

# ---------- construcción de Entrada_AWS y Teletón ----------
GENESYS_COLS = [
    "contact_id","doc_tipo","doc_num","nombre","telefono",
    "canal_sugerido","franja","prioridad","infinivirt",
    "score","saldo","credit_number"
]

def build_entrada_aws(df_in: pd.DataFrame, ley_params: dict):
    df = df_in.copy()

    # Limpieza básica
    df, phone_cols = basic_clean(df)

    # Telefono elegido (móvil preferente si existe)
    tel_col = None
    for pref in ("celular","movil","móvil","mobile","phone","ani","telefono","teléfono"):
        if pref in (c.lower() for c in df.columns):
            tel_col = [c for c in df.columns if c.lower()==pref][0]; break
    if tel_col is None and phone_cols:
        tel_col = phone_cols[0]
    if tel_col is None:
        tel_col = "__no_phone__"
        df[tel_col] = ""

    # Nombre / doc / saldo / crédito
    def first_col(*names):
        for n in names:
            if n in df.columns: return n
        return None

    c_nombre = first_col("Upper","nombre","cliente","name")
    c_doc    = first_col("Identificacion","documento","doc","id","identificación")
    c_saldo  = first_col("ValorCap","saldo","deuda","montopendiente")
    c_credit = first_col("CreditNumber","creditnumber","credito","account")

    # Score
    s = score_df(df)

    # Canal sugerido (placeholder rápido: si hay teléfono móvil válido → 'TEL'; se podrán añadir 'WA/SMS')
    canal = pd.Series("TEL", index=df.index)

    # Franja actual sugerida (según ley; placeholder: 'operativa')
    franja = pd.Series("operativa", index=df.index)

    # Infinivirt (placeholder vacío: se llenará cuando esté operativa la integración)
    infinivirt = pd.Series("", index=df.index)

    # Prioridad = ranking por score
    prio = (s.rank(ascending=False, method="first")).astype(int)

    out = pd.DataFrame({
        "contact_id": [hashlib.md5(f"{_safe_get({'d':df.at[i,c_doc] if c_doc else ''},'d','')}{df.at[i,tel_col]}".encode()).hexdigest()[:12]
                       for i in df.index],
        "doc_tipo": "CC",
        "doc_num": df[c_doc] if c_doc else "",
        "nombre":  df[c_nombre] if c_nombre else "",
        "telefono": df[tel_col],
        "canal_sugerido": canal,
        "franja": franja,
        "prioridad": prio,
        "infinivirt": infinivirt,
        "score": s.round(4),
        "saldo": df[c_saldo] if c_saldo else "",
        "credit_number": df[c_credit] if c_credit else "",
    })

    # Orden final
    out = out.sort_values(["prioridad","score"], ascending=[True, False])

    # Ley 2300 (siempre que haya señales útiles; sino solo marca 'aplicada:True' sin cambios)
    out_ley, ley_info = apply_ley2300(out, ley_params or {})

    # Teletón (reporte simple)
    teleton = (out_ley
               .assign(franja=lambda d: d["franja"].fillna("operativa"),
                       canal_sugerido=lambda d: d["canal_sugerido"].fillna("TEL"))
               .groupby(["canal_sugerido","franja"], dropna=False)
               .agg(registros=("contact_id","count"),
                    saldo_total=("saldo", lambda x: pd.to_numeric(x, errors="coerce").fillna(0).sum()))
               .reset_index()
               .sort_values(["registros"], ascending=False))

    return out_ley.reset_index(drop=True), teleton

def export_full_dataset(
    cols, rows, fmt_key: str, file_stub: str, limpiar: bool = True, ley_params: dict = None
):
    """
    Toma lo que está en _last (preview), lo limpia/ordena como Entrada_AWS y sube a SALIDAS.
    fmt_key: 'csv_comma' | 'csv_semicolon' | 'xlsx'
    """
    df_in = _rows_to_df(cols, rows)
    if limpiar:
        df_in, _ = basic_clean(df_in)

    entrada, teleton = build_entrada_aws(df_in, ley_params or {})

    # Bytes según formato
    ts  = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    sep = "," if fmt_key == "csv_comma" else ";"
    if fmt_key in ("csv_comma","csv_semicolon"):
        data = _to_csv_bytes(entrada, sep=sep)
        name = f"Entrada_AWS_{ts}.csv"
        fid  = _upload_bytes_salida(name, data, "text/csv")
    else:
        data = _to_xlsx_bytes(entrada)
        name = f"Entrada_AWS_{ts}.xlsx"
        fid  = _upload_bytes_salida(name, data, "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")

    return {"name": name, "file_id": fid, "entrada": entrada, "teleton": teleton}

In [260]:
#SinergIA0740
# === Core SinergIA Cobros · Limpieza + Heurística mínima (Entrada_AWS / Teletón / Listas) ===
import pandas as pd, numpy as np
from io import BytesIO

# --- Utilidades de columnas tolerantes a esquema ---
def _pick(df, *alts):
    for a in alts:
        if a in df.columns: return a
        # búsqueda laxa por regex
        hits = [c for c in df.columns if pd.Series([c]).str.fullmatch(a, case=False, regex=True).any()]
        if hits: return hits[0]
    return None

def _norm_cols(df):  # intenta mapear nombres típicos
    mapper = {}
    mapper["id"]      = _pick(df, r"(?i)(id_(cli|cliente)|num_?doc|documento|cedula|ident.*)")
    mapper["fecha"]   = _pick(df, r"(?i)(fecha(_?(ges|gestion|contacto|evento))?|fch_ges)")
    mapper["canal"]   = _pick(df, r"(?i)(canal|canal_?contacto)")
    mapper["codigo"]  = _pick(df, r"(?i)(cod(igo)?(_?(tip|res(ult)?))?|tipif?_?cod)")
    mapper["prio"]    = _pick(df, r"(?i)(prio(ridad)?|prioridad|rank)")
    mapper["saldo"]   = _pick(df, r"(?i)(saldo(_?total)?|monto|valor(_?saldo)?)")
    mapper["dias"]    = _pick(df, r"(?i)(dias?_?mora|max_?dias?_?mora|dias)")
    return mapper

# --- Export util ---
def df_to_bytes(df: pd.DataFrame, fmt: str) -> tuple[str, bytes, str]:
    if fmt == "csv_comma":
        data = df.to_csv(index=False).encode("utf-8"); ext="csv"; mime="text/csv"
    elif fmt == "csv_semicolon":
        data = df.to_csv(index=False, sep=";").encode("utf-8"); ext="csv"; mime="text/csv"
    elif fmt == "xlsx":
        bio = BytesIO(); df.to_excel(bio, index=False); data=bio.getvalue(); ext="xlsx"; mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    else:
        raise ValueError("Formato no soportado")
    return ext, data, mime

# --- Heurística mínima inspirada en los audios/TXT ---
def best_by_priority(df: pd.DataFrame, key_col: str, fecha_col: str, prio_col: str) -> pd.DataFrame:
    # prioridad baja = mejor (1 mejor que 99).  Fechas más recientes predominan.
    _df = df.copy()
    _df[prio_col] = pd.to_numeric(_df[prio_col], errors="coerce").fillna(9999)
    _df[fecha_col] = pd.to_datetime(_df[fecha_col], errors="coerce")
    _df = _df.sort_values([key_col, prio_col, fecha_col], ascending=[True, True, False])
    return _df.drop_duplicates(subset=[key_col], keep="first")

def apply_channel_defaults(df: pd.DataFrame, canal_col: str, prio_col: str) -> pd.DataFrame:
    if canal_col not in df.columns: return df
    _df = df.copy()
    mask_wa  = _df[canal_col].astype(str).str.contains("whats", case=False, na=False)
    mask_sms = _df[canal_col].astype(str).str.contains("sms|texto", case=False, na=False)
    _df.loc[mask_wa & _df[prio_col].isna(), prio_col] = 37   # WhatsApp → 37
    _df.loc[mask_sms & _df[prio_col].isna(), prio_col] = 37  # SMS → 37
    return _df

# --- Construcción Entrada_AWS ---
def build_entrada_aws(df_rutina: pd.DataFrame, df_log: pd.DataFrame|None, df_prom: pd.DataFrame|None, df_pagos: pd.DataFrame|None) -> pd.DataFrame:
    if df_rutina is None or df_rutina.empty:
        raise ValueError("No hay base de Rutina (IN_SISTECREDITO)")

    # Normaliza
    m_r = _norm_cols(df_rutina);  df_r = df_rutina.copy()
    id_r = m_r["id"] or df_r.columns[0]

    # Log / gestiones (opcional)
    best_log = None
    if df_log is not None and not df_log.empty:
        m_l = _norm_cols(df_log); df_l = df_log.rename(columns={m_l["id"]: "id_cli", m_l["fecha"]: "fecha", m_l["canal"]: "canal",
                                                                m_l["codigo"]: "cod", m_l["prio"]: "prio"}, errors="ignore").copy()
        for req in ["id_cli","fecha","prio"]:
            if req not in df_l.columns: df_l[req] = np.nan
        df_l = apply_channel_defaults(df_l, "canal", "prio")
        best_log = best_by_priority(df_l, "id_cli", "fecha", "prio")[["id_cli","fecha","canal","cod","prio"]]

    # Join “mejor gestión” (si existe)
    df = df_r.copy()
    if best_log is not None:
        df = df.merge(best_log, how="left", left_on=id_r, right_on="id_cli").drop(columns=["id_cli"])

    # “Orden” para listas de trabajo: prioridad asc, luego fecha reciente
    if "prio" in df.columns:
        df["orden"] = pd.to_numeric(df["prio"], errors="coerce").fillna(9999)
    else:
        df["orden"] = 9999
    if "fecha" in df.columns:
        df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

    # TODO: incorporar promesas y pagos como señales de orden (si se aportan)
    # (ejemplo: bajar prioridad si tiene promesa vigente futura; subir si promesa rota reciente; sumar pagos por día)
    # Referencia pagos: “sumar pagos por día si hay varios en un mismo crédito”.  (TXT)

    return df

# --- Construcción Teletón (resumen por cliente) ---
def build_teleton(df_aws: pd.DataFrame) -> pd.DataFrame:
    if df_aws is None or df_aws.empty:
        return pd.DataFrame()

    m = _norm_cols(df_aws)
    idc = m["id"]   or _pick(df_aws, r"(?i)(id_cli.*|documento|cedula)")
    val = m["saldo"] or _pick(df_aws, r"(?i)(valor|monto|capital)")
    dm  = m["dias"]  or _pick(df_aws, r"(?i)(dias.*mora|mora_dias)")
    fecha = _pick(df_aws, r"(?i)fecha")

    g = df_aws.copy()
    if fecha: g[fecha] = pd.to_datetime(g[fecha], errors="coerce")
    agg = { }
    if val:  agg[val]  = "sum"
    if dm:   agg[dm]   = "max"
    if fecha: agg[fecha] = "max"

    tel = g.groupby(idc, as_index=False).agg(agg)
    tel.rename(columns={val: "saldo_total", dm: "max_dias_mora", fecha:"ult_gestion"}, inplace=True)
    return tel

# --- Derivación de listas por canal (para Genesys) ---
def build_listas_por_canal(df_aws: pd.DataFrame) -> dict[str, pd.DataFrame]:
    if df_aws is None or df_aws.empty:
        return {"Voz": pd.DataFrame(), "WhatsApp": pd.DataFrame(), "SMS": pd.DataFrame()}
    canal = _norm_cols(df_aws)["canal"] or "canal"
    if canal not in df_aws.columns:
        df_aws = df_aws.copy(); df_aws[canal] = np.nan

    voz = df_aws[df_aws[canal].str.contains("voz|call|phone|llama", case=False, na=False)]
    wpp = df_aws[df_aws[canal].str.contains("whats", case=False, na=False)]
    sms = df_aws[df_aws[canal].str.contains("sms|texto", case=False, na=False)]
    # Si no hay canal marcado, envíalos a Voz por defecto
    resto = df_aws[~df_aws.index.isin(pd.concat([voz,wpp,sms]).index)]
    voz = pd.concat([voz, resto], ignore_index=True)

    return {"Voz": voz, "WhatsApp": wpp, "SMS": sms}

In [261]:
#SinergIA0760
# ==============================================
# G2 — GUI v9 (reemplazo completo)
# Incluye: Ley2300 embebido + pestañas Entrada_AWS / Teletón / Listas / Optimización
# Requiere helpers: drive_list_recent, drive_download_file, drive_upload_bytes, read_preview_any, df_to_bytes
# ==============================================
import ipywidgets as W
from IPython.display import display, HTML, clear_output
import pandas as pd
import csv, io, datetime as _dt

def render_sinergia_gui_like_gradio():
    # Estilos
    display(HTML("""
    <style>
      .sg-card { border:1px solid #3d3f44; border-radius:18px; padding:14px; box-shadow:0 3px 12px rgba(0,0,0,.18); }
      .sg-title { font-weight:700; font-size:22px; margin:2px 0 8px; }
      .sg-sub   { color:#a6adb4; font-size:12px; margin:0 0 8px; }
      .sg-btn  button { border-radius:999px !important; }
    </style>
    """))

    def _render_table(cols, rows, max_rows=300):
        head = "".join(f"<th style='padding:6px 8px;border-bottom:1px solid #ddd'>{c}</th>" for c in cols)
        body = []
        for r in rows[:max_rows]:
            tds = "".join(f"<td style='padding:4px 8px;border-bottom:1px solid #eee'>{'' if v is None else v}</td>" for v in r)
            body.append(f"<tr>{tds}</tr>")
        return HTML(f"""
          <div style='max-height:480px;overflow:auto;border:1px solid #aaa;border-radius:10px'>
            <table style='border-collapse:collapse;width:100%'>
              <thead style='position:sticky;top:0;background:#f5f5f5'><tr>{head}</tr></thead>
              <tbody>{''.join(body)}</tbody>
            </table>
          </div>""")

    def _make_source_box(default_folder_id: str, subtitulo: str):
        origen = W.ToggleButtons(options=["Drive","Local"], value="Drive", description="Origen:")
        folder = W.Text(value=default_folder_id, description="Folder ID:", layout=W.Layout(flex="1 1 auto", width="100%"))
        refresh = W.Button(description="Actualizar", layout=W.Layout(width="130px"))
        files = W.Dropdown(options=[], description="Archivo:", layout=W.Layout(width="100%"))
        upload = W.FileUpload(accept=".csv,.xlsx,.xls", multiple=False, description="Subir local")
        def _toggle(change):
            drive = (change["new"] == "Drive")
            folder.layout.display  = "" if drive else "none"
            refresh.layout.display = "" if drive else "none"
            files.layout.display   = "" if drive else "none"
            upload.layout.display  = "none" if drive else ""
        origen.observe(_toggle, names="value"); _toggle({"new":"Drive"})
        box = W.VBox([
            W.HTML(f"<div class='sg-sub'>{subtitulo}</div>"),
            origen, W.HBox([folder, refresh]), files, upload
        ])
        return {"origen": origen, "folder": folder, "refresh": refresh, "files": files, "upload": upload, "box": box}

    # pestañas izquierda (IDs desde tu MapaID)
    rut = _make_source_box(RUTINAS_ID, "Rutina · IN_SISTECREDITO")
    aws = _make_source_box(AWS_LOG_ID, "AWS · Logs/Base")
    pro = _make_source_box(PROMESAS_ID, "Promesas")
    pag = _make_source_box(PAGOS_ID, "Pagos")
    left_tabs = W.Tab(children=[rut["box"], aws["box"], pro["box"], pag["box"]])
    for i, t in enumerate(["Rutina","AWS","Promesas","Pagos"]): left_tabs.set_title(i, t)

    # salidas (derecha)
    out_preview, out_log, out_clean = W.Output(), W.Output(), W.Output()
    out_aws, out_tel, out_lists, out_opt = W.Output(), W.Output(), W.Output(), W.Output()
    right_tabs = W.Tab(children=[out_preview, out_log, out_clean, out_aws, out_tel, out_lists, out_opt])
    for i, t in enumerate(["Preview / IN","Log","Datos limpios","Entrada_AWS","Teletón","Listas","Optimización"]):
        right_tabs.set_title(i, t)

    # acciones
    btn_preview   = W.Button(description="Previsualizar");             btn_preview.add_class("sg-btn")
    btn_validate  = W.Button(description="Validar estructura");        btn_validate.add_class("sg-btn")
    btn_clean     = W.Button(description="Aplicar limpieza");          btn_clean.add_class("sg-btn")
    btn_savecsv   = W.Button(description="Guardar CSV (muestra)");     btn_savecsv.add_class("sg-btn")
    chk_clean_exp = W.Checkbox(value=True, description="Limpiar al exportar completo")
    det_cols      = W.ToggleButtons(options=["Automática","Estructuras"], value="Automática", description="Detección:")
    btn_genesys   = W.Button(description="Exportar Genesys CSV");      btn_genesys.add_class("sg-btn")
    btn_dwl       = W.Button(description="Descargar local (muestra)"); btn_dwl.add_class("sg-btn")
    fmt_dd        = W.Dropdown(
        options=[("CSV (coma)", "csv_comma"), ("CSV (punto y coma)", "csv_semicolon"), ("XLSX", "xlsx")],
        value="csv_comma", description="Formato:"
    )
    # Pipeline
    btn_make_aws  = W.Button(description="Construir Entrada_AWS");  btn_make_aws.add_class("sg-btn")
    btn_exp_aws   = W.Button(description="Exportar Entrada_AWS");   btn_exp_aws.add_class("sg-btn")
    btn_make_tel  = W.Button(description="Generar Teletón");        btn_make_tel.add_class("sg-btn")
    btn_make_lst  = W.Button(description="Derivar Listas");         btn_make_lst.add_class("sg-btn")

    actions = W.VBox([
        W.HBox([btn_preview, btn_validate, btn_clean]),
        W.HBox([btn_savecsv, btn_genesys, chk_clean_exp]),
        W.HBox([det_cols]),
        W.HBox([fmt_dd, btn_dwl]),
        W.HTML("<hr>"),
        W.HTML("<b>Pipeline</b>"),
        W.HBox([btn_make_aws, btn_exp_aws]),
        W.HBox([btn_make_tel, btn_make_lst]),
    ])

    # Panel Ley2300 embebido
    try:
        panel_l2300 = l2300_make_panel()
    except Exception:
        # fallback mínimo si el panel no existe en este notebook
        panel_l2300 = W.Accordion([W.VBox([W.HTML("<i>Panel Ley2300 no disponible en este notebook.</i>")])])
        panel_l2300.set_title(0, "Ley2300")
    col_left  = W.VBox([W.HTML("<div class='sg-title'>Cargar archivos primarios</div>"), left_tabs, actions, panel_l2300],
                       layout=W.Layout(width="520px"))
    col_right = W.VBox([right_tabs], layout=W.Layout(flex="1 1 auto"))

    display(HTML("<div class='sg-card'>"))
    display(W.VBox([W.HTML("<div class='sg-title'>SinergIA Cobros</div>"),
                    W.HBox([col_left, W.HTML("<div style='width:16px'></div>"), col_right])]))
    display(HTML("</div>"))

    # estado
    STATE = {"Rutina": None, "AWS": None, "Promesas": None, "Pagos": None,
             "Entrada_AWS": None, "Teleton": None, "Listas": {}}

    # helpers locales
    def _refresh(tab):
        with out_log: clear_output(wait=True)
        try:
            items = drive_list_recent(tab["folder"].value, top=12)
            tab["files"].options = [(f'{it["name"]}  —  {it.get("modifiedTime","")}', it["id"]) for it in items] or []
            with out_log: print(f"Listados {len(tab['files'].options)} archivos.")
        except Exception as e:
            with out_log: print("[ERROR]", e)

    for box in (rut, aws, pro, pag):
        box["refresh"].on_click(lambda _b, box=box: _refresh(box))

    def _tab_name(idx: int) -> str:
        return ["Rutina","AWS","Promesas","Pagos"][idx]

    # --- Normalización de direcciones (Rutina: IN_SISTECREDITO / TER_UNISONO)
    def _normalize_address_value(x):
        """
        Reglas de normalización a nivel de celda:

        1) Vacío / None -> "SIN DIRECCION"
        2) Solo números (ignorando espacios) -> "SIN DIRECCION"
        3) Números al inicio antes de la primera palabra (>=2 letras) ->
           "CERCA A " + resto de la dirección
        """
        # vacío → SIN DIRECCION (usa helper global si existe)
        if "_is_empty" in globals():
            try:
                if _is_empty(x):
                    return "SIN DIRECCION"
            except Exception:
                pass

        # normalizar espacios (usa _norm_spaces si existe)
        if "_norm_spaces" in globals():
            try:
                txt = _norm_spaces(x)
            except Exception:
                txt = "" if x is None else str(x).strip()
        else:
            txt = "" if x is None else str(x).strip()

        if txt == "":
            return "SIN DIRECCION"

        # solo dígitos/espacios → SIN DIRECCION
        if all(ch.isdigit() or ch.isspace() for ch in txt):
            return "SIN DIRECCION"

        tokens = txt.split()
        if not tokens:
            return "SIN DIRECCION"

        def _letters(t: str) -> int:
            return sum(1 for c in t if c.isalpha())

        # primera palabra "real" (≥ 2 letras)
        idx = None
        for i, tok in enumerate(tokens):
            if _letters(tok) >= 2:
                idx = i
                break

        if idx is None:
            return "SIN DIRECCION"

        # si antes de esa palabra solo hay números → CERCA A + resto
        if idx > 0 and all(tok.isdigit() for tok in tokens[:idx]):
            resto = " ".join(tokens[idx:])
            return "CERCA A " + resto

        return txt

    def _normalize_addresses_df(df: pd.DataFrame) -> pd.DataFrame:
        """
        Aplica _normalize_address_value sobre columnas de dirección.
        No toca encabezados, solo valores.
        """
        if df is None or df.empty:
            return df
        df = df.copy()
        # criterio simple: columnas que contengan "direcci"
        cols = [c for c in df.columns if "direcci" in str(c).lower()]
        if not cols:
            return df
        for c in cols:
            df[c] = df[c].apply(_normalize_address_value)
        return df


    """
    def _normalize_address_value(x):
        # vacío o None --> SIN DIRECCION
        if "_is_empty" in globals():
            try:
                if _is_empty(x):
                    return "SIN DIRECCION"
            except:
                pass

        # normalizar espacios
        if "_norm_spaces" in globals():
            try:
                txt = _norm_spaces(x)
            except:
                txt = "" if x is None else str(x).strip()
        else:
            txt = "" if x is None else str(x).strip()

        if txt == "":
            return "SIN DIRECCION"

        # solo números --> SIN DIRECCION
        if all(ch.isdigit() or ch.isspace() for ch in txt):
            return "SIN DIRECCION"

        tokens = txt.split()
        if not tokens:
            return "SIN DIRECCION"

        def _letters(t):
            return sum(1 for c in t if c.isalpha())

        # encontrar primera palabra válida
        idx = None
        for i, tok in enumerate(tokens):
            if _letters(tok) >= 2:
                idx = i
                break

        if idx is None:
            return "SIN DIRECCION"

        # si antes hay solo números --> CERCA A
        if idx > 0 and all(tok.isdigit() for tok in tokens[:idx]):
            resto = " ".join(tokens[idx:])
            return "CERCA A " + resto

        return txt



    def _normalize_addresses_df(df):
        df = df.copy()
        cols = [c for c in df.columns if "direcci" in c.lower()]
        for c in cols:
            df[c] = df[c].apply(_normalize_address_value)
        return df
    """

    def _do_preview():
        idx = left_tabs.selected_index or 0
        box = [rut, aws, pro, pag][idx]
        with out_log: clear_output(wait=True)
        with out_preview: clear_output(wait=True)
        with out_clean: clear_output(wait=True)
        try:
            if box["origen"].value == "Drive":
                if not box["files"].value:
                    with out_log: print("Seleccione un archivo del listado."); return
                name, data = drive_download_file(box["files"].value)
                cols, rows = read_preview_any((name, data), nrows=300)
            else:
                if not box["upload"].value:
                    with out_log: print("Suba un archivo local .csv/.xlsx."); return
                up = list(box["upload"].value.values())[0]
                name = up["metadata"]["name"]; data = up["content"]
                cols, rows = read_preview_any((name, data), nrows=300)

            df = pd.DataFrame(rows, columns=cols)
            STATE[_tab_name(idx)] = df

            with out_log: print(f"Preview de '{name}' · Filas {len(rows)} · Columnas {len(cols)}")
            with out_preview: display(_render_table(cols, rows))
            with out_clean:   display(_render_table(cols, rows))
            right_tabs.selected_index = 0
        except Exception as e:
            with out_log: print("[ERROR]", e); right_tabs.selected_index = 1

    def _do_validate():
        with out_log: clear_output(wait=True)
        try:
            idx = left_tabs.selected_index or 0
            nombre = _tab_name(idx)
            df = STATE[nombre]
            if df is None:
                with out_log: print("Genere una preview antes de validar."); return
            archivo_logico = {"Rutina":"IN_SISTECREDITO","AWS":"ENTRADA_AWS","Promesas":"PROMESAS","Pagos":"PAGOS"}.get(nombre,"IN_SISTECREDITO")
            expected = get_column_names_from_estructuras(archivo_logico)
            cols = list(map(str, df.columns))
            missing = [c for c in expected if c not in cols]
            extras  = [c for c in cols if c not in expected]
            if not missing and not extras:
                with out_log: print(f"Validación OK: {archivo_logico} coincide ({len(cols)} columnas).")
            else:
                with out_log:
                    print(f"Desalineado vs {archivo_logico}.  Esperadas={len(expected)} · Leídas={len(cols)}")
                    if missing: print("Faltan:", ", ".join(missing))
                    if extras:  print("Sobran:", ", ".join(extras))
            right_tabs.selected_index = 1
        except Exception as e:
            with out_log: print("[ERROR]", e); right_tabs.selected_index = 1

"""
    def _do_clean():
        with out_log: clear_output(wait=True)
        with out_clean: clear_output(wait=True)
        try:
            df_in = STATE["Rutina"]
            if df_in is None or df_in.empty:
                with out_log:
                    print("Primero previsualiza la Rutina/IN.")
                return

            # copia original para poder comparar
            df_orig = df_in.copy()

            # 1) Normalización de direcciones
            df_norm = _normalize_addresses_df(df_orig)

            # 2) Trim ligero sobre todas las columnas object
            for c in df_norm.columns:
                if df_norm[c].dtype == "object":
                    df_norm[c] = df_norm[c].astype(str).str.strip()

            # 3) Construir tabla de cambios en direcciones (para probar que sí se normaliza)
            cols_dir = [c for c in df_norm.columns if "direcci" in str(c).lower()]
            df_diff = None
            if cols_dir:
                # comparamos solo columnas de dirección
                orig_dir = df_orig[cols_dir].astype(str)
                norm_dir = df_norm[cols_dir].astype(str)
                mask_changed = (orig_dir != norm_dir).any(axis=1)
                if mask_changed.any():
                    df_diff = pd.concat(
                        [
                            orig_dir[mask_changed].add_suffix("_ORIG"),
                            norm_dir[mask_changed].add_suffix("_NORM"),
                        ],
                        axis=1,
                    ).head(100)

            # 4) Mostrar resultados en la pestaña "Datos limpios"
            with out_clean:
                display(HTML("<b>Datos limpios (muestra)</b>"))
                display(df_norm.head(300))

                if df_diff is not None and not df_diff.empty:
                    display(HTML("<hr><b>Cambios detectados en direcciones (ORIG vs NORM)</b>"))
                    display(df_diff)

            right_tabs.selected_index = 2

        except Exception as e:
            with out_log:
                print("[ERROR] Limpiar:", e)
            right_tabs.selected_index = 1
"""

#SinergIA033 · Reemplazo completo de _do_clean

def _do_clean():
    """
    Botón: 'Aplicar limpieza (solo Rutina)'.

    Toma la muestra TER ya cargada en STATE["Rutina"], construye un
    dataframe IN_SISTECREDITO aplicando reglas de normalización básicas
    y lo muestra en el panel 'IN_SISTECREDITO (limpia)'.

    NOTA: aquí trabajamos con la muestra (300 filas).  Una vez validada
    la lógica, puedes extenderlo a leer el archivo completo.
    """
    import re
    import numpy as np
    import pandas as pd
    from IPython.display import HTML, display
    from IPython.display import clear_output

    with out_log:
        clear_output(wait=True)
        print("[LIMPIEZA] Inicio limpieza TER → IN_SISTECREDITO (muestra).")

    with out_clean:
        clear_output(wait=True)

    # 1) Obtener la muestra de Rutina ya previsualizada
    df_in = STATE.get("Rutina")

    if df_in is None or df_in.empty:
        with out_log:
            print("[LIMPIEZA][ERROR] No hay datos en STATE['Rutina'].")
            print("Ejecuta primero 'Previsualizar' para cargar la TER.")
        return

    # Trabajamos sobre una copia
    df = df_in.copy()

    # 2) Normalización ligera de textos
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = (
                df[col]
                .astype(str)
                .str.strip()
            )

    # 3) Resolver columna de nombre del cliente
    #    Preferencia: 'Upper'  →  si no existe, usar 'NombreCompleto'
    if "Upper" in df.columns:
        serie_nombre = df["Upper"]
    elif "NombreCompleto" in df.columns:
        serie_nombre = df["NombreCompleto"]
    else:
        # si no existe ninguna, creamos una vacía
        serie_nombre = pd.Series([""] * len(df), index=df.index)

    # 4) Helper para teléfonos móviles (solo dígitos, sin fijos)
    def _solo_celulares_validos(val):
        """Devuelve una lista de celulares limpios (solo dígitos, len==10, inicia en 3)."""
        if isinstance(val, (list, tuple, np.ndarray)):
            candidatos = val
        else:
            candidatos = [val]

        celulares = []
        for v in candidatos:
            if v is None or (isinstance(v, float) and np.isnan(v)):
                continue
            s = str(v)
            # quitar todo lo que no sea número
            s_digits = re.sub(r"[^0-9]", "", s)
            # descartar blancos o cortos
            if len(s_digits) != 10:
                continue
            # en Colombia, móviles típicamente empiezan en 3
            if not s_digits.startswith("3"):
                continue
            if s_digits not in celulares:
                celulares.append(s_digits)
        return celulares

    def _telefonos_row(row):
        """A partir de las columnas Celular/Fijo/... genera hasta 4 móviles."""
        candidatos = []
        for c in ["Celular", "Celular2", "Fijo", "Fijo2", "PrimerTelefono"]:
            if c in row.index:
                candidatos.append(row[c])
        celulares = _solo_celulares_validos(candidatos)
        # devolver siempre lista de 4 posiciones (relleno con None)
        celulares = (celulares + [None, None, None, None])[:4]
        return celulares

    # 5) Construir dataframe de salida IN_SISTECREDITO según mapeo que definiste
    out = pd.DataFrame(index=df.index)

    # Mapeo directo de columnas existentes
    def _col_or_empty(col_name):
        return df[col_name] if col_name in df.columns else pd.Series([None] * len(df), index=df.index)

    out["Documento de identidad"]   = _col_or_empty("Identificacion")
    out["Nombre Completo"]          = serie_nombre
    out["Dirección Del Cliente"]    = _col_or_empty("DireccionResidencia")
    out["Fecha Nacimiento"]         = _col_or_empty("FechaNacimiento")
    out["Fecha Expedición"]         = _col_or_empty("FechaExpedicion")
    out["Reportado a Centrales"]    = _col_or_empty("EstadoCentrales")
    out["Crédito"]                  = _col_or_empty("Codigo")          # según tu nota
    out["Fecha Creación del crédito"] = _col_or_empty("FechaCreacion")
    out["Valor"]                    = _col_or_empty("ValorCapital")
    out["Número de cuotas vencidas"] = _col_or_empty("CuotasVencidas")
    out["Fecha de vencimiento"]     = _col_or_empty("FechaMasVencida")
    out["Días en Mora"]             = _col_or_empty("DiasMora")
    out["Valor Mora"]               = _col_or_empty("SaldoMora")
    out["Cargos Jurídico"]          = _col_or_empty("CargosJuridico")
    out["Saldo Total"]              = _col_or_empty("SaldoTotal")

    # 6) Teléfonos 1–4 a partir de Celular/Fijo/Fijo2/Celular2
    telefonos = df.apply(_telefonos_row, axis=1, result_type="expand")
    telefonos.columns = ["Telefono 1", "Telefono 2", "Telefono 3", "Telefono 4"]
    for c in telefonos.columns:
        out[c] = telefonos[c]

    # 7) Datos del almacén
    #    Buscamos columnas tipo 'Nombre', 'Nombre.1', 'Direccion', etc.
    nombre_cols = [c for c in df.columns if c.lower().startswith("nombre")]
    if len(nombre_cols) >= 1:
        col_almacen = nombre_cols[0]
    else:
        col_almacen = None

    if len(nombre_cols) >= 2:
        col_municipio = nombre_cols[1]
    else:
        col_municipio = None

    out["Almacen"]           = _col_or_empty(col_almacen)   if col_almacen   else pd.Series([None] * len(df), index=df.index)
    out["Municipio Almacen"] = _col_or_empty(col_municipio) if col_municipio else pd.Series([None] * len(df), index=df.index)

    # Dirección almacén
    # (ajusta si el nombre exacto de columna es otro)
    if "DireccionAlmacen" in df.columns:
        out["Direccion Almacen"] = df["DireccionAlmacen"]
    elif "Direccion" in df.columns:
        out["Direccion Almacen"] = df["Direccion"]
    else:
        out["Direccion Almacen"] = pd.Series([None] * len(df), index=df.index)

    # Teléfono almacén
    out["Teléfono Almacén"] = _col_or_empty("PrimerTelefono")

    # Status ← CreditNumber (según indicaste)
    out["Status"] = _col_or_empty("CreditNumber")

    # Servicio fijo 'General'
    out["Servicio"] = "General"

    # 8) Franja desde DíasMora (rango duro, usando hoja Parameters cuando exista)
    try:
        # Si la columna no existe, devolvemos franjas vacías pero SIN romper el DataFrame.
        dias_mora = df_clean.get("DiasMora", None)

        franjas = _compute_franja(dias_mora)

        if franjas is None or getattr(franjas, "empty", False):
            # No hay información de días en mora → creamos una columna rellena con vacío.
            # Pandas permite asignar un escalar y lo replica a todas las filas.
            out["Franja"] = ""
            log("[LIMPIEZA] Franja no calculada (no hay DiasMora). Columna creada vacía.", level="warn")
        else:
            # Serie con el mismo índice que df_clean → asignación segura.
            out["Franja"] = franjas
            log("[LIMPIEZA] Franja calculada correctamente.", level="info")

    except Exception as e:
        log(f"[LIMPIEZA] Error calculando Franja: {e}", level="error")


    out["Franja"] = out["Días en Mora"].apply(_franja)

    # IN_PRIORIDAD vacío por ahora
    out["IN_PRIORIDAD"] = None

    # Tipo de Documento
    out["Tipo de Documento"] = _col_or_empty("Tipo Documento")

    # In_Serviceid constante en 1
    out["In_Serviceid"] = 1

    # 9) Reset de índice para evitar errores de longitud
    out = out.reset_index(drop=True)

    # 10) Mostrar resultado en el panel 'IN_SISTECREDITO (limpia)'
    with out_clean:
        clear_output(wait=True)
        display(HTML("<b>IN_SISTECREDITO (limpia) — muestra normalizada</b>"))
        display(out.head(500))

    # Guardar en STATE si quieres reutilizarlo luego
    STATE["IN_SISTECREDITO"] = out

    with out_log:
        print(f"[LIMPIEZA] Limpieza completada. Filas={len(out)} · Columnas={len(out.columns)}")
        print("[LIMPIEZA] Vista previa (500 filas) mostrada en 'IN_SISTECREDITO (limpia)'.")


def _save_csv_muestra():
    with out_log: clear_output(wait=True)
    try:
        idx = left_tabs.selected_index or 0
        df = STATE[_tab_name(idx)]
        if df is None or df.empty:
            with out_log: print("Genere una preview antes de guardar."); return
        ext, data, mime = df_to_bytes(df, "csv_comma")
        name = f"preview_{_tab_name(idx).lower()}_{_dt.datetime.now().strftime('%Y%m%d_%H%M%S')}.{ext}"
        fid = drive_upload_bytes(SALIDAS_ID, name, data, mime)
        with out_log: print(f"CSV guardado en Salidas · file_id={fid}")
        right_tabs.selected_index = 1
    except Exception as e:
        with out_log: print("[ERROR] al subir CSV a Salidas:", e)


    def _make_aws():
        with out_log: clear_output(wait=True)
        with out_aws: clear_output(wait=True)

        try:
            df_r = STATE["Rutina"]
            if df_r is None or df_r.empty:
                with out_log: print("Primero previsualiza la Rutina/IN.")
                return

            # **NORMALIZACIÓN ANTES DEL PIPELINE**
            df_r = _normalize_addresses_df(df_r)

            out = build_entrada_aws(df_r, STATE["AWS"], STATE["Promesas"], STATE["Pagos"])
            STATE["Entrada_AWS"] = out

            with out_log: print(f"Entrada_AWS construida · filas={len(out)}")
            with out_aws:
                display(HTML("<b>Entrada_AWS (ordenada)</b>"))
                display(out.head(300))
            right_tabs.selected_index = 3

        except Exception as e:
            with out_log:
                print("[ERROR] Entrada_AWS:", e)
            right_tabs.selected_index = 1


"""
    def _make_aws():
        with out_log: clear_output(wait=True)
        with out_aws: clear_output(wait=True)
        try:
            df_r, df_l, df_pr, df_pg = STATE["Rutina"], STATE["AWS"], STATE["Promesas"], STATE["Pagos"]
            out = build_entrada_aws(df_r, df_l, df_pr, df_pg)
            STATE["Entrada_AWS"] = out
            with out_log: print(f"Entrada_AWS construida · filas={len(out)}")
            with out_aws:
                display(HTML("<b>Entrada_AWS (ordenada)</b>"))
                display(out.head(300))
            right_tabs.selected_index = 3
        except Exception as e:
            with out_log: print("[ERROR] Entrada_AWS:", e); right_tabs.selected_index = 1
"""

def _exp_aws():
    with out_log: clear_output(wait=True)
    try:
        df = STATE["Entrada_AWS"]
        if df is None or df.empty:
            with out_log: print("Genere primero Entrada_AWS."); return
        ext, data, mime = df_to_bytes(df, fmt_dd.value)
        name = f"Entrada_AWS_{_dt.datetime.now().strftime('%Y%m%d_%H%M%S')}.{ext}"
        fid = drive_upload_bytes(SALIDAS_ID, name, data, mime)
        with out_log: print(f"Entrada_AWS exportada ({fmt_dd.value}) · file_id={fid}")
        right_tabs.selected_index = 1
    except Exception as e:
        with out_log: print("[ERROR] Exportando Entrada_AWS:", e)

    def _make_tel():
        with out_log: clear_output(wait=True)
        with out_tel: clear_output(wait=True)
        try:
            df = STATE["Entrada_AWS"]
            if df is None or df.empty:
                with out_log: print("Construya primero Entrada_AWS."); return
            tel = build_teleton(df)
            STATE["Teleton"] = tel
            with out_log: print(f"Teletón generada · filas={len(tel)}")
            with out_tel:
                display(HTML("<b>Teletón (diligenciable)</b>"))
                display(tel.head(300))
            right_tabs.selected_index = 4
        except Exception as e:
            with out_log: print("[ERROR] Teletón:", e); right_tabs.selected_index = 1

    def _make_lists():
        with out_log: clear_output(wait=True)
        with out_lists: clear_output(wait=True)
        try:
            df = STATE["Entrada_AWS"]
            if df is None or df.empty:
                with out_log: print("Construya primero Entrada_AWS."); return
            d = build_listas_por_canal(df)
            STATE["Listas"] = d
            tabs = W.Tab(children=[W.Output(), W.Output(), W.Output()])
            for i,t in enumerate(["Voz","WhatsApp","SMS"]): tabs.set_title(i,t)
            for i,t in enumerate(["Voz","WhatsApp","SMS"]):
                with tabs.children[i]:
                    _df = d[t]
                    display(W.HTML(f"<b>{t}</b> · filas={len(_df)}"))
                    if not _df.empty:
                        display(_render_table(list(_df.columns), _df.head(300).values.tolist()))
            with out_lists: display(tabs)
            right_tabs.selected_index = 5
        except Exception as e:
            with out_log: print("[ERROR] Listas:", e); right_tabs.selected_index = 1

    def _do_genesys():
        with out_log: clear_output(wait=True)
        try:
            df = STATE["Entrada_AWS"]
            if df is None or df.empty:
                with out_log: print("No hay 'Entrada_AWS' en memoria."); return
            cand_cols = [c for c in ["ani","identificacion","upper"] if c in df.columns]
            g = df[cand_cols].copy()
            if "ani" in g.columns:
                g = g.rename(columns={"ani":"ANI"})
            g["Campaign"] = "SinergIA"
            ext, data, mime = df_to_bytes(g, "csv_comma")
            name = f"Genesys_{_dt.datetime.now().strftime('%Y%m%d_%H%M%S')}.{ext}"
            fid = drive_upload_bytes(SALIDAS_ID, name, data, mime)
            with out_log: print(f"Genesys CSV exportado: {name} · file_id={fid}")
        except Exception as e:
            with out_log: print("[ERROR] Exportar Genesys:", e)

    # wiring
    btn_preview.on_click(lambda _:_do_preview())
    btn_validate.on_click(lambda _:_do_validate())
    btn_clean.on_click(lambda _:_do_clean())
    btn_savecsv.on_click(lambda _:_save_csv_muestra())
    btn_make_aws.on_click(lambda _:_make_aws())
    btn_exp_aws.on_click(lambda _:_exp_aws())
    btn_make_tel.on_click(lambda _:_make_tel())
    btn_make_lst.on_click(lambda _:_make_lists())
    btn_genesys.on_click(lambda _:_do_genesys())

    # ==========================
    # Contexto que devuelve la GUI
    # ==========================
    return {
        # contenedores principales
        "left_tabs": left_tabs,
        "right_tabs": right_tabs,
        "STATE": STATE,

        # cajas de origen de archivos (para el launcher / modo UI)
        "rut": rut,
        "aws": aws,
        "pro": pro,
        "pag": pag,

        # controles técnicos que el launcher quiere ocultar/mostrar
        "det_cols": det_cols,
        "fmt_dd": fmt_dd,

        # accesos rápidos usados por el launcher
        "folder_rut":  rut["folder"],
        "refresh_rut": rut["refresh"],
    }

# OJO: no la llames aquí; el launcher (SinergIA048) es quien debe invocarla
# _ = render_sinergia_gui_like_gradio()


##Modelos de análisis (0800-1110)

In [262]:
#SinergIA0800
# === BLOQUE 1 · Integración de fuentes de análisis (Pagos, Promesas, Log AWS, TER, Teleton) ===
import pandas as pd
import numpy as np
from datetime import datetime as _dt

def integrar_fuentes(df_ter, df_pag, df_pro, df_aws, df_tel):
    """
    Une las diferentes fuentes para generar una base maestra de análisis.
    - df_ter: TER_UNISONO o IN_SISTECREDITO
    - df_pag: pagos históricos
    - df_pro: promesas
    - df_aws: log de gestiones / resultados Genesys
    - df_tel: teletón diario
    Devuelve un DataFrame consolidado listo para aplicar modelos.
    """
    print("Integrando fuentes...")

    # --- Normalización mínima ---
    def _norm(df, key):
        if df is None: return pd.DataFrame()
        df.columns = [c.strip().title() for c in df.columns]
        if key not in df.columns:
            key_alt = [c for c in df.columns if "ident" in c.lower() or "docu" in c.lower()]
            if key_alt: df.rename(columns={key_alt[0]: key}, inplace=True)
        return df

    key = "Identificacion"
    df_ter = _norm(df_ter, key)
    df_pag = _norm(df_pag, key)
    df_pro = _norm(df_pro, key)
    df_aws = _norm(df_aws, key)
    df_tel = _norm(df_tel, key)

    # --- Unificación ---
    base = df_ter.copy()
    for name, df in [("Pagos", df_pag), ("Promesas", df_pro), ("AWS", df_aws), ("Teleton", df_tel)]:
        if df is not None and not df.empty:
            base = base.merge(df, on=key, how="left", suffixes=("", f"_{name}"))

    # --- Limpieza básica ---
    base["DiasMora"] = pd.to_numeric(base.get("DiasMora", np.nan), errors="coerce")
    base["Saldo"] = pd.to_numeric(base.get("Saldo", np.nan), errors="coerce")
    base["UltimaGestion"] = pd.to_datetime(base.get("UltimaGestion", np.nan), errors="coerce")

    # --- Derivados ---
    base["TienePromesa"] = ~base.get("Estado_Promesas", "").astype(str).str.lower().isin(["", "nan", "none"])
    base["TienePago"] = base.get("Valor_Pagos", 0).fillna(0) > 0
    base["DiasDesdeUltimaGestion"] = (_dt.now() - base["UltimaGestion"]).dt.days.fillna(999)

    print(f"Integración completa · {len(base)} registros")
    return base

In [263]:
#SinergIA0820
# === BLOQUE 2 · Aplicación de modelos (Heurístico + ML) ===
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier

def aplicar_modelos(df, modelo="Heurístico"):
    """
    Calcula el score de contacto y propensión según el modelo seleccionado.
    - modelo: 'Heurístico', 'XGBoost', 'LightGBM', 'RandomForest'
    """
    if df is None or df.empty:
        raise ValueError("DataFrame vacío o no válido.")

    print(f"Aplicando modelo: {modelo}")

    # === Modelo heurístico ===
    if modelo == "Heurístico":
        df = df.copy()
        df["ScoreHeuristico"] = (
            (df["Contactabilidad"].fillna(0).astype(float) * 0.6)
            + np.sqrt(df["Saldo"].fillna(0).astype(float)) * 0.2
            + np.sqrt(1 / (df["DiasMora"].fillna(1).astype(float))) * 0.2
        )
        df["ScoreFinal"] = df["ScoreHeuristico"]
        df = df.sort_values("ScoreFinal", ascending=False)
        print("Modelo heurístico aplicado.")

    # === Modelos ML ===
    else:
        # features mínimas (simples para versión demostrativa)
        X_cols = ["DiasMora", "Saldo", "DiasDesdeUltimaGestion"]
        X = df[X_cols].fillna(0)
        y = np.where(df.get("TienePago", False), 1, 0)

        # Entrena un modelo simple si no existe pre-entrenado
        if modelo == "XGBoost":
            model = XGBClassifier(n_estimators=80, learning_rate=0.1, max_depth=3, random_state=42)
        elif modelo == "LightGBM":
            from lightgbm import LGBMClassifier
            model = LGBMClassifier(n_estimators=100, learning_rate=0.05, num_leaves=16)
        elif modelo == "RandomForest":
            from sklearn.ensemble import RandomForestClassifier
            model = RandomForestClassifier(n_estimators=80, max_depth=6, random_state=42)
        else:
            raise ValueError(f"Modelo '{modelo}' no reconocido.")

        model.fit(X, y)
        df["ScoreML"] = model.predict_proba(X)[:, 1]
        scaler = MinMaxScaler()
        df["ScoreFinal"] = scaler.fit_transform(df[["ScoreML"]])
        df = df.sort_values("ScoreFinal", ascending=False)
        print(f"Modelo {modelo} aplicado y calibrado.")

    return df

In [264]:
#SinergIA0840
# === BLOQUE 3 · Integración del flujo con la GUI ===
import ipywidgets as W
from IPython.display import display, HTML, clear_output

def ejecutar_modelo_desde_gui(state: dict, modelo_sel: str, out_log, out_aws, right_tabs):
    """
    Ejecuta el modelo seleccionado (Heurístico o ML) usando los datos cargados
    en las pestañas GUI: Rutina, AWS, Promesas, Pagos y Teletón.
    """
    with out_log:
        out_log.clear_output(wait=True)
        print(f"Iniciando ejecución del modelo: {modelo_sel}")

        try:
            df_ter = state.get("Rutina")
            df_aws = state.get("AWS")
            df_pro = state.get("Promesas")
            df_pag = state.get("Pagos")
            df_tel = state.get("Teleton")

            if df_ter is None:
                print("❌ No se encontró la base de Rutina cargada."); return

            # --- Paso 1: integrar fuentes ---
            df_maestro = integrar_fuentes(df_ter, df_pag, df_pro, df_aws, df_tel)
            state["BaseIntegrada"] = df_maestro

            # --- Paso 2: aplicar modelo ---
            df_result = aplicar_modelos(df_maestro, modelo=modelo_sel)
            state["Entrada_AWS"] = df_result

            # --- Mostrar resultados ---
            with out_aws:
                out_aws.clear_output(wait=True)
                display(HTML(f"<b>Modelo: {modelo_sel}</b> — Ranking top 300"))
                display(df_result.head(300))

            # --- Paso 3: generar resumen para Teleton ---
            resumen = (
                df_result.groupby("Canal", dropna=False)
                .agg({"ScoreFinal": "mean", "Identificacion": "count"})
                .rename(columns={"ScoreFinal": "PromedioScore", "Identificacion": "Cantidad"})
                .reset_index()
            )
            state["TeletonResumen"] = resumen

            right_tabs.selected_index = 3  # pestaña Entrada_AWS
            print("✅ Modelo ejecutado y resultados actualizados correctamente.")

        except Exception as e:
            print("[ERROR durante ejecución del modelo]:", e)

In [265]:
#SinergIA0860
# === BLOQUE 4 · Vínculo GUI ↔ Motor de modelos ===

def vincular_panel_modelos(gui_refs: dict, panel_modelos: W.Accordion):
    """
    Conecta el panel de Modelos con el flujo real de ejecución.
    """
    try:
        # Localiza widgets relevantes
        out_log   = gui_refs.get("out_log")
        out_aws   = gui_refs.get("out_aws")
        right_tab = gui_refs.get("right_tabs")
        state     = gui_refs.get("STATE")

        # Localiza el desplegable de modelos (si no existe aún, créalo)
        w_modelo = W.Dropdown(
            description="Modelo:",
            options=[
                ("Heurístico calibrable", "Heurístico"),
                ("XGBoost aprendido (propensión de pago)", "XGBoost"),
                ("LightGBM (eficiencia de gestión)", "LightGBM"),
                ("Random Forest (patrones mixtos)", "RandomForest"),
            ],
            value="Heurístico",
            layout=W.Layout(width="340px")
        )

        btn_run = W.Button(description="Ejecutar modelo", button_style="info", icon="play")

        def _run(_):
            ejecutar_modelo_desde_gui(state, w_modelo.value, out_log, out_aws, right_tab)

        btn_run.on_click(_run)

        new_box = W.VBox([W.HTML("<b>Seleccionar modelo y ejecutar análisis</b>"),
                           W.HBox([w_modelo, btn_run])])
        panel_modelos.children = tuple(list(panel_modelos.children) + [new_box])
        panel_modelos.set_title(len(panel_modelos.children)-1, "Ejecución de modelo")
        print("✅ Panel de Modelos vinculado correctamente.")

    except Exception as e:
        print("[ERROR] Vinculando panel de modelos:", e)

In [266]:
#SinergIA0900
# === BLOQUE 5 · Generación de listas por canal ===
import pandas as pd
from IPython.display import HTML, display

def generar_listas_por_canal(df_result, state: dict, out_log=None):
    """
    Divide la base Entrada_AWS ordenada en listas por canal de contacto:
    Voz, SMS, WhatsApp.
    Aplica reglas simples (por ahora) que luego pueden reemplazarse con reglas
    específicas de tu hoja ArbolHeuristico.
    """
    if df_result is None or df_result.empty:
        raise ValueError("No hay datos de Entrada_AWS para generar listas.")

    if out_log:
        with out_log:
            out_log.clear_output(wait=True)
            print("Generando listas por canal...")

    # Reglas simples iniciales
    cond_voz = df_result["Canal"].astype(str).str.contains("VOZ", case=False, na=False)
    cond_sms = df_result["Canal"].astype(str).str.contains("SMS", case=False, na=False)
    cond_wpp = df_result["Canal"].astype(str).str.contains("WHATS", case=False, na=False)

    df_voz = df_result[cond_voz].copy()
    df_sms = df_result[cond_sms].copy()
    df_wpp = df_result[cond_wpp].copy()

    # Si no hay canales, aplica heurística básica
    if df_voz.empty and df_sms.empty and df_wpp.empty:
        top = len(df_result) // 3
        df_voz = df_result.head(top).copy()
        df_sms = df_result.iloc[top:2*top].copy()
        df_wpp = df_result.iloc[2*top:].copy()

    # Guarda en el estado
    state["Lista_Voz"] = df_voz
    state["Lista_SMS"] = df_sms
    state["Lista_WPP"] = df_wpp

    # Muestra resumen
    resumen = pd.DataFrame({
        "Canal": ["Voz", "SMS", "WhatsApp"],
        "Cantidad": [len(df_voz), len(df_sms), len(df_wpp)]
    })
    display(HTML("<b>Resumen de listas por canal</b>"))
    display(resumen)

    if out_log:
        with out_log:
            print("Listas generadas correctamente.")
    return resumen

In [267]:
#SinergIA0920
# === BLOQUE 6 · Exportación de listas (Drive y local) ===
import io, csv, datetime as _dt

def exportar_listas_canal(state: dict, formato="csv_comma", out_log=None):
    """
    Exporta las listas Voz, SMS y WhatsApp al Drive configurado o descarga local.
    """
    def _save_csv(df, nombre_base):
        buf = io.StringIO()
        writer = csv.writer(buf)
        writer.writerow(df.columns)
        writer.writerows(df.values)
        data = buf.getvalue().encode("utf-8")
        fecha = _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        nombre = f"{nombre_base}_{fecha}.csv"
        return nombre, data

    canales = [("Voz", "Lista_Voz"), ("SMS", "Lista_SMS"), ("WhatsApp", "Lista_WPP")]
    for canal, key in canales:
        df = state.get(key)
        if df is None or df.empty:
            if out_log:
                with out_log: print(f"[WARN] Sin datos para {canal}.")
            continue

        nombre, data = _save_csv(df, f"{canal}_Contactos")
        if out_log:
            with out_log: print(f"Exportando {canal} → {nombre}")

        try:
            fid = drive_upload_bytes(SALIDAS_ID, nombre, data, "text/csv")
            if out_log:
                with out_log: print(f"✅ {canal} subido a Drive (file_id={fid})")
        except Exception as e:
            if out_log:
                with out_log: print(f"[ERROR] Subiendo {canal}:", e)
            # fallback a descarga local
            from google.colab import files
            files.download(nombre)

    if out_log:
        with out_log: print("🎯 Exportación finalizada.")

In [268]:
#SinergIA0940
# === BLOQUE 7 · Vínculo GUI ↔ Generador y Exportador de Listas ===
import ipywidgets as W
from IPython.display import display, HTML, clear_output

def vincular_listas_a_gui(gui_refs: dict):
    """
    Inserta dentro del panel derecho de la GUI los botones para:
      - Generar listas por canal (Voz, SMS, WhatsApp)
      - Exportarlas automáticamente
    """
    try:
        state     = gui_refs.get("STATE")
        out_log   = gui_refs.get("out_log")
        out_lists = gui_refs.get("out_listas")
        right_tab = gui_refs.get("right_tabs")

        # --- botones principales ---
        btn_gen_listas = W.Button(description="Generar Listas", button_style="info", icon="list")
        btn_exp_listas = W.Button(description="Exportar Listas", button_style="success", icon="cloud-upload")

        def _gen(_):
            with out_lists:
                clear_output(wait=True)
                try:
                    df_result = state.get("Entrada_AWS")
                    if df_result is None or df_result.empty:
                        print("No hay Entrada_AWS disponible.  Ejecuta un modelo primero.")
                        return
                    resumen = generar_listas_por_canal(df_result, state, out_log)
                    display(HTML("<b>Listas generadas correctamente</b>"))
                    display(resumen)
                    right_tab.selected_index = 5  # pestaña Listas
                except Exception as e:
                    print("[ERROR] Generando listas:", e)
                    right_tab.selected_index = 1

        def _exp(_):
            with out_log:
                clear_output(wait=True)
                try:
                    exportar_listas_canal(state, out_log=out_log)
                    print("✅ Listas exportadas (Drive o local).")
                except Exception as e:
                    print("[ERROR] Exportando listas:", e)

        btn_gen_listas.on_click(_gen)
        btn_exp_listas.on_click(_exp)

        panel_box = W.VBox([
            W.HTML("<b>Listas de Contacto</b>"),
            W.HBox([btn_gen_listas, btn_exp_listas])
        ])

        display(panel_box)
        print("✅ Controles de listas integrados en la GUI.")

    except Exception as e:
        print("[ERROR] Vinculando listas a GUI:", e)

In [269]:
#SinergIA0960
# === Panel GUI: Modelos de análisis (Heurístico calibrable) ===
import ipywidgets as W
from IPython.display import display, clear_output, HTML

def modelos_make_panel(gui_refs: dict|None=None):
    """
    Crea un acordeón 'Modelos de análisis' con subpanel 'Heurístico'.
    Si gui_refs se entrega con claves: STATE, out_log, out_aws, right_tabs,
    el botón 'Aplicar ahora' recalcula el score y actualiza la pestaña Entrada_AWS.
    """
    # Widgets
    p0 = modelcfg_load_params()
    w_w_contact = W.FloatSlider(description="w_contact", value=p0["w_contact"], min=0, max=1, step=0.05, readout_format=".2f")
    w_w_saldo   = W.FloatSlider(description="w_saldo",   value=p0["w_saldo"],   min=0, max=1, step=0.05, readout_format=".2f")
    w_w_mora    = W.FloatSlider(description="w_mora",    value=p0["w_mora"],    min=0, max=1, step=0.05, readout_format=".2f")
    w_norm      = W.Dropdown(description="Saldo · norm", options=[("log1p","log1p"),("raíz","sqrt"),("min–max","minmax")],
                             value=p0.get("saldo_norm","log1p"))
    w_half      = W.BoundedIntText(description="Mora · half-life días", value=p0.get("mora_half_life_days",15), min=1, max=120)
    w_tree      = W.Checkbox(description="Usar ÁrbolHeuristico", value=bool(p0.get("use_tree", True)))
    w_cap       = W.FloatSlider(description="Cap |rules|", value=p0.get("max_rule_weight_abs",0.5), min=0, max=1, step=0.05)

    btn_save  = W.Button(description="Guardar", button_style="success")
    btn_reset = W.Button(description="Restablecer")
    btn_apply = W.Button(description="Aplicar ahora", button_style="info")

    def _widgets_to_params():
        return {
            "w_contact": float(w_w_contact.value),
            "w_saldo":   float(w_w_saldo.value),
            "w_mora":    float(w_w_mora.value),
            "saldo_norm": str(w_norm.value),
            "mora_half_life_days": int(w_half.value),
            "use_tree": bool(w_tree.value),
            "max_rule_weight_abs": float(w_cap.value),
        }

    out_msg = W.Output()

    def _on_save(_):
        with out_msg:
            out_msg.clear_output()
            try:
                fid = modelcfg_save_params(_widgets_to_params())
                print(f"Parámetros guardados en MODELOS · file_id={fid}")
            except Exception as e:
                print("[ERROR] Guardando parámetros:", e)

    def _on_reset(_):
        with out_msg:
            out_msg.clear_output()
            try:
                p = modelcfg_default_params()
                w_w_contact.value = p["w_contact"]
                w_w_saldo.value   = p["w_saldo"]
                w_w_mora.value    = p["w_mora"]
                w_norm.value      = p["saldo_norm"]
                w_half.value      = p["mora_half_life_days"]
                w_tree.value      = p["use_tree"]
                w_cap.value       = p["max_rule_weight_abs"]
                print("Parámetros restablecidos (no guardado).")
            except Exception as e:
                print("[ERROR] Reset:", e)

    def _on_apply(_):
        if gui_refs is None:
            with out_msg:
                out_msg.clear_output()
                print("Aplicación local: no tengo referencias de la GUI.  Guardado OK.")
            return
        STATE     = gui_refs.get("STATE")
        out_log   = gui_refs.get("out_log")
        out_aws   = gui_refs.get("out_aws")
        right_tab = gui_refs.get("right_tabs")
        with out_log:
            out_log.clear_output()
            try:
                # Base de trabajo: si ya hay Entrada_AWS en memoria, úsala.  Si no, usa la última ‘Rutina’.
                df_base = STATE.get("Entrada_AWS") if STATE.get("Entrada_AWS") is not None else STATE.get("Rutina")
                if df_base is None or df_base.empty:
                    print("No hay datos en memoria.  Previsualiza y construye Entrada_AWS primero.")
                    return
                heur_df = load_arbol_heuristico() if w_tree.value else pd.DataFrame()
                params  = _widgets_to_params()
                scored  = apply_model_heuristico(df_base, heur_df=heur_df, params=params)
                STATE["Entrada_AWS"] = scored
                with out_aws:
                    out_aws.clear_output()
                    display(HTML("<b>Entrada_AWS (recalculada)</b>"))
                    display(scored.head(300))
                right_tab.selected_index = 3  # pestaña Entrada_AWS
                print("Aplicado OK.  Ranking actualizado con parámetros actuales.")
            except Exception as e:
                print("[ERROR] Aplicando heurístico:", e)

    btn_save.on_click(_on_save)
    btn_reset.on_click(_on_reset)
    btn_apply.on_click(_on_apply)

    panel_heur = W.VBox([
        W.HTML("<b>Heurístico calibrable</b>"),
        W.HBox([w_w_contact, w_w_saldo, w_w_mora]),
        W.HBox([w_norm, w_half]),
        W.HBox([w_tree, w_cap]),
        W.HBox([btn_save, btn_reset, btn_apply]),
        out_msg
    ])

    acc = W.Accordion(children=[panel_heur])
    acc.set_title(0, "Modelos de análisis")
    acc.selected_index = None
    return acc

# === Cómo montarlo debajo de tu GUI actual ===
# Si tu render retorna refs, pásalas aquí para habilitar “Aplicar ahora” con refresco en la pestaña Entrada_AWS.
try:
    gui_refs  # si ya existe por una ejecución previa
except NameError:
    gui_refs = None

panel_modelos = modelos_make_panel(gui_refs)
display(panel_modelos)

Accordion(children=(VBox(children=(HTML(value='<b>Heurístico calibrable</b>'), HBox(children=(FloatSlider(valu…

In [270]:
#SinergIA1020
# === Módulo 1 — Panel de modelos de análisis completo ===
import ipywidgets as W
from IPython.display import display, HTML, clear_output
import pandas as pd

def modelos_make_panel_v2(gui_refs: dict|None=None):
    """
    Panel extendido con selección de modelo (Heurístico, XGBoost, LightGBM, RandomForest).
    Permite configurar pesos heurísticos y recalcular Entrada_AWS desde la GUI.
    """
    modelo_desc = {
        "Heurístico": "Modelo asistido: usa pesos configurables y reglas del ÁrbolHeurístico.",
        "XGBoost": "Modelo de boosting basado en árboles. Ideal para grandes volúmenes y no linealidades.",
        "LightGBM": "Boosting rápido optimizado por histogramas. Excelente para datasets medianos-grandes.",
        "RandomForest": "Ensemble de árboles aleatorios. Robusto y fácil de interpretar."
    }

    dd_modelo = W.Dropdown(
        description="Modelo:",
        options=list(modelo_desc.keys()),
        value="Heurístico"
    )

    w_w_contact = W.FloatSlider(description="w_contact", value=0.6, min=0, max=1, step=0.05)
    w_w_saldo   = W.FloatSlider(description="w_saldo",   value=0.2, min=0, max=1, step=0.05)
    w_w_mora    = W.FloatSlider(description="w_mora",    value=0.2, min=0, max=1, step=0.05)
    w_norm      = W.Dropdown(description="Norm Saldo", options=["log1p","sqrt","minmax"], value="log1p")

    btn_apply = W.Button(description="Aplicar modelo", button_style="info")
    out_log   = W.Output()

    def _on_apply(_):
        with out_log:
            out_log.clear_output()
            modelo = dd_modelo.value
            print(f"Aplicando modelo: {modelo}")
            if gui_refs is None:
                print("Sin conexión a GUI. Solo modo visual.")
                return
            STATE = gui_refs.get("STATE")
            out_aws = gui_refs.get("out_aws")
            if STATE is None:
                print("[ERROR] No hay datos activos en memoria.")
                return

            # Simulación del recalculo heurístico
            df = STATE.get("Entrada_AWS") or STATE.get("Rutina")
            if df is None or df.empty:
                print("No hay datos para procesar. Usa 'Construir Entrada_AWS' primero.")
                return

            df = df.copy()
            if modelo == "Heurístico":
                df["_score"] = (
                    df.get("Contactabilidad", pd.Series(0)).fillna(0) * w_w_contact.value +
                    df.get("Saldo", pd.Series(0)).fillna(0).astype(float).pow(0.25) * w_w_saldo.value +
                    df.get("DiasMora", pd.Series(0)).fillna(0).rpow(0.25) * w_w_mora.value
                )
            else:
                # Simulación: agregar columna de score ficticio (será reemplazado por modelo real)
                import numpy as np
                df["_score"] = np.random.rand(len(df))

            df.sort_values(by="_score", ascending=False, inplace=True)
            STATE["Entrada_AWS"] = df

            with out_aws:
                out_aws.clear_output()
                display(HTML(f"<b>Entrada_AWS — {modelo}</b>"))
                display(df.head(20))

            print(f"Ranking actualizado con {modelo}.")

    btn_apply.on_click(_on_apply)

    panel = W.VBox([
        dd_modelo,
        W.HTML("<b>Pesos heurísticos (solo aplica a modo Heurístico):</b>"),
        W.HBox([w_w_contact, w_w_saldo, w_w_mora]),
        w_norm,
        btn_apply,
        out_log
    ])

    acc = W.Accordion(children=[panel])
    acc.set_title(0, "Modelos de análisis")
    acc.selected_index = None
    return acc

# Montar debajo de la GUI principal (usa gui_refs si existe)
try:
    gui_refs
except NameError:
    gui_refs = None

panel_modelos = modelos_make_panel_v2(gui_refs)
display(panel_modelos)

Accordion(children=(VBox(children=(Dropdown(description='Modelo:', options=('Heurístico', 'XGBoost', 'LightGBM…

In [271]:
#SinergIA1040
# === Integración de panel de Modelos de análisis en GUI existente ===
try:
    # Si GUI ya existe, agregamos el panel dentro del lado derecho (right_tabs)
    gui_refs = GUI if isinstance(GUI, dict) else None
except NameError:
    gui_refs = None

if gui_refs:
    try:
        panel_modelos = modelos_make_panel(gui_refs)
        if "right_tabs" in gui_refs and gui_refs["right_tabs"] is not None:
            rt = gui_refs["right_tabs"]
            # Agregar nueva pestaña si no existe
            tabs = list(rt.children)
            tabs.append(panel_modelos)
            rt.children = tuple(tabs)
            rt.set_title(len(rt.children)-1, "Modelos de análisis")
            print("✅ Panel 'Modelos de análisis' agregado dentro de la GUI existente.")
        else:
            display(panel_modelos)
            print("⚠️ No encontré right_tabs, mostrándolo debajo temporalmente.")
    except Exception as e:
        print("[ERROR] Al intentar agregar el panel de Modelos:", e)
else:
    print("[WARN] GUI aún no inicializada. Ejecuta primero el launcher.")

[WARN] GUI aún no inicializada. Ejecuta primero el launcher.


In [272]:
#SinergIA1060
# === Integración robusta del panel "Modelos de análisis" ===
import time
from IPython.display import display

# Intentar esperar si GUI aún no existe
for i in range(3):
    try:
        if 'GUI' in globals() and isinstance(GUI, dict):
            break
    except:
        pass
    time.sleep(0.5)

if 'GUI' not in globals() or not isinstance(GUI, dict):
    print("[WARN] GUI aún no disponible. Se mostrará el panel temporalmente.")
    gui_refs = None
else:
    gui_refs = GUI

try:
    panel_modelos = modelos_make_panel(gui_refs)
    if gui_refs and "right_tabs" in gui_refs and gui_refs["right_tabs"] is not None:
        rt = gui_refs["right_tabs"]
        tabs = list(rt.children)
        tabs.append(panel_modelos)
        rt.children = tuple(tabs)
        rt.set_title(len(rt.children)-1, "Modelos de análisis")
        print("✅ Panel 'Modelos de análisis' integrado dentro de la GUI.")
    else:
        display(panel_modelos)
        print("⚠️ Mostrando el panel de Modelos debajo (right_tabs no detectado).")
except Exception as e:
    print("[ERROR] Integrando panel de modelos:", e)

[WARN] GUI aún no disponible. Se mostrará el panel temporalmente.


Accordion(children=(VBox(children=(HTML(value='<b>Heurístico calibrable</b>'), HBox(children=(FloatSlider(valu…

⚠️ Mostrando el panel de Modelos debajo (right_tabs no detectado).


In [273]:
#SinergIA1080
# === Módulo 2 — Tema naranja SinergIA ===
from IPython.display import HTML, display

def aplicar_tema_sinergia():
    style = """
    <style>
    /* ----------- SinergIA Visual Theme ----------- */
    :root {
        --sinergia-bg: #181818;
        --sinergia-card: #222;
        --sinergia-border: #ff8c00;
        --sinergia-text: #eaeaea;
        --sinergia-accent: #ffa733;
    }
    .sg-card {
        background: var(--sinergia-card);
        border: 1px solid var(--sinergia-border);
        border-radius: 14px;
        box-shadow: 0 3px 14px rgba(255, 140, 0, 0.15);
        padding: 16px;
    }
    .sg-title {
        color: var(--sinergia-accent);
        font-weight: 700;
        font-size: 22px;
        margin: 2px 0 10px;
    }
    .sg-sub {
        color: var(--sinergia-text);
        opacity: 0.7;
        font-size: 13px;
        margin-bottom: 6px;
    }
    .sg-btn button {
        background: var(--sinergia-border) !important;
        color: #fff !important;
        border-radius: 999px !important;
        border: none !important;
        padding: 6px 14px !important;
        transition: background 0.2s;
    }
    .sg-btn button:hover {
        background: var(--sinergia-accent) !important;
        color: #000 !important;
    }
    .widget-label { color: var(--sinergia-text) !important; }
    input, select, textarea {
        background-color: #2a2a2a !important;
        color: #f5f5f5 !important;
        border: 1px solid var(--sinergia-border) !important;
    }
    </style>
    """
    display(HTML(style))
    print("✅ Tema SinergIA naranja aplicado.")

# Ejecuta para aplicar el tema
aplicar_tema_sinergia()

✅ Tema SinergIA naranja aplicado.


In [274]:
#SinergIA1100
# === Panel "Modelos de análisis" con selector de modelo principal ===
import ipywidgets as W
from IPython.display import display, HTML

def modelos_panel_v2(gui_refs: dict|None=None):
    modelos = [
        ("Heurístico asistido", "Modelo basado en reglas ajustables según pesos de contacto, mora y saldo."),
        ("XGBoost aprendido", "Modelo supervisado: predice la probabilidad de pago en ventana objetivo."),
        ("LightGBM", "Modelo eficiente para grandes volúmenes de datos tabulares."),
        ("Random Forest", "Modelo estable y robusto para priorización de clientes."),
    ]

    sel_modelo = W.Dropdown(
        options=[m[0] for m in modelos],
        value="Heurístico asistido",
        description="Modelo activo:"
    )

    info = W.HTML("<i>Selecciona el modelo para aplicar al ranking de clientes.</i>")
    btn_aplicar = W.Button(description="Aplicar modelo", button_style="info")
    out_msg = W.Output()

    def _on_aplicar(_):
        with out_msg:
            out_msg.clear_output()
            m = sel_modelo.value
            print(f"🔹 Modelo seleccionado: {m}")
            if m == "Heurístico asistido":
                print("→ Se aplicará el motor heurístico calibrado.")
            elif m == "XGBoost aprendido":
                print("→ Ejecutando modelo supervisado (probabilidad de pago).")
            elif m == "LightGBM":
                print("→ Ejecutando LightGBM para ranking optimizado.")
            elif m == "Random Forest":
                print("→ Ejecutando bosque aleatorio para priorización.")
            print("Actualiza la pestaña Entrada_AWS para visualizar los resultados.")

    btn_aplicar.on_click(_on_aplicar)

    panel = W.VBox([
        W.HTML("<b>Modelos de análisis</b>"),
        sel_modelo,
        info,
        btn_aplicar,
        out_msg
    ])

    return panel

# === Inserción directa en GUI existente ===
try:
    if GUI.get("right_tabs") is not None:
        rt = GUI["right_tabs"]
        tabs = list(rt.children)
        panel_modelos = modelos_panel_v2(GUI)
        tabs.append(panel_modelos)
        rt.children = tuple(tabs)
        rt.set_title(len(rt.children)-1, "Modelos de análisis")
        print("✅ Panel 'Modelos de análisis' agregado correctamente a la GUI.")
    else:
        print("[WARN] GUI no tiene right_tabs, mostrando panel independiente.")
        display(modelos_panel_v2(GUI))
except Exception as e:
    print("[ERROR] No se pudo insertar el panel de modelos:", e)

[ERROR] No se pudo insertar el panel de modelos: name 'GUI' is not defined


##Cableado de la GUI (1111-1239)

In [275]:
#SinergIA1120
#  este es el launcher en ejecucion
# === Launcher robusto + Modo UI (con tema SinergIA aplicado automáticamente) ===

"""
import ipywidgets as W
from IPython.display import display, HTML

def _find_gui_func():
    cands = [(k,v) for k,v in globals().items() if callable(v) and k.startswith("render_sinergia_gui_like_gradio")]
    if not cands:
        raise NameError("No hay ninguna función 'render_sinergia_gui_like_gradio*' definida aún.")
    if "render_sinergia_gui_like_gradio" in globals():
        return globals()["render_sinergia_gui_like_gradio"]
    return cands[-1][1]

# Aplica tema naranja (llama al módulo 2 desde aquí)
try:
    aplicar_tema_sinergia()
except NameError:
    print("[WARN] Tema SinergIA no definido todavía (ejecuta Módulo 2).")

# 1) Crea/obtiene la GUI
_gui_func = _find_gui_func()
_gui_result = _gui_func()

# Normaliza el resultado a diccionario (aunque la función devuelva None u otra cosa)
if isinstance(_gui_result, dict):
    GUI = _gui_result
else:
    GUI = {}

# 2) Construye un mapeo tolerante a faltantes para el Modo UI
def _dig(d, *keys):
    cur = d
    for k in keys:
        if not isinstance(cur, dict):
            return None
        cur = cur.get(k)
    return cur

_widgets = {
    "folder_rut":  GUI.get("folder_rut")  or _dig(GUI, "rut", "folder"),
    "refresh_rut": GUI.get("refresh_rut") or _dig(GUI, "rut", "refresh"),
    "det_cols":    GUI.get("det_cols"),
    "fmt_dd":      GUI.get("fmt_dd"),
}

# 3) Modo UI: mostrar/ocultar controles técnicos
def _set_dev_visibility(dev: bool, wd: dict):
    for key in ("folder_rut", "refresh_rut", "det_cols", "fmt_dd"):
        w = wd.get(key)
        if w is not None:
            w.layout.display = "" if dev else "none"

_mode_tb = W.ToggleButtons(
    options=["Desarrollo","Operación"],
    value="Desarrollo",
    description="Modo UI:"
)
display(_mode_tb)

def _on_mode_change(change):
    _set_dev_visibility(change["new"] == "Desarrollo", _widgets)

_set_dev_visibility(True, _widgets)
_mode_tb.observe(_on_mode_change, "value")
"""

'\nimport ipywidgets as W\nfrom IPython.display import display, HTML\n\ndef _find_gui_func():\n    cands = [(k,v) for k,v in globals().items() if callable(v) and k.startswith("render_sinergia_gui_like_gradio")]\n    if not cands:\n        raise NameError("No hay ninguna función \'render_sinergia_gui_like_gradio*\' definida aún.")\n    if "render_sinergia_gui_like_gradio" in globals():\n        return globals()["render_sinergia_gui_like_gradio"]\n    return cands[-1][1]\n\n# Aplica tema naranja (llama al módulo 2 desde aquí)\ntry:\n    aplicar_tema_sinergia()\nexcept NameError:\n    print("[WARN] Tema SinergIA no definido todavía (ejecuta Módulo 2).")\n\n# 1) Crea/obtiene la GUI\n_gui_func = _find_gui_func()\n_gui_result = _gui_func()\n\n# Normaliza el resultado a diccionario (aunque la función devuelva None u otra cosa)\nif isinstance(_gui_result, dict):\n    GUI = _gui_result\nelse:\n    GUI = {}\n\n# 2) Construye un mapeo tolerante a faltantes para el Modo UI\ndef _dig(d, *keys):\n  

In [276]:
#SinergIA1140
# === BLOQUE 8 · Integración final GUI + Modelos + Listas ===
try:
    vincular_panel_modelos(GUI, panel_modelos)
except Exception as e:
    print("[WARN] Panel de Modelos no vinculado:", e)

try:
    vincular_listas_a_gui(GUI)
except Exception as e:
    print("[WARN] Listas no integradas:", e)

print("🚀 GUI SinergIA Cobros totalmente operativa con Modelos + Listas.")

[WARN] Panel de Modelos no vinculado: name 'GUI' is not defined
[WARN] Listas no integradas: name 'GUI' is not defined
🚀 GUI SinergIA Cobros totalmente operativa con Modelos + Listas.


In [277]:
#SinergIA1160
# === BLOQUE 9 · Visualización de métricas de desempeño del modelo ===
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, display, clear_output

def visualizar_desempeno_modelo(state: dict, out_viz=None):
    """
    Genera visualizaciones simples para el modelo actual:
    - Distribución de Score
    - Promedio de Score por canal
    - Correlación con pago (si existen columnas de Promesa/Pago)
    """
    df = state.get("Entrada_AWS")
    if df is None or df.empty:
        if out_viz:
            with out_viz:
                clear_output()
                print("No hay resultados para visualizar.")
        return

    with out_viz:
        clear_output(wait=True)
        print("📊 Visualizando desempeño del modelo...")
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))

        # --- Histograma de Score ---
        axes[0].hist(df["ScoreFinal"], bins=20, color="darkorange", edgecolor="white")
        axes[0].set_title("Distribución de Score")
        axes[0].set_xlabel("ScoreFinal")
        axes[0].set_ylabel("Frecuencia")

        # --- Promedio de Score por canal ---
        if "Canal" in df.columns:
            canal_score = df.groupby("Canal")["ScoreFinal"].mean().sort_values(ascending=False)
            canal_score.plot(kind="barh", ax=axes[1], color="orange")
            axes[1].set_title("Promedio de Score por Canal")
        else:
            axes[1].axis("off")

        # --- Correlación con pago/promesa ---
        if "Pago" in df.columns or "Promesa" in df.columns:
            var_target = "Pago" if "Pago" in df.columns else "Promesa"
            df[var_target] = df[var_target].astype(float)
            axes[2].scatter(df["ScoreFinal"], df[var_target], alpha=0.3, color="orangered")
            axes[2].set_title(f"Relación Score vs {var_target}")
            axes[2].set_xlabel("ScoreFinal")
            axes[2].set_ylabel(var_target)
        else:
            axes[2].axis("off")

        plt.tight_layout()
        plt.show()

        # --- Resumen numérico ---
        resumen = {
            "Filas procesadas": len(df),
            "Score promedio": df["ScoreFinal"].mean(),
            "Score máximo": df["ScoreFinal"].max(),
            "Score mínimo": df["ScoreFinal"].min(),
        }
        display(HTML("<b>Resumen del modelo</b>"))
        display(pd.DataFrame([resumen]))

In [278]:
#SinergIA1180
# === BLOQUE 10 · Vincular visualización con GUI ===
import ipywidgets as W
from IPython.display import display, clear_output

def vincular_visualizacion_a_gui(gui_refs: dict):
    """
    Crea una pestaña adicional en la GUI llamada 'Visualización'
    y agrega un botón para mostrar gráficos de desempeño.
    """
    try:
        state     = gui_refs.get("STATE")
        right_tab = gui_refs.get("right_tabs")
        out_log   = gui_refs.get("out_log")

        # Crear pestaña de visualización si no existe
        out_viz = W.Output()
        if len(right_tab.children) < 7:
            right_tab.children = tuple(list(right_tab.children) + [out_viz])
            right_tab.set_title(len(right_tab.children)-1, "Visualización")

        btn_viz = W.Button(description="Mostrar visualización", button_style="warning", icon="bar-chart-2")

        def _viz(_):
            visualizar_desempeno_modelo(state, out_viz)
            right_tab.selected_index = len(right_tab.children)-1

        btn_viz.on_click(_viz)
        display(W.VBox([
            W.HTML("<b>Visualización de resultados</b>"),
            btn_viz
        ]))
        print("✅ Visualización integrada correctamente a la GUI.")

    except Exception as e:
        print("[ERROR] Vinculando visualización:", e)

In [279]:
#SinergIA1200
# === BLOQUE 11 · Generador de Export Consolidado ===
import io, pandas as pd, datetime as _dt
from IPython.display import HTML, display

def generar_export_consolidado(state: dict, out_log=None):
    """
    Combina todos los resultados del pipeline en un solo archivo Excel:
      - Entrada_AWS
      - Listas (Voz, SMS, WhatsApp)
      - Teletón
      - Métricas resumen (si existen)
    Devuelve bytes listos para subir o descargar.
    """
    fecha = _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre = f"SinergIA_Consolidado_{fecha}.xlsx"
    buf = io.BytesIO()

    try:
        with pd.ExcelWriter(buf, engine="xlsxwriter") as writer:
            # Entrada_AWS
            if (df := state.get("Entrada_AWS")) is not None and not df.empty:
                df.to_excel(writer, sheet_name="Entrada_AWS", index=False)
            # Teletón
            if (tel := state.get("Teleton")) is not None and not tel.empty:
                tel.to_excel(writer, sheet_name="Teleton", index=False)
            # Listas
            for canal in ["Lista_Voz", "Lista_SMS", "Lista_WPP"]:
                if (df := state.get(canal)) is not None and not df.empty:
                    df.to_excel(writer, sheet_name=canal.replace("Lista_",""), index=False)
            # Métricas básicas
            if (aws := state.get("Entrada_AWS")) is not None and "ScoreFinal" in aws.columns:
                resumen = pd.DataFrame([{
                    "Filas": len(aws),
                    "Score promedio": aws["ScoreFinal"].mean(),
                    "Score máx": aws["ScoreFinal"].max(),
                    "Score mín": aws["ScoreFinal"].min()
                }])
                resumen.to_excel(writer, sheet_name="Métricas", index=False)

        data = buf.getvalue()

        if out_log:
            with out_log:
                print(f"✅ Consolidado generado: {nombre} · {len(data)//1024} KB")
        return nombre, data

    except Exception as e:
        if out_log:
            with out_log:
                print("[ERROR] Generando consolidado:", e)
        raise

In [280]:
#SinergIA1220
# === BLOQUE 12 · Vincular Export Consolidado a GUI ===
import ipywidgets as W
from IPython.display import display, clear_output

def vincular_export_consolidado_a_gui(gui_refs: dict):
    """
    Agrega un botón al panel derecho que permite exportar todo el pipeline
    (Entrada_AWS, Listas, Teletón, Métricas) en un único archivo Excel.
    """
    try:
        state     = gui_refs.get("STATE")
        out_log   = gui_refs.get("out_log")
        right_tab = gui_refs.get("right_tabs")

        # Crear botón
        btn_export_all = W.Button(
            description="Exportar Consolidado",
            button_style="warning",
            icon="file-excel"
        )

        def _on_export(_):
            with out_log:
                clear_output(wait=True)
                try:
                    nombre, data = generar_export_consolidado(state, out_log)
                    fid = drive_upload_bytes(SALIDAS_ID, nombre, data,
                                             "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
                    print(f"📦 Export consolidado subido a Drive: {nombre} · file_id={fid}")
                    # Descarga local inmediata
                    from google.colab import files
                    files.download(nombre)
                except Exception as e:
                    print("[ERROR] Export consolidado:", e)

        btn_export_all.on_click(_on_export)
        display(W.VBox([
            W.HTML("<b>Export Consolidado</b>"),
            btn_export_all
        ]))
        print("✅ Botón de export consolidado agregado a la GUI.")

    except Exception as e:
        print("[ERROR] Vinculando export consolidado:", e)

##Constructor GUI (1240-1360)

In [281]:
#SinergIA1240
#Helpers de Estructuras + Limpieza Rutina v1
import pandas as pd

# ID del gsheet de configuración (si usas read_gsheet)
CONFIG_SPREADSHEET_ID = "1q0yWNo35lPeNXmdOgMRKvYx1TcHgxJhPPDo4zXq9p_0"

# Nombre del archivo local, por si lo tienes descargado en el entorno
# CONFIG_LOCAL_XLSX = "Config_SinergIA.xlsx"
# No se lee desde local sino desde el archivo gsheet con la ID hardcodeada

_ESTRUCTURAS_DF = None

def _load_estructuras_df():
    """
    Carga la hoja 'Estructuras' del Config_SinergIA.
    1) Si existe read_gsheet → usa el gSheet.
    2) Si no → intenta leer el XLSX local.
    Deja el resultado en caché global (_ESTRUCTURAS_DF).
    """
    global _ESTRUCTURAS_DF
    if _ESTRUCTURAS_DF is not None:
        return _ESTRUCTURAS_DF

    # Preferimos gSheet si existe el helper
    if "read_gsheet" in globals():
        df = read_gsheet(CONFIG_SPREADSHEET_ID, "Estructuras")
    else:
        # Fallback: archivo XLSX en el entorno
        df = pd.read_excel(CONFIG_LOCAL_XLSX, sheet_name="Estructuras")

    _ESTRUCTURAS_DF = df
    return df


def get_column_names_from_estructuras(archivo_logico: str, sistema: str = "SinergIA_Cobros"):
    """
    Devuelve la lista de columnas esperadas para un ArchivoLogico
    según la hoja Estructuras (campo NombreColumna).
    """
    df = _load_estructuras_df()

    mask = (
        df["Sistema"].astype(str).str.upper() == sistema.upper()
    ) & (
        df["ArchivoLogico"].astype(str).str.upper() == archivo_logico.upper()
    )

    cols = df.loc[mask, "NombreColumna"].tolist()
    return cols


def basic_clean(df: pd.DataFrame):
    """
    Limpieza base para TER_UNISONO → IN_SISTECREDITO (v1):
    - Normaliza nombres de columnas (strip).
    - Tipifica columnas según Estructuras (Fecha / Numero).
    - Limpia columnas de teléfono (solo dígitos, vacía si <7).
    Devuelve: df_limpio, lista_columnas_telefono
    """
    df = df.copy()

    # Normalizar nombres (evitar espacios raros al inicio/fin)
    df.columns = [str(c).strip() for c in df.columns]

    # Intentamos cargar Estructuras
    try:
        estr = _load_estructuras_df()
        estr_ter = estr[estr["ArchivoLogico"].astype(str).str.upper() == "TER_UNISONO"]
    except Exception:
        # Si algo falla, hacemos limpieza mínima sin tipado
        estr_ter = pd.DataFrame(columns=["NombreColumna", "AliasCanonico", "TipoDato"])

    # Columnas por tipo de dato
    fecha_cols = estr_ter.loc[estr_ter["TipoDato"] == "Fecha", "NombreColumna"].tolist()
    num_cols   = estr_ter.loc[estr_ter["TipoDato"] == "Numero", "NombreColumna"].tolist()

    # Convertir fechas
    for col in fecha_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)

    # Convertir números
    for col in num_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Detectar columnas de teléfono (por nombre y por AliasCanonico)
    tel_nombres_basicos = ["Celular", "Celular2", "Fijo", "Fijo2", "PrimerTelefono"]
    alias_tel = estr_ter.loc[
        estr_ter["AliasCanonico"].astype(str).str.contains("Telefono", case=False, na=False),
        "NombreColumna"
    ].tolist()

    tel_candidates = set(tel_nombres_basicos + alias_tel)
    phone_cols = [c for c in df.columns if c in tel_candidates]

    # Limpiar teléfonos: solo dígitos; vaciar si muy cortos
    for col in phone_cols:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(r"\D+", "", regex=True)  # solo dígitos
        )
        # Si queda con menos de 7 dígitos, lo dejamos vacío
        df[col] = df[col].where(df[col].str.len() >= 7, "")

    return df, phone_cols


In [282]:
#SinergIA1260
#Helpers de Estructuras + Limpieza Rutina v1 (solo gSheet)
import pandas as pd

# Si ya tienes una constante global en SinergIA001, por ejemplo CONFIG_SINERGIA_ID,
# puedes reutilizarla.  Si no, usa directamente el ID que compartiste:
#   1q0yWNo35lPeNXmdOgMRKvYx1TcHgxJhPPDo4zXq9p_0

try:
    CONFIG_SPREADSHEET_ID = CONFIG_SPREADSHEET_ID  # si ya existe, se respeta
except NameError:
    CONFIG_SPREADSHEET_ID = "1q0yWNo35lPeNXmdOgMRKvYx1TcHgxJhPPDo4zXq9p_0"

_ESTRUCTURAS_DF = None


def _load_estructuras_df():
    """
    Carga la hoja 'Estructuras' del gSheet Config_SinergIA usando read_gsheet.
    Deja el resultado en caché global (_ESTRUCTURAS_DF).
    """
    global _ESTRUCTURAS_DF

    if _ESTRUCTURAS_DF is not None:
        return _ESTRUCTURAS_DF

    if "read_gsheet" not in globals():
        raise RuntimeError(
            "read_gsheet no está definida en el entorno. "
            "Ejecuta primero la celda donde se declara read_gsheet (SinergIA001)."
        )

    df = read_gsheet(CONFIG_SPREADSHEET_ID, "Estructuras")
    _ESTRUCTURAS_DF = df
    return df


def get_column_names_from_estructuras(archivo_logico: str, sistema: str = "SinergIA_Cobros"):
    """
    Devuelve la lista de columnas esperadas para un ArchivoLogico
    según la hoja Estructuras (columna NombreColumna).
    Filtra por Sistema y ArchivoLogico.
    """
    df = _load_estructuras_df()

    mask = (
        df["Sistema"].astype(str).str.upper() == sistema.upper()
    ) & (
        df["ArchivoLogico"].astype(str).str.upper() == archivo_logico.upper()
    )

    cols = df.loc[mask, "NombreColumna"].tolist()
    return cols


def basic_clean(df: pd.DataFrame):
    """
    Limpieza base para TER_UNISONO → IN_SISTECREDITO (v1):

    - Normaliza nombres de columnas (strip).
    - Tipifica columnas según Estructuras (TipoDato = Fecha / Numero).
    - Detecta y limpia columnas de teléfono (solo dígitos, vacía si <7 dígitos).

    Devuelve: df_limpio, lista_columnas_telefono
    """
    df = df.copy()

    # Normalizar nombres (evitar espacios raros en encabezados)
    df.columns = [str(c).strip() for c in df.columns]

    try:
        estr = _load_estructuras_df()
        estr_ter = estr[estr["ArchivoLogico"].astype(str).str.upper() == "TER_UNISONO"]
    except Exception:
        estr_ter = pd.DataFrame(columns=["NombreColumna", "AliasCanonico", "TipoDato"])

    # Columnas por tipo de dato (según la hoja Estructuras)
    fecha_cols = estr_ter.loc[estr_ter["TipoDato"] == "Fecha", "NombreColumna"].tolist()
    num_cols   = estr_ter.loc[estr_ter["TipoDato"] == "Numero", "NombreColumna"].tolist()

    # Convertir fechas
    for col in fecha_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)

    # Convertir números
    for col in num_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Detectar columnas de teléfono (por nombre y por AliasCanonico)
    tel_nombres_basicos = ["Celular", "Celular2", "Fijo", "Fijo2", "PrimerTelefono"]
    alias_tel = estr_ter.loc[
        estr_ter["AliasCanonico"].astype(str).str.contains("Telefono", case=False, na=False),
        "NombreColumna"
    ].tolist()

    tel_candidates = set(tel_nombres_basicos + alias_tel)
    phone_cols = [c for c in df.columns if c in tel_candidates]

    # Limpiar teléfonos: solo dígitos; vaciar si muy cortos
    for col in phone_cols:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(r"\D+", "", regex=True)  # solo dígitos
        )
        df[col] = df[col].where(df[col].str.len() >= 7, "")

    return df, phone_cols


In [283]:
#SinergIA1280
#DEV_FLAGS
DEV_FAST = True  # En producción ponlo en False

# Caché en memoria para TER
DF_TER_CACHE = None
DF_TER_CACHE_KEY = None  # (folder_id, archivo, nrows)


In [284]:
#SinergIA1300
#Versión limpia de build_sinergia_gui_parity_v7

import gradio as gr
import pandas as pd
import time

def build_sinergia_gui_parity_v7():
    # ===== Helpers internos =====
    def _log_append(log, msg):
        ts = time.strftime("%H:%M:%S")
        log = log or ""
        return f"{log}\n[{ts}] {msg}".lstrip()

    def _folder_for_tipo(tipo: str) -> str:
        mapping = {
            "Rutina": RUTINAS_ID,
            "AWS": AWS_LOG_ID,
            "Promesas": PROMESAS_ID,
            "Pagos": PAGOS_ID,
        }
        return mapping.get(tipo, RUTINAS_ID)

    def _archivo_logico(tipo: str) -> str:
        mapping = {
            "Rutina": "TER_UNISONO",   # según hoja Estructuras
            "AWS": "AWS_Log",
            "Promesas": "Promesas",
            "Pagos": "Pagos",
        }
        return mapping.get(tipo)

    # ===== States (no se tocan en la UI) =====
    df_rutina_state = gr.State(None)      # TER o IN_SISTECREDITO según etapa
    df_aws_state = gr.State(None)
    df_promesas_state = gr.State(None)
    df_pagos_state = gr.State(None)

    # ===== Callbacks de soporte =====
    def cb_tipo_change(tipo, log):
        folder = _folder_for_tipo(tipo)
        log = _log_append(log, f"Tipo cambiado a {tipo}. Folder por defecto actualizado.")
        dd_clear = gr.update(choices=[], value=None, label="Archivo en Drive")
        return folder, dd_clear, log

    def cb_refresh(tipo, folder_id, log):
        folder_id = folder_id or _folder_for_tipo(tipo)
        if not folder_id:
            log = _log_append(log, f"No hay Folder ID configurado para el tipo '{tipo}'.")
            return gr.update(choices=[], value=None, label="Archivo en Drive"), log

        try:
            if "drive_list_recent" not in globals():
                raise RuntimeError("drive_list_recent no está disponible en este notebook.")

            files = drive_list_recent(folder_id=folder_id)
            names = [f.get("name", "") for f in files if f.get("name")]

            names = names[:6]  # solo 6 más recientes

            if not names:
                log = _log_append(log, "La carpeta está vacía o no hay archivos visibles.")
                return gr.update(choices=[], value=None, label="Archivo en Drive"), log

            log = _log_append(log, f"Se encontraron {len(names)} archivo(s) en la carpeta.")
            return gr.update(choices=names, value=names[0], label="Archivo en Drive"), log

        except Exception as e:
            log = _log_append(log, f"Error al listar archivos en Drive: {e}")
            return gr.update(choices=[], value=None, label="Archivo en Drive"), log

    def _leer_df_desde_drive(tipo, folder_id, archivo, log, nrows=200000):
        """
        Lee un archivo desde Drive usando los helpers de SinergIA.

        Respeta DF_TER_CACHE / DEV_FAST para tipo Rutina.
        """
        import pandas as pd
        global DF_TER_CACHE, DF_TER_CACHE_KEY, DEV_FAST

        folder_id = folder_id or _folder_for_tipo(tipo)

        # 1) Intentar usar caché
        cache_key = (folder_id, archivo, nrows)
        if DEV_FAST and tipo == "Rutina" and DF_TER_CACHE is not None and DF_TER_CACHE_KEY == cache_key:
            log = _log_append(log, "[DEV] Usando TER desde caché en memoria (DF_TER_CACHE).")
            return DF_TER_CACHE.copy(), log

        # 2) Lectura normal
        try:
            fid, _ = drive_find_by_name(folder_id, archivo)
            if not fid:
                log = _log_append(log, f"No se encontró '{archivo}' en la carpeta.")
                return None, log

            name, data = drive_download_file(fid)
            cols, rows = read_preview_any((name, data), nrows=nrows)
            if not cols:
                log = _log_append(log, "El archivo no tiene columnas detectables.")
                return None, log

            df = pd.DataFrame(rows, columns=[str(c) for c in cols])
            log = _log_append(log, f"Archivo '{archivo}' descargado desde Drive. Filas leídas={len(df)}.")

            if DEV_FAST and tipo == "Rutina":
                DF_TER_CACHE = df.copy()
                DF_TER_CACHE_KEY = cache_key
                log = _log_append(log, "[DEV] Caché DF_TER_CACHE actualizado para TER.")

            return df, log

        except Exception as e:
            log = _log_append(log, f"Error leyendo desde Drive: {e}")
            return None, log

    def _leer_df_desde_local(file, log, nrows=200000):
        if file is None:
            log = _log_append(log, "No hay archivo local cargado.")
            return None, log
        try:
            name = file.name
            data = file.read()
            cols, rows = read_preview_any((name, data), nrows=nrows)
            if not cols:
                log = _log_append(log, "El archivo local no tiene columnas detectables.")
                return None, log
            df = pd.DataFrame(rows, columns=[str(c) for c in cols])
            log = _log_append(log, f"Archivo local '{name}' leído. Filas leídas={len(df)}.")
            return df, log
        except Exception as e:
            log = _log_append(log, f"Error leyendo archivo local: {e}")
            return None, log

    def _validar_estructura(tipo, df, log):
        archivo_logico = _archivo_logico(tipo)
        if not archivo_logico:
            log = _log_append(log, f"[VALIDACIÓN] No hay archivo lógico para tipo '{tipo}'.")
            return True, log

        if "get_column_names_from_estructuras" not in globals():
            log = _log_append(
                log,
                "[VALIDACIÓN] get_column_names_from_estructuras no está definida; se omite validación."
            )
            return True, log

        try:
            req_cols = get_column_names_from_estructuras(archivo_logico)
            present_cols = list(df.columns)
            missing = [c for c in req_cols if c not in present_cols]

            log = _log_append(log, f"[VALIDACIÓN] Archivo lógico: {archivo_logico}")
            log = _log_append(log, f"[VALIDACIÓN] Columnas presentes ({len(present_cols)}).")
            log = _log_append(log, f"[VALIDACIÓN] Columnas requeridas ({len(req_cols)}).")

            if missing:
                log = _log_append(log, f"[VALIDACIÓN] FALTAN {len(missing)} columna(s): {missing}")
                return False, log

            log = _log_append(log, f"[VALIDACIÓN] Estructura {archivo_logico} verificada OK.")
            return True, log

        except Exception as e:
            log = _log_append(log, f"[VALIDACIÓN] Error consultando Estructuras ({archivo_logico}): {e}")
            return True, log

    # ===== Callbacks de alto nivel =====
    # Nota: cb_clean_rutina_v2 debe estar definida en otra celda.

    def cb_build(df_rut, df_aws, df_prom, df_pag, log):
        if df_rut is None:
            log = _log_append(log, "No existe Rutina limpia (IN_SISTECREDITO).")
            return pd.DataFrame(), log

        msg = [
            "Preparando Entrada_AWS (Fase 1):",
            f"- Rutina limpia: {len(df_rut)} filas.",
            f"- AWS_Log: {'OK' if df_aws is not None else 'No cargado'}",
            f"- Promesas: {'OK' if df_prom is not None else 'No cargado'}",
            f"- Pagos: {'OK' if df_pag is not None else 'No cargado'}",
        ]

        try:
            ea_cols = get_column_names_from_estructuras("Entrada_AWS")
            df_tmp = pd.DataFrame(columns=ea_cols)
            df_prev = df_tmp.head(50)
            log = _log_append(log, "\n".join(msg))
            log = _log_append(log, "Mostrando estructura base de Entrada_AWS.")
            return df_prev, log
        except Exception as e:
            log = _log_append(log, f"Error leyendo estructura Entrada_AWS: {e}")
            return pd.DataFrame(), log

    def cb_preview(tipo, folder_id, archivo_drive, file_local, muestra_ter, log):
        """
        Previsualiza el archivo origen y, si pasa validación, refresca los estados.
        """
        import pandas as pd

        try:
            log = _log_append(log, f"[PREVIEW] Inicio preview tipo={tipo}")

            # 1) Origen
            if file_local is not None:
                log = _log_append(log, "[PREVIEW] Usando archivo local.")
                df, log = _leer_df_desde_local(file_local, log, nrows=int(muestra_ter or 300))
            else:
                if not archivo_drive:
                    log = _log_append(log, "[PREVIEW] No hay archivo seleccionado en 'Archivo en Drive'.")
                    return (
                        pd.DataFrame(),
                        None, None, None, None,
                        log,
                    )

                log = _log_append(log, f"[PREVIEW] Usando archivo en Drive: {archivo_drive}")
                df, log = _leer_df_desde_drive(
                    tipo, folder_id, archivo_drive, log, nrows=int(muestra_ter or 300)
                )

            # 2) df válido
            if df is None:
                log = _log_append(log, "[PREVIEW] df es None después de la lectura. No se muestra nada.")
                return (
                    pd.DataFrame(),
                    None, None, None, None,
                    log,
                )

            if not isinstance(df, pd.DataFrame):
                log = _log_append(log, f"[PREVIEW] La lectura no devolvió un DataFrame. Tipo={type(df)}")
                return (
                    pd.DataFrame(),
                    None, None, None, None,
                    log,
                )

            log = _log_append(
                log,
                f"[PREVIEW] df leído con shape={df.shape}. Primeras columnas={list(df.columns)[:10]}"
            )

            # 3) Validación de estructura
            ok, log = _validar_estructura(tipo, df, log)

            # 4) Muestra
            try:
                n = int(muestra_ter or 300)
            except Exception:
                n = 300
            n = max(1, min(n, len(df)))
            preview = df.head(n)

            # 5) States
            df_rut = df_aws = df_prom = df_pag = None
            if ok:
                if tipo == "Rutina":
                    df_rut = df
                elif tipo == "AWS":
                    df_aws = df
                elif tipo == "Promesas":
                    df_prom = df
                elif tipo == "Pagos":
                    df_pag = df

                log = _log_append(
                    log,
                    f"[PREVIEW] Datos cargados y validados para {tipo}. Mostrando {n} filas."
                )
            else:
                log = _log_append(
                    log,
                    "[PREVIEW] Estructura no coincide. Se muestra preview pero NO se guarda en state."
                )

            return preview, df_rut, df_aws, df_prom, df_pag, log

        except Exception as e:
            log = _log_append(log, f"[PREVIEW][ERROR] Excepción inesperada: {repr(e)}")
            return pd.DataFrame(), None, None, None, None, log

    # ===== Callbacks de pipeline =====
    # ===== Callbacks de pipeline =====
    def cb_clean(tipo, folder_id, archivo_drive, file_local, muestra_clean, log):
        """
        Limpieza TER → IN_SISTECREDITO (solo tipo 'Rutina').

        - Relee el archivo completo (local o Drive).
        - Aplica basic_clean_sinergia() si existe, si no basic_clean().
        - Construye el DataFrame IN_SISTECREDITO con el mapeo definido.
        - Aplica reglas adicionales de dirección y teléfonos.
        - Calcula Franja usando la hoja Parameters (si read_gsheet está disponible).
        - Devuelve una muestra para la previsualización y el log actualizado.
        """
        import pandas as pd, re, unicodedata

        # --- helpers internos (solo dentro de cb_clean) ---

        def _normalize_address_value(val: str) -> str:
            s = str(val or "").strip()
            if s == "":
                return "SIN DIRECCION"
            # solo números → SIN DIRECCION
            if re.fullmatch(r"\d+", s):
                return "SIN DIRECCION"

            tokens = s.split()
            if not tokens:
                return "SIN DIRECCION"

            # índice de la primera palabra con ≥2 letras
            first_word_idx = None
            for i, tok in enumerate(tokens):
                if re.search(r"[A-Za-zÁÉÍÓÚÜÑ]{2,}", tok):
                    first_word_idx = i
                    break

            if first_word_idx is None:
                return "SIN DIRECCION"

            # si antes de esa palabra hay algún token puramente numérico → CERCA A + resto
            has_leading_numbers = any(
                re.fullmatch(r"\d+", t) for t in tokens[:first_word_idx]
            )
            if has_leading_numbers:
                resto = " ".join(tokens[first_word_idx:])
                return f"CERCA A {resto}"

            return s

        def _normalize_addresses_df(df: pd.DataFrame) -> pd.DataFrame:
            df = df.copy()
            addr_cols = [c for c in df.columns if "direccion" in str(c).lower()]
            for c in addr_cols:
                df[c] = df[c].apply(_normalize_address_value)
            return df

        def _compute_franja(dias_mora_series: pd.Series) -> pd.Series:
            """
            Usa Parameters/FranjaMora si read_gsheet+CONFIG_SPREADSHEET_ID están disponibles.
            Si no, aplica un fallback simple con los rangos que se ven en tu hoja.
            """
            nonlocal log
            try:
                if "read_gsheet" in globals() and "CONFIG_SPREADSHEET_ID" in globals():
                    df_param = read_gsheet(CONFIG_SPREADSHEET_ID, "Parameters")
                    df_franja = df_param[df_param["Parametro"] == "FranjaMora"].copy()
                    df_franja["Desde"] = pd.to_numeric(df_franja["Desde"], errors="coerce").fillna(0)
                    df_franja["Hasta"] = pd.to_numeric(df_franja["Hasta"], errors="coerce").fillna(0)

                    def _f(d):
                        try:
                            x = float(d)
                        except Exception:
                            return ""
                        row = df_franja[(df_franja["Desde"] <= x) & (x <= df_franja["Hasta"])]
                        if not row.empty:
                            return str(row.iloc[0]["Etiqueta"])
                        return ""

                    return dias_mora_series.apply(_f)

            except Exception as e:
                # caemos al fallback
                log = _log_append(
                    log,
                    f"[LIMPIEZA] No se pudo leer Parameters/FranjaMora desde Config: {repr(e)}. "
                    "Se usará fallback local."
                )

            # Fallback con rangos fijos
            def _f_simple(d):
                try:
                    x = float(d)
                except Exception:
                    return ""
                if x < 180:
                    return "Franja_00"
                elif x < 359:
                    return "Franja_01"
                elif x < 499:
                    return "Franja_02"
                elif x < 1999:
                    return "Franja_03"
                else:
                    return "Franja_04"

            return dias_mora_series.apply(_f_simple)

        def _build_telefonos(df_clean: pd.DataFrame, cand_cols) -> pd.DataFrame:
            """
            Construye las columnas 'Telefono 1'...'Telefono 4' a partir de las columnas
            candidatas de teléfono en df_clean.

            - Si no hay columnas candidatas, devuelve 4 columnas vacías (""), pero
              con la MISMA cantidad de filas que df_clean.
            - Solo conserva números de celular: 10 dígitos, empezando por '3'.
            """
            cols_out = ["Telefono 1", "Telefono 2", "Telefono 3", "Telefono 4"]

            # Caso límite: no hay columnas candidatas
            if not cand_cols:
                return pd.DataFrame(
                    {c: [""] * len(df_clean) for c in cols_out},
                    index=df_clean.index,
                )

            telefonos_limpios = []

            # Recorremos fila por fila las columnas candidatas
            for _, row in df_clean[cand_cols].iterrows():
                nums = []

                for c in cand_cols:
                    v = row.get(c, "")

                    if pd.isna(v):
                        continue

                    s = str(v)

                    # Nos quedamos solo con dígitos
                    s = "".join(ch for ch in s if ch.isdigit())

                    if not s:
                        continue

                    # Regla de celulares Colombia: 10 dígitos y empieza por '3'
                    if len(s) != 10 or not s.startswith("3"):
                        continue

                    # Evitar duplicados en la misma fila
                    if s not in nums:
                        nums.append(s)

                # Rellenamos/podamos a máximo 4 teléfonos
                nums = (nums + [""] * 4)[:4]
                telefonos_limpios.append(nums)

            # Construimos DataFrame alineado al índice original
            out = pd.DataFrame(
                telefonos_limpios,
                columns=cols_out,
                index=df_clean.index,
            )

            return out

        def _build_in_siste(df_clean: pd.DataFrame, phone_cols=None) -> pd.DataFrame:
            """
            Mapea TER_UNISONO (ya limpio) → IN_SISTECREDITO, con las reglas de negocio.
            """
            out = pd.DataFrame(index=df_clean.index)

            # Campos 1:1 directos
            if "Identificacion" in df_clean.columns:
                out["Documento de identidad"] = df_clean["Identificacion"]
            else:
                out["Documento de identidad"] = ""

            if "Upper" in df_clean.columns:
                out["Nombre Completo"] = df_clean["Upper"]
            elif "NombreCompleto" in df_clean.columns:
                out["Nombre Completo"] = df_clean["NombreCompleto"]
            else:
                out["Nombre Completo"] = ""

            out["Dirección Del Cliente"] = df_clean.get("DireccionResidencia", "")

            out["Fecha Nacimiento"]      = df_clean.get("FechaNacimiento", "")
            out["Fecha Expedición"]      = df_clean.get("FechaExpedicion", "")
            out["Reportado a Centrales"] = df_clean.get("EstadoCentrales", "")

            # Crédito / fechas / montos
            out["Crédito"]                    = df_clean.get("Codigo", df_clean.get("CreditNumber", ""))
            out["Fecha Creación del crédito"] = df_clean.get("FechaCreacion", "")
            out["Valor"]                      = df_clean.get("ValorCapital", df_clean.get("Valor", ""))
            out["Número de cuotas vencidas"]  = df_clean.get("CuotasVencidas", "")
            out["Fecha de vencimiento"]       = df_clean.get("FechaMasVencida", "")
            out["Días en Mora"]               = df_clean.get("DiasMora", "")
            out["Valor Mora"]                 = df_clean.get("SaldoMora", "")
            out["Cargos Jurídico"]            = df_clean.get("CargosJuridico", "")
            out["Saldo Total"]                = df_clean.get("SaldoTotal", "")

            # Teléfonos 1..4
            if not phone_cols:
                # heurística por nombre de columna
                phone_cols = [
                    c for c in df_clean.columns
                    if any(k in str(c).lower() for k in ("cel", "movil", "tel", "fono"))
                ]
            telefonos = _build_telefonos(df_clean, phone_cols)
            for col in ["Telefono 1", "Telefono 2", "Telefono 3", "Telefono 4"]:
                if col in telefonos.columns:
                    out[col] = telefonos[col]
                else:
                    out[col] = ""

            # Datos de almacén
            if "Nombre" in df_clean.columns:
                out["Almacen"] = df_clean["Nombre"]
            else:
                out["Almacen"] = ""

            if "Nombre.1" in df_clean.columns:
                out["Municipio Almacen"] = df_clean["Nombre.1"]
            else:
                out["Municipio Almacen"] = df_clean.get("Municipio", "")

            out["Direccion Almacen"] = df_clean.get("Direccion", "")
            out["Teléfono Almacén"]  = df_clean.get("PrimerTelefono", "")

            # Status / Servicio / Franja / IN_PRIORIDAD / Tipo Documento / In_Serviceid
            out["Status"] = df_clean.get("CreditNumber", "")

            # Servicio constante "General"
            out["Servicio"] = "General"

            # Franja desde DiasMora
            out["Franja"] = _compute_franja(df_clean.get("DiasMora", pd.Series([], index=df_clean.index)))

            # IN_PRIORIDAD vacío por ahora
            out["IN_PRIORIDAD"] = ""

            # Tipo de Documento viene del TER
            out["Tipo de Documento"] = df_clean.get("Tipo Documento", "")

            # In_Serviceid constante 1
            out["In_Serviceid"] = 1

            # Intentamos ordenar columnas según Estructuras.IN_SISTECREDITO, si está disponible
            try:
                if "get_column_names_from_estructuras" in globals():
                    expected = get_column_names_from_estructuras("IN_SISTECREDITO")
                    if expected:
                        cols_ord = [c for c in expected if c in out.columns]
                        extras   = [c for c in out.columns if c not in cols_ord]
                        if cols_ord:
                            out = out[cols_ord + extras]
            except Exception:
                pass

            return out

        # --- flujo principal de cb_clean ---

        try:
            log = _log_append(log, "[LIMPIEZA] Inicio limpieza TER → IN_SISTECREDITO.")

            # 0) Solo aplica a tipo 'Rutina'
            if tipo != "Rutina":
                log = _log_append(
                    log,
                    "[LIMPIEZA] El tipo seleccionado no es 'Rutina'; no se aplica limpieza."
                )
                return pd.DataFrame(), log

            # 1) Relectura COMPLETA del archivo origen
            if file_local is not None:
                log = _log_append(log, "[LIMPIEZA] Releyendo archivo completo desde LOCAL.")
                df_raw, log = _leer_df_desde_local(
                    file_local, log, nrows=2_000_000
                )
            else:
                if not archivo_drive:
                    log = _log_append(
                        log,
                        "[LIMPIEZA] No hay archivo seleccionado en 'Archivo en Drive'."
                    )
                    return pd.DataFrame(), log

                log = _log_append(
                    log,
                    f"[LIMPIEZA] Releyendo archivo completo desde DRIVE: {archivo_drive}"
                )
                df_raw, log = _leer_df_desde_drive(
                    "Rutina", folder_id, archivo_drive, log, nrows=2_000_000
                )

            if df_raw is None or not isinstance(df_raw, pd.DataFrame) or df_raw.empty:
                log = _log_append(
                    log,
                    "[LIMPIEZA] df_raw vacío o inválido después de la lectura."
                )
                return pd.DataFrame(), log

            log = _log_append(
                log,
                f"[LIMPIEZA] df_raw leído con shape={df_raw.shape}."
            )

            # 2) Normalización suave de columnas (NombreCompleto → Upper)
            if "NombreCompleto" in df_raw.columns and "Upper" not in df_raw.columns:
                df_raw = df_raw.rename(columns={"NombreCompleto": "Upper"})
                log = _log_append(
                    log,
                    "[LIMPIEZA] Columna 'NombreCompleto' renombrada a 'Upper'."
                )

            # 3) Limpieza principal con cascada de fallbacks
            df_clean = None
            phone_cols = None  # columnas candidatas de teléfono (si basic_clean las devuelve)

            if "basic_clean_sinergia" in globals():
                try:
                    log = _log_append(
                        log,
                        "[LIMPIEZA] Ejecutando basic_clean_sinergia()…"
                    )
                    df_clean, log = basic_clean_sinergia(
                        df_raw,
                        log,
                        valores_enteros=True,
                        todos_celulares=False,
                        todo_mayusculas=False,
                        ordenar_por=None,
                    )
                except Exception as e:
                    log = _log_append(
                        log,
                        f"[LIMPIEZA] Error en basic_clean_sinergia: {repr(e)}. Se usará fallback."
                    )
                    df_clean = None

            if df_clean is None and "basic_clean" in globals():
                try:
                    log = _log_append(
                        log,
                        "[LIMPIEZA] Ejecutando basic_clean() como fallback…"
                    )
                    df_clean, phone_cols = basic_clean(df_raw)
                    log = _log_append(
                        log,
                        f"[LIMPIEZA] Limpieza fallback aplicada. Filas={len(df_clean)}. "
                        f"Cols teléfono={phone_cols}"
                    )
                except Exception as e:
                    log = _log_append(
                        log,
                        f"[LIMPIEZA] Error en basic_clean: {repr(e)}. Se usará df_raw sin limpieza dura."
                    )
                    df_clean = None

            if df_clean is None:
                log = _log_append(
                    log,
                    "[LIMPIEZA] No se pudo aplicar ninguna función de limpieza; usando df_raw."
                )
                df_clean = df_raw.copy()

            if df_clean.empty:
                log = _log_append(
                    log,
                    "[LIMPIEZA] df_clean resultó vacío; no hay datos para mostrar."
                )
                return pd.DataFrame(), log

            # 4) Construimos IN_SISTECREDITO a partir de df_clean
            df_siste = _build_in_siste(df_clean, phone_cols)
            log = _log_append(
                log,
                f"[LIMPIEZA] IN_SISTECREDITO construido con shape={df_siste.shape}."
            )

            # 5) Reglas adicionales sobre DIRECCIONES (cliente + almacén)
            df_siste = _normalize_addresses_df(df_siste)
            log = _log_append(
                log,
                "[LIMPIEZA] Reglas de dirección aplicadas (SIN DIRECCION / CERCA A …) sobre IN_SISTECREDITO."
            )

            # 6) Muestra para previsualización
            try:
                n = int(muestra_clean or 500)
            except Exception:
                n = 500
            n = max(1, min(n, len(df_siste)))
            df_show = df_siste.head(n)

            log = _log_append(
                log,
                f"[LIMPIEZA] Mostrando {n} filas de IN_SISTECREDITO (limpia)."
            )

            return df_show, log

        except Exception as e:
            log = _log_append(
                log,
                f"[LIMPIEZA][ERROR] Excepción inesperada en cb_clean: {repr(e)}"
            )
            return pd.DataFrame(), log





    # ===== UI =====
    with gr.Blocks(title="SinergIA Cobros — Loader Parity v7") as app:
        with gr.Row():
            # Columna izquierda
            with gr.Column(scale=1):
                gr.Markdown("## SinergIA Cobros)")

                # Tipo de base
                with gr.Group():
                    gr.Markdown("### Tipo de base")
                    dd_tipo = gr.Dropdown(
                        ["Rutina", "AWS", "Promesas", "Pagos"],
                        value="Rutina",
                        label=None,
                    )

                # Carga
                with gr.Group():
                    gr.Markdown("### Carga")
                    with gr.Tabs():
                        with gr.Tab("Drive"):
                            with gr.Row():
                                txt_folder = gr.Textbox(
                                    label="Folder ID",
                                    value=_folder_for_tipo("Rutina"),
                                    interactive=False,
                                )
                                btn_refresh = gr.Button("⟳", scale=0)
                            dd_archivo = gr.Dropdown(
                                choices=[],
                                value=None,
                                label="Archivo en Drive",
                            )
                        with gr.Tab("Local"):
                            file_local = gr.File(
                                label="Archivo local",
                                interactive=True,
                            )

                    sl_muestra_ter = gr.Slider(
                        minimum=100,
                        maximum=10000,
                        value=300,
                        step=100,
                        label="Muestra TER (filas para previsualizar)",
                    )

                # Botón preview
                btn_prev = gr.Button("Previsualizar")

                # Normalización
                with gr.Accordion("Normalización de la data TER", open=False):
                    sl_muestra_clean = gr.Slider(
                        minimum=100,
                        maximum=10000,
                        value=500,
                        step=100,
                        label="Muestra limpia (filas para previsualizar)",
                    )

                    chk_valores_enteros = gr.Checkbox(
                        value=True,
                        label="Forzar valores numéricos a enteros",
                    )
                    chk_todos_cel = gr.Checkbox(
                        value=False,
                        label="Usar todos los celulares disponibles",
                    )
                    chk_mayus = gr.Checkbox(
                        value=True,
                        label="Texto en MAYÚSCULAS",
                    )

                    # Multi-orden: grupo de checks
                    gr.Markdown("**Ordenar por (puedes elegir varios)**")
                    cg_ordenar = gr.CheckboxGroup(
                        choices=[
                            "Nombre",
                            "Identificación",
                            "Fecha nacimiento",
                            "Fecha expedición",
                            "Días mora",
                            "Ciudad",
                            "Compromiso",
                        ],
                        value=["Nombre"],   # predeterminado
                        label=None,
                    )

                btn_clean = gr.Button("Aplicar limpieza (solo Rutina)")

                # Opciones de guardado (placeholder)
                with gr.Accordion("Opciones de guardado", open=False):
                    btn_save = gr.Button("Guardar CSV")
                    btn_exp = gr.Button("Exportar formato Genesys")
                    btn_build = gr.Button("Construir Entrada_AWS")
                    btn_listas = gr.Button("Derivar Listas")
                    fmt = gr.Dropdown(
                        ["CSV (coma)", "CSV (punto y coma)", "XLSX", "Parquet"],
                        value="CSV (coma)",
                        label="Formato de exportación",
                    )
                    btn_desc_local = gr.Button("Descargar local")

                with gr.Accordion("Modelos de análisis", open=False):
                    dd_modelo = gr.Dropdown(
                        ["Heurístico", "XGBoost", "RandomForest", "LightGBM", "Regresión logística"],
                        value="Heurístico",
                        label="Modelo",
                    )
                    s_umbral = gr.Slider(0, 1, value=0.5, label="Umbral decisión")
                    s_peso = gr.Slider(0, 1, value=0.3, label="Peso contacto")
                    btn_calc = gr.Button("Calcular")

                with gr.Accordion("Ley 2300", open=False):
                    s_recontacto = gr.Slider(1, 10, value=4, label="Días recontacto")
                    lv_desde = gr.Textbox(value="07:00", label="L-V desde")
                    lv_hasta = gr.Textbox(value="19:00", label="L-V hasta")
                    sb_desde = gr.Textbox(value="08:00", label="Sáb desde")
                    sb_hasta = gr.Textbox(value="15:00", label="Sáb hasta")
                    chk_dom = gr.Checkbox(label="Permitir domingo", value=False)
                    chk_canal = gr.Checkbox(label="1 canal por semana", value=True)
                    btn_ley = gr.Button("Guardar Ley2300")

                txt_log = gr.Textbox(label="Log de proceso", lines=10)

            # Columna derecha
            with gr.Column(scale=3):
                with gr.Tabs():
                    with gr.Tab("Preview"):
                        with gr.Accordion("Preview archivo origen", open=True):
                            df_prev_rut = gr.Dataframe(label=None)
                        with gr.Accordion("IN_SISTECREDITO (limpia)", open=False):
                            df_prev_in = gr.Dataframe(label=None)
                        with gr.Accordion("Entrada_AWS (preview)", open=False):
                            df_prev_ea = gr.Dataframe(label=None)
                    with gr.Tab("Bases operativas"):
                        df_listas_show = gr.Dataframe(label="Listas por canal (futuro)")

        # ==== Wiring ====

        dd_tipo.change(
            cb_tipo_change,
            inputs=[dd_tipo, txt_log],
            outputs=[txt_folder, dd_archivo, txt_log],
        )

        btn_refresh.click(
            cb_refresh,
            inputs=[dd_tipo, txt_folder, txt_log],
            outputs=[dd_archivo, txt_log],
        )

        btn_prev.click(
            cb_preview,
            inputs=[
                dd_tipo,
                txt_folder,
                dd_archivo,
                file_local,
                sl_muestra_ter,
                txt_log,
            ],
            outputs=[
                df_prev_rut,
                df_rutina_state, df_aws_state, df_promesas_state, df_pagos_state,
                txt_log,
            ],
        )

        # Limpieza Rutina → cb_clean_rutina_v2 YA DEBE EXISTIR
        #SinergIA140_fix · wiring simple del botón de limpieza
        btn_clean.click(
            cb_clean,
            inputs=[
                dd_tipo,         # tipo (debe ser "Rutina")
                txt_folder,      # folder_id
                dd_archivo,      # archivo en Drive
                file_local,      # archivo local (si se usa)
                sl_muestra_clean,# filas para mostrar de la IN limpia
                txt_log,         # log actual
            ],
            outputs=[
                df_prev_in,      # preview IN_SISTECREDITO (limpia)
                txt_log,         # log actualizado
            ],
        )


        btn_build.click(
            cb_build,
            inputs=[df_rutina_state, df_aws_state, df_promesas_state, df_pagos_state, txt_log],
            outputs=[df_prev_ea, txt_log],
        )

    return app


In [285]:
#SinergIA1320
# Orquestadores de backend para SinergIA Cobros (listos para enganchar a Gradio Blocks)

import datetime as dt
from typing import Optional, Dict, Any

import pandas as pd  # ya está en el entorno, pero lo declaramos explícito por claridad


def _fmt_label_to_key(fmt_label: str) -> str:
    """
    Traduce la etiqueta de la GUI a la clave de formato usada por df_to_bytes:
      - "CSV (coma)"          -> "csv_comma"
      - "CSV (punto y coma)"  -> "csv_semicolon"
      - "XLSX"                -> "xlsx"
    """
    txt = (fmt_label or "").lower()
    if "punto y coma" in txt or "semicolon" in txt:
        return "csv_semicolon"
    if "xlsx" in txt or "excel" in txt:
        return "xlsx"
    return "csv_comma"


def sinergia_build_entrada_aws_from_sources(
    df_rutina: pd.DataFrame,
    df_log: Optional[pd.DataFrame] = None,
    df_aws: Optional[pd.DataFrame] = None,
    df_promesas: Optional[pd.DataFrame] = None,
    df_pagos: Optional[pd.DataFrame] = None,
) -> pd.DataFrame:
    """
    Orquesta la construcción de Entrada_AWS a partir de las fuentes disponibles.

    Requiere:
      - df_rutina: base de IN_SISTECREDITO (obligatoria).
      - df_log, df_aws, df_promesas, df_pagos: opcionales (pueden venir como None).

    Usa la **última versión** de build_entrada_aws definida en el notebook.
    Si no existe '_score_total', intenta calcularlo con score_df para ordenar.
    """
    if df_rutina is None or df_rutina.empty:
        raise ValueError("Se requiere al menos la base de Rutina (IN_SISTECREDITO).")

    # build_entrada_aws final tiene firma:
    # build_entrada_aws(df_rutina, df_log, df_aws, df_promesas, df_pagos)
    df_entrada = build_entrada_aws(df_rutina, df_log, df_aws, df_promesas, df_pagos)

    # Score defensivo
    try:
        if "_score_total" not in df_entrada.columns:
            s = score_df(df_entrada)
            if "_score_total" not in df_entrada.columns:
                df_entrada["_score_total"] = s
        # Orden estándar: score descendente
        df_entrada = df_entrada.sort_values("_score_total", ascending=False)
    except Exception:
        # Si algo falla en score, seguimos con la tabla tal cual
        pass

    return df_entrada.reset_index(drop=True)


def sinergia_derivar_listas_desde_entrada(
    df_entrada: pd.DataFrame,
    ley_params: Optional[dict] = None,
) -> Dict[str, pd.DataFrame]:
    """
    Deriva listas por canal a partir de Entrada_AWS, aplicando Ley 2300.

    Usa build_listas_por_canal ya definido:
      - Voz    -> requiere ANI
      - WhatsApp / SMS -> por ahora reutilizan ANI
    """
    if df_entrada is None or df_entrada.empty:
        return {"Voz": pd.DataFrame(), "WhatsApp": pd.DataFrame(), "SMS": pd.DataFrame()}

    # build_listas_por_canal ya llama l2300_get_params si params es None,
    # pero dejamos el hook por si más adelante quieres inyectar overrides.
    listas = build_listas_por_canal(df_entrada, params=ley_params)
    return listas


def sinergia_exportar_entrada_aws(
    df_entrada: pd.DataFrame,
    fmt_label: str = "CSV (coma)",
    file_stub: str = "Entrada_AWS",
) -> Dict[str, Any]:
    """
    Exporta Entrada_AWS a la carpeta SALIDAS_ID usando df_to_bytes + _upload_bytes_salida.

    Devuelve dict con:
      - name: nombre del archivo creado
      - file_id: id en Drive (o None si algo falla)
      - fmt_key: clave interna de formato (csv_comma/csv_semicolon/xlsx)
    """
    if df_entrada is None or df_entrada.empty:
        raise ValueError("No hay datos en df_entrada para exportar.")

    fmt_key = _fmt_label_to_key(fmt_label)
    ext, data, mime = df_to_bytes(df_entrada, fmt_key)

    ts = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"{file_stub}_{ts}.{ext}"

    file_id = None
    try:
        # _upload_bytes_salida usa SALIDAS_ID de tu config
        file_id = _upload_bytes_salida(file_name, data, mime)
    except Exception:
        # Mantenemos fallo silencioso para no romper la GUI; puedes loguear luego.
        file_id = None

    return {"name": file_name, "file_id": file_id, "fmt_key": fmt_key}


def sinergia_exportar_listas_por_canal(
    listas_por_canal: Dict[str, pd.DataFrame],
    fmt_label: str = "CSV (coma)",
    file_stub: str = "Lista",
) -> Dict[str, Dict[str, Any]]:
    """
    Exporta cada DataFrame de listas_por_canal a SALIDAS_ID con nombres diferenciados por canal.

    Retorna un dict:
        {
          "Voz":      {"name": ..., "file_id": ...},
          "WhatsApp": {"name": ..., "file_id": ...},
          "SMS":      {"name": ..., "file_id": ...},
          ...
        }
    """
    resultados: Dict[str, Dict[str, Any]] = {}
    if not listas_por_canal:
        return resultados

    fmt_key = _fmt_label_to_key(fmt_label)

    for canal, df_canal in listas_por_canal.items():
        if df_canal is None or df_canal.empty:
            continue

        ext, data, mime = df_to_bytes(df_canal, fmt_key)
        ts = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        file_name = f"{file_stub}_{canal}_{ts}.{ext}"

        file_id = None
        try:
            file_id = _upload_bytes_salida(file_name, data, mime)
        except Exception:
            file_id = None

        resultados[canal] = {
            "name": file_name,
            "file_id": file_id,
            "fmt_key": fmt_key,
        }

    return resultados


In [286]:
#SinergIA1340
# Versión v2 — Limpieza TER → IN_SISTECREDITO

import pandas as pd
import numpy as np
import unicodedata

def _text_normalize(value, todo_mayusculas: bool) -> str:
    """
    Limpieza de texto:
    - str, strip
    - quita tildes
    - deja solo [A-Za-z0-9] + algunos símbolos básicos
    - opcional: mayúsculas sostenidas
    """
    if pd.isna(value):
        return ""

    s = str(value).strip()
    if not s:
        return ""

    # Quitar tildes / marcas diacríticas
    s_norm = unicodedata.normalize("NFD", s)
    s_no_acc = "".join(ch for ch in s_norm
                       if unicodedata.category(ch) != "Mn")

    # Whitelist básica + dígitos
    allowed_extra = " #º°.,-_/"

    allowed = (
        "abcdefghijklmnopqrstuvwxyz"
        "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        "0123456789"
        + allowed_extra
    )

    s_clean = "".join(ch for ch in s_no_acc if ch in allowed)

    if todo_mayusculas:
        s_clean = s_clean.upper()

    return s_clean


def basic_clean_sinergia(
    df: pd.DataFrame,
    log,
    valores_enteros: bool = True,
    todos_celulares: bool = False,
    todo_mayusculas: bool = False,
    ordenar_por: str = None,
):
    """
    Normalización base reutilizable para TER/IN_SISTECREDITO.

    df: dataframe de entrada.
    log: lista para acumular mensajes de log (también admite str o None).
    """

    # --- PARCHE DE SEGURIDAD PARA EL LOG ---
    if log is None:
        log = []
    elif not isinstance(log, list):
        # Si viene un string u otro tipo → lo envolvemos en lista
        log = [str(log)]
    # ---------------------------------------
    """
    Limpieza TER → IN_SISTECREDITO.
    Reglas basadas en Codex Técnico-Operativo y documentos de limpieza.

    Parámetros de control:
    - valores_enteros: True → montos como enteros (sin decimales)
    - todos_celulares: True → usar hasta 4 celulares, False → solo 2
    - todo_mayusculas: True → texto a MAYÚSCULAS
    - ordenar_por: nombre de columna para ordenar (ej. 'documento_cliente')
    """
    try:
        df = df.copy()

        # ==========================
        # 1. DOCUMENTO CLIENTE
        # ==========================
        if "documento_cliente" in df.columns:
            df["documento_cliente"] = (
                df["documento_cliente"]
                .astype(str)
                .str.replace(r"[^0-9]", "", regex=True)
            )
        else:
            log.append("[LIMPIEZA] No existe columna 'documento_cliente'.")

        # ==========================
        # 2. TELEFONOS (normalización básica)
        # ==========================
        cols_tel = [
            c for c in df.columns
            if "tel" in c.lower() or "cel" in c.lower()
        ]
        for c in cols_tel:
            df[c] = (
                df[c]
                .astype(str)
                .str.replace(r"[^0-9]", "", regex=True)
                .str.slice(-10)  # últimos 10 dígitos
            )

        # ==========================
        # 3. EMAIL
        # ==========================
        if "email" in df.columns:
            df["email"] = df["email"].astype(str)
            df.loc[~df["email"].str.contains("@"), "email"] = ""

        # ==========================
        # 4. DIAS MORA → FRANJA (según Parameters.Config_SinergIA)
        # ==========================
        # Intentamos detectar la columna de días de mora
        cand_dias = None
        for c in df.columns:
            cl = c.lower().replace(" ", "")
            if cl in ("diasmora", "dias_mora"):
                cand_dias = c
                break

        if cand_dias is not None:
            df[cand_dias] = pd.to_numeric(df[cand_dias], errors="coerce").fillna(0)

            # Definición tomada de Parameters:
            # Parametro   Nombre      Etiqueta          Desde  Hasta
            # FranjaMora  Franja_00   00. < 180        0      179
            # FranjaMora  Franja_01   01. 180 a 359    180    359
            # FranjaMora  Franja_02   02. 360 a 499    360    499
            # FranjaMora  Franja_03   03. 500 a 1999   500    1999
            # FranjaMora  Franja_04   04. >= 2000      2000   99999

            rangos = [
                (0, 179,   "00. < 180"),
                (180, 359, "01. 180 a 359"),
                (360, 499, "02. 360 a 499"),
                (500, 1999,"03. 500 a 1999"),
                (2000, 99999, "04. >= 2000"),
            ]

            def _franja(d):
                if pd.isna(d):
                    return ""
                for d_desde, d_hasta, etiqueta in rangos:
                    if d_desde <= d <= d_hasta:
                        return etiqueta
                return ""

            df["Franja"] = df[cand_dias].apply(_franja)
        else:
            log.append("[LIMPIEZA] No se encontró columna de días de mora para calcular 'Franja'.")


        # ==========================
        # 5. MONTOS (VALORES NUMÉRICOS CLAVE)
        # ==========================
        # Aquí puedes ampliar la lista según Config_SinergIA
        monto_cols = [
            "saldo_total",
            "saldo_mora",
            "saldo_capital",
            "valor_capital",
            "cargos_juridico",
        ]
        for c in monto_cols:
            if c in df.columns:
                serie = (
                    df[c]
                    .astype(str)
                    .str.replace(r"[^0-9\.-]", "", regex=True)
                )
                serie = pd.to_numeric(serie, errors="coerce")

                if valores_enteros:
                    serie = serie.round(0).astype("Int64")
                else:
                    serie = serie.round(2)

                df[c] = serie

        # ==========================
        # 6. FECHAS
        # ==========================
        fecha_cols = [c for c in df.columns if "fecha" in c.lower()]
        for c in fecha_cols:
            try:
                df[c] = pd.to_datetime(df[c], errors="coerce")
            except Exception:
                df[c] = pd.NaT

        # ==========================
        # 7. FLAGS DE CONTACTO
        # ==========================
        flags = ["no_llamar", "no_whatsapp", "no_email"]
        for c in flags:
            if c in df.columns:
                df[c] = (
                    df[c]
                    .fillna("")
                    .astype(str)
                    .str.lower()
                    .isin(["1", "si", "sí", "true", "x"])
                )

        # ==========================
        # 8. DISTRIBUCIÓN DE CELULARES
        #     → Telefono1..Telefono4
        # ==========================
        # Detectamos columna de teléfono de negocio (si existe)
        primer_col = None
        for cand in ["primertelefono", "telefono_negocio", "tel_negocio"]:
            hits = [c for c in df.columns if c.lower() == cand]
            if hits:
                primer_col = hits[0]
                break

        base_tel_cols = [
            c for c in df.columns
            if any(k in c.lower() for k in ["tel", "cel", "fijo"])
        ]

        if base_tel_cols:
            def _row_phones(row):
                primer = ""
                if primer_col and primer_col in row.index:
                    primer = str(row[primer_col] or "").strip()

                candidatos = []
                for c in base_tel_cols:
                    v = str(row.get(c, "") or "").strip()
                    if not v:
                        continue
                    # solo valores de 10 dígitos
                    if len(v) == 10 and v.isdigit():
                        if primer and v == primer:
                            continue
                        candidatos.append(v)

                # Solo celulares (Colombia: empiezan por '3')
                celulares = [p for p in candidatos if p.startswith("3")]

                if todos_celulares:
                    sel = celulares[:4]
                else:
                    sel = celulares[:2]

                sel = sel + [""] * (4 - len(sel))
                return pd.Series(
                    sel[:4],
                    index=["Telefono 1", "Telefono 2", "Telefono 3", "Telefono 4"],
                )

            telefonos_df = df.apply(_row_phones, axis=1)

            for c in ["Telefono 1", "Telefono 2", "Telefono 3", "Telefono 4"]:
                df[c] = telefonos_df[c]

        # ==========================
        # 9. LIMPIEZA DE TEXTO / UNICODE
        # ==========================
        text_cols = [
            c for c in df.columns
            if df[c].dtype == "object"
        ]

        for c in text_cols:
            df[c] = df[c].apply(
                lambda v: _text_normalize(v, todo_mayusculas)
            )

        # ==========================
        # 10. ORDENAMIENTO
        # ==========================
        if ordenar_por and ordenar_por in df.columns:
            if ordenar_por != "documento_cliente" and "documento_cliente" in df.columns:
                df = df.sort_values(
                    by=[ordenar_por, "documento_cliente"]
                )
            else:
                df = df.sort_values(by=[ordenar_por])

            df = df.reset_index(drop=True)

        # ==========================
        # 11. DUPLICADOS
        # ==========================
        clave = ["documento_cliente"]
        if all(c in df.columns for c in clave):
            df = df.drop_duplicates(subset=clave, keep="last")

        log.append(
            "[LIMPIEZA] Limpieza TER completada. "
            "Se generó IN_SISTECREDITO (normalizada)."
        )

        return df, log

    except Exception as e:
        log.append(f"[ERROR-LIMPIEZA] {e}")
        return pd.DataFrame(), log


In [287]:
#SinergIA1360
# Construcción de IN_SISTECREDITO (preview) a partir de TER_UNISONO
import re
import pandas as pd

def build_in_sistecredito_preview(df_ter: pd.DataFrame) -> pd.DataFrame:
    """
    Construye un dataframe IN_SISTECREDITO usando solo las columnas
    disponibles en TER_UNISONO.  Pensado para PREVIEW (muestra).
    No toca el dataframe original.
    """

    if df_ter is None or df_ter.empty:
        return pd.DataFrame()

    df = df_ter.copy()
    n = len(df)
    idx = df.index

    def col(*names, default=""):
        """Devuelve la primera columna existente entre names; si no, una serie vacía."""
        for name in names:
            if name in df.columns:
                return df[name]
        return pd.Series([default] * n, index=idx)

    out = pd.DataFrame(index=idx)

    # --- Mapeo columnas cliente ---
    out["Documento de identidad"]      = col("Identificacion", "Identificación")
    out["Nombre Completo"]            = col("Upper", "NombreCompleto", "Nombre Completo")
    out["Dirección Del Cliente"]      = col("DireccionResidencia", "DirecciónResidencia")
    out["Fecha Nacimiento"]           = col("FechaNacimiento")
    out["Fecha Expedición"]           = col("FechaExpedicion")
    out["Reportado a Centrales"]      = col("EstadoCentrales")
    out["Crédito"]                    = col("Codigo", "CreditNumber")
    out["Fecha Creación del crédito"] = col("FechaCreacion", "FechaCreacionCredito")
    out["Valor"]                      = col("ValorCapital", "Valor")
    out["Número de cuotas vencidas"]  = col("CuotasVencidas")
    out["Fecha de vencimiento"]       = col("FechaMasVencida", "FechaVencimiento")
    out["Días en Mora"]               = col("DiasMora")
    out["Valor Mora"]                 = col("SaldoMora", "ValorMora")
    out["Cargos Jurídico"]            = col("CargosJuridico", "Cargos Jurídico")
    out["Saldo Total"]                = col("SaldoTotal")

    # --- Teléfonos 1–4 solo celulares ---
    phone_cols = [c for c in ["Celular", "Fijo", "Fijo2", "Celular2", "PrimerTelefono"] if c in df.columns]

    def _clean_number(x) -> str:
        s = re.sub(r"\D", "", str(x or ""))
        return s

    def _is_mobile(num: str) -> bool:
        # Regla simple Colombia: celulares empiezan por 3 y tienen >= 9 dígitos
        return num.startswith("3") and len(num) >= 9

    def _phones_row(row):
        nums = []
        for c in phone_cols:
            raw = row.get(c, "")
            num = _clean_number(raw)
            if num and _is_mobile(num) and num not in nums:
                nums.append(num)
        while len(nums) < 4:
            nums.append("")
        return pd.Series(nums[:4], index=["Telefono 1", "Telefono 2", "Telefono 3", "Telefono 4"])

    if phone_cols:
        phones_df = df[phone_cols].apply(_phones_row, axis=1, result_type="expand")
        out = pd.concat([out, phones_df], axis=1)
    else:
        for k in ["Telefono 1", "Telefono 2", "Telefono 3", "Telefono 4"]:
            out[k] = ""

    # --- Datos del almacén ---
    out["Almacen"]           = col("Nombre", "NombreAlmacen")
    out["Municipio Almacen"] = col("MunicipioAlmacen", "Nombre.1")
    out["Direccion Almacen"] = col("DireccionAlmacen", "Direccion")
    out["Teléfono Almacén"]  = col("TelefonoAlmacen", "PrimerTelefono")
    out["Status"]            = col("CreditNumber", "Status")

    # --- Servicio fijo ---
    out["Servicio"] = "General"

    # --- Franja por días de mora (Parameters.FranjaMora) ---
    def _franja(dias):
        try:
            d = float(dias)
        except (TypeError, ValueError):
            return ""
        if d < 180:
            return "Franja_00"  # 00. < 180
        elif d < 360:
            return "Franja_01"  # 01. 180 a 359
        elif d < 500:
            return "Franja_02"  # 02. 360 a 499
        elif d < 2000:
            return "Franja_03"  # 03. 500 a 1999
        else:
            return "Franja_04"  # 04. >= 2000

    out["Franja"] = out["Días en Mora"].apply(_franja)

    # --- Campos adicionales ---
    out["IN_PRIORIDAD"]      = ""
    out["Tipo de Documento"] = col("Tipo Documento", "TipoDocumento")
    out["In_Serviceid"]      = 1

    # --- Orden de columnas según IN_SISTECREDITO ---
    desired_cols = [
        "Documento de identidad",
        "Nombre Completo",
        "Dirección Del Cliente",
        "Fecha Nacimiento",
        "Fecha Expedición",
        "Reportado a Centrales",
        "Crédito",
        "Fecha Creación del crédito",
        "Valor",
        "Número de cuotas vencidas",
        "Fecha de vencimiento",
        "Días en Mora",
        "Valor Mora",
        "Cargos Jurídico",
        "Saldo Total",
        "Telefono 1",
        "Telefono 2",
        "Telefono 3",
        "Telefono 4",
        "Almacen",
        "Municipio Almacen",
        "Direccion Almacen",
        "Teléfono Almacén",
        "Status",
        "Servicio",
        "Franja",
        "IN_PRIORIDAD",
        "Tipo de Documento",
        "In_Serviceid",
    ]

    ordered = [c for c in desired_cols if c in out.columns]
    rest    = [c for c in out.columns if c not in ordered]
    out = out[ordered + rest]

    return out


##Launcher

In [288]:
#SinergIA_Laucher
gr.close_all()
_app = build_sinergia_gui_parity_v7()
_port = 7860 + int(time.time()) % 200
_app.queue().launch(share=True, inline=False, debug=True, show_error=True, server_port=_port)

Closing server running on port: 7985
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5a6c7209e4dc1e95b3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:8035 <> https://5a6c7209e4dc1e95b3.gradio.live
